# 載入資料

##通用程式、參數

In [ ]:
kbars_csv="kbars.csv"
tick_log_csv='tick_log.csv'
trade_log_csv="trade_log.csv"
ss_log_csv="ss_log.csv"
key_price_log_csv="key_price_log.csv"


In [ ]:
def convert_5m_to_60m(df_5m):

    if not isinstance(df_5m.index, pd.DatetimeIndex):
        df_5m.index = pd.to_datetime(df_5m.index)

    df_5m = df_5m.sort_index()
    out = []

    for file_date, g in df_5m.groupby('file_date'):
        g = g.sort_index()

        t = g.index.time

        day_mask = (
            (t >= pd.to_datetime("08:45").time()) &
            (t <= pd.to_datetime("13:45").time())
        )

        night_mask = (
            (t >= pd.to_datetime("15:00").time()) |
            (t <= pd.to_datetime("05:00").time())
        )

        def r60(df, offset):
            r = df.resample(
                '60min',
                label='right',
                closed='right',
                offset=offset
            )
            k = pd.DataFrame({
                'open': r['open'].first(),
                'high': r['high'].max(),
                'low': r['low'].min(),
                'close': r['close'].last(),
            })
            if 'volume' in df.columns:
                k['volume'] = r['volume'].sum()
            k['file_date'] = file_date
            return k.dropna()

        if day_mask.any():
            out.append(r60(g[day_mask], offset='45min'))

        if night_mask.any():
            out.append(r60(g[night_mask], offset='0min'))

    if not out:
        return None

    return pd.concat(out).sort_index()

##讀取

In [ ]:
from google.colab import drive
import os

drive.mount('/content/drive')

drive_path = "/content/drive/My Drive/Colab Notebooks/DATA/"
folder_path = drive_path+"/CLEAN_TICK"



if os.path.exists(folder_path):
  file_list = os.listdir(folder_path)
  file_list.sort(reverse=True)
  print(file_list)
else:
  print(f"Folder not found: {folder_path}")

!pip install mplfinance



Mounted at /content/drive
['CLEAN_TICK_Daily_2025_12_31.parquet', 'CLEAN_TICK_Daily_2025_12_30.parquet', 'CLEAN_TICK_Daily_2025_12_29.parquet', 'CLEAN_TICK_Daily_2025_12_26.parquet', 'CLEAN_TICK_Daily_2025_12_24.parquet', 'CLEAN_TICK_Daily_2025_12_23.parquet', 'CLEAN_TICK_Daily_2025_12_22.parquet', 'CLEAN_TICK_Daily_2025_12_19.parquet', 'CLEAN_TICK_Daily_2025_12_18.parquet', 'CLEAN_TICK_Daily_2025_12_17.parquet', 'CLEAN_TICK_Daily_2025_12_16.parquet', 'CLEAN_TICK_Daily_2025_12_15.parquet', 'CLEAN_TICK_Daily_2025_12_12.parquet', 'CLEAN_TICK_Daily_2025_12_11.parquet', 'CLEAN_TICK_Daily_2025_12_10.parquet', 'CLEAN_TICK_Daily_2025_12_09.parquet', 'CLEAN_TICK_Daily_2025_12_08.parquet', 'CLEAN_TICK_Daily_2025_12_05.parquet', 'CLEAN_TICK_Daily_2025_12_04.parquet', 'CLEAN_TICK_Daily_2025_12_03.parquet', 'CLEAN_TICK_Daily_2025_12_02.parquet', 'CLEAN_TICK_Daily_2025_12_01.parquet', 'CLEAN_TICK_Daily_2025_11_28.parquet', 'CLEAN_TICK_Daily_2025_11_27.parquet', 'CLEAN_TICK_Daily_2025_11_26.parquet'

In [ ]:
import pandas as pd
import numpy as np
# 3. 讀取並處理每個 .rpt 檔案
def process_rpt_file(file_path,min):
    import os
    ext = os.path.splitext(file_path)[1].lower()

    if ext == '.csv' or ext == '.rpt' or ext == '.txt':
        ticks0 = pd.read_csv(file_path, encoding='big5', dtype=str)
    elif ext == '.parquet':
        ticks0 = pd.read_parquet(file_path)
        ticks0 = ticks0.astype(str)  # 轉成字串以保證欄位處理一致
        # print(ticks0)

    else:
        raise ValueError(f"不支援的檔案格式：{file_path}")


    merged_ticks = ticks0[['成交日期', '成交時間', '成交價格', '成交數量']].copy()
    merged_ticks['成交時間'] = merged_ticks['成交時間'].str.zfill(6).str.slice(0, 6)
    merged_ticks['成交日期時間'] = pd.to_datetime(
        merged_ticks['成交日期'] + ' ' + merged_ticks['成交時間'],
        format='%Y%m%d %H%M%S', errors='coerce'
    )
    merged_ticks = merged_ticks.dropna(subset=['成交日期時間'])
    merged_ticks.set_index('成交日期時間', inplace=True)

    key_date = os.path.splitext(os.path.basename(file_path))[0]

    # 清理
    merged_ticks['成交價格'] = pd.to_numeric(merged_ticks['成交價格'], errors='coerce')
    merged_ticks['成交數量'] = pd.to_numeric(merged_ticks['成交數量'], errors='coerce')
    merged_ticks = merged_ticks.dropna(subset=['成交價格', '成交數量'])
    merged_ticks['歸屬日期']=key_date
    # print(merged_ticks)
    km = merged_ticks['成交價格'].resample(min,label='right').ohlc()
    km['volume'] = merged_ticks['成交數量'].resample(min,label='right').sum()
    km = km.dropna(subset=['open'])
    km = km.reset_index()
    # print(km)



    ticks0["time"] = pd.to_datetime(ticks0["成交日期時間"])
    ticks0['price'] = pd.to_numeric(ticks0['成交價格'])
    ticks0['volume'] = pd.to_numeric(ticks0['成交數量'])
    # km = km.copy()
    km["start_time"] = pd.to_datetime(km["成交日期時間"])
    km['file_date']=key_date
    km[['open','high','low','close','volume']] = km[['open','high','low','close','volume']].apply(pd.to_numeric)
    km_=km.copy()
    km_.index=km_["start_time"]
    km_.index=pd.to_datetime(km_.index)
    km60 = convert_5m_to_60m(km_)
    km60["start_time"] = km60.index
    km60 = km60.reset_index(drop=True)
    # print(km.head())
    # print(km60.head(5))
    return ticks0,km,km60

##繪圖

In [ ]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from typing import Optional, List, Tuple


# def convert_5m_to_60m(kbar_5m_df):
#     """
#     將5分鐘K線轉換為60分鐘K線

#     Parameters:
#     -----------
#     kbar_5m_df : pd.DataFrame
#         5分鐘K線資料，index為start_time

#     Returns:
#     --------
#     kbar_60m_df : pd.DataFrame
#         60分鐘K線資料
#     """
#     if kbar_5m_df is None or len(kbar_5m_df) == 0:
#         return None

#     # 確保必要欄位存在
#     required_cols = ['open', 'high', 'low', 'close']
#     if not all(col in kbar_5m_df.columns for col in required_cols):
#         print(f"⚠️ 缺少必要欄位，無法轉換")
#         return None

#     try:
#         # 複製資料避免修改原始資料
#         df = kbar_5m_df.copy()

#         # 確保索引是DatetimeIndex
#         if not isinstance(df.index, pd.DatetimeIndex):
#             df.index = pd.to_datetime(df.index)

#         # 重採樣為60分鐘
#         kbar_60m = pd.DataFrame()
#         kbar_60m['open'] = df['open'].resample('60min', label='right', closed='right').first()
#         kbar_60m['high'] = df['high'].resample('60min', label='right', closed='right').max()
#         kbar_60m['low'] = df['low'].resample('60min', label='right', closed='right').min()
#         kbar_60m['close'] = df['close'].resample('60min', label='right', closed='right').last()

#         # 如果有成交量欄位也一併轉換
#         if 'volume' in df.columns:
#             kbar_60m['volume'] = df['volume'].resample('60min', label='right', closed='right').sum()

#         # 如果有file_date欄位
#         if 'file_date' in df.columns:
#             kbar_60m['file_date'] = df['file_date'].resample('60min', label='right', closed='right').first()

#         # 移除空值（沒有資料的時間段）
#         kbar_60m = kbar_60m.dropna(subset=['open', 'high', 'low', 'close'])

#         return kbar_60m

#     except Exception as e:
#         print(f"⚠️ 轉換60分K線時發生錯誤: {e}")
#         return None

# def convert_5m_to_60m_tw_tx(
#     df_5m: pd.DataFrame,
# ):
#     """
#     台指專用 5m → 60m
#     日盤：08:45 開，60K 收在 HH:45
#     夜盤：15:00 開，60K 收在 HH:00（隔日 05:00）
#     """

#     df = df_5m.copy()

#     if not isinstance(df.index, pd.DatetimeIndex):
#         df.index = pd.to_datetime(df.index)

#     df = df.sort_index()

#     # ===== 定義時段 =====
#     t = df.index.time

#     day_mask = (t >= pd.to_datetime("08:45").time()) & (t <= pd.to_datetime("13:45").time())
#     night_mask = (t >= pd.to_datetime("15:00").time()) | (t <= pd.to_datetime("05:00").time())

#     day_df = df[day_mask]
#     night_df = df[night_mask]

#     def resample_60(df, offset=None):
#         r = df.resample(
#             '60min',
#             label='right',
#             closed='right',
#             offset=offset
#         )
#         out = pd.DataFrame({
#             'open': r['open'].first(),
#             'high': r['high'].max(),
#             'low': r['low'].min(),
#             'close': r['close'].last(),
#         })
#         if 'volume' in df.columns:
#             out['volume'] = r['volume'].sum()
#         return out.dropna()

#     # 日盤 → 收在 HH:45
#     k60_day = resample_60(day_df, offset='45min')

#     # 夜盤 → 整點收
#     k60_night = resample_60(night_df, offset='0min')

#     # 合併
#     k60 = pd.concat([k60_day, k60_night]).sort_index()

#     return k60

def plot_kbar_analysis(
    start_date: str,
    end_date: str,
    kbar_csv_path: str=kbars_csv,
    trade_csv_path: str=trade_log_csv,
    interval_minutes: int = 5,
    ss_log_path: Optional[str] = ss_log_csv,
    key_price_log_path: Optional[str] = key_price_log_csv,
    kbar_60m_csv_path: Optional[str] = None,
    output_html: Optional[str] = None,
    show_ma: bool = True,
    show_bb: bool = False,
    show_ema: bool = False,
    show_rf_lines: bool = False,
    show_kd: bool = False,
    show_rf_signals: bool = False,
    show_60m_overlay: bool = True,
):
    """
    繪製 K 線圖表，包含交易點位、技術指標、關鍵價格等，自動偵測可用欄位
    新增：可疊加60分鐘K線，若無60分K線資料則自動從5分K轉換

    Parameters:
    -----------
    kbar_csv_path : str
        K線資料 CSV 檔案路徑
    trade_csv_path : str
        交易紀錄 CSV 檔案路徑
    start_date : str
        起始日期，格式: "YYYY-MM-DD"
    end_date : str
        結束日期，格式: "YYYY-MM-DD"
    interval_minutes : int
        K線時間間隔（分鐘），預設 5
    key_price_log_path : str, optional
        關鍵價格日誌 CSV 檔案路徑
    kbar_60m_csv_path : str, optional
        60分鐘K線資料 CSV 檔案路徑（若無則自動從5分K轉換）
    show_60m_overlay : bool
        是否顯示60分鐘K線疊加，預設 True
    output_html : str, optional
        輸出 HTML 檔案路徑，若為 None 則直接顯示

    Returns:
    --------
    fig : plotly.graph_objects.Figure
        繪製的圖表物件
    """

    # ==================== 讀取資料 ====================
    if end_date is None:
        end_date = start_date
    print("📊 讀取資料中...")


    kbar_df = None
    if kbar_csv_path:
        try:
            kbar_df = pd.read_csv(kbar_csv_path, parse_dates=["start_time"])
            kbar_df = kbar_df.set_index("start_time").sort_index()
            print(kbar_df.columns)
            print(kbar_df.head())
        except Exception as e:
            print(f"⚠️ 無法讀取 kbar_csv {e}")
    print(kbar_df)
    # 讀取60分鐘K線
    kbar_60m_df = None
    if show_60m_overlay:
        if kbar_60m_csv_path:
            try:
                kbar_60m_df = pd.read_csv(kbar_60m_csv_path, parse_dates=["start_time"])
                kbar_60m_df = kbar_60m_df.set_index("start_time").sort_index()
                print(f"✅ 已讀取 60分K線，共 {len(kbar_60m_df)} 筆")
            except Exception as e:
                print(f"⚠️ 無法讀取 60分K線: {e}")
                print(f"📊 將自動從5分K線轉換為60分K線...")
                kbar_60m_df = None

        # 如果沒有60分K線資料，從5分K線轉換
        if kbar_60m_df is None and kbar_df is not None:
            print(f"📊 正在從5分K線轉換為60分K線...")
            kbar_60m_df = convert_5m_to_60m(kbar_df)
            if kbar_60m_df is not None:
                print(f"✅ 轉換完成，共 {len(kbar_60m_df)} 筆60分K線")

    # 讀取交易紀錄
    trade_df = None
    if trade_csv_path:
        try:
            trade_df = pd.read_csv(trade_csv_path, parse_dates=["tick_time"])
            print(trade_df.columns)
            print(trade_df.head())
        except Exception as e:
            print(f"⚠️ 無法讀取 trade_csv: {e}")


    # 讀取關鍵價格日誌
    ss_df = None
    if ss_log_path:
        try:
            ss_df = pd.read_csv(ss_log_path, parse_dates=["start_time"])
            print(f"✅ 已讀取 ss_log，共 {len(ss_df)} 筆")
            print(ss_df.columns)
            print(ss_df.head())
        except Exception as e:
            print(f"⚠️ 無法讀取 ss_log: {e}")

    key_price_df = None
    if key_price_log_path:
        try:
            key_price_df = pd.read_csv(key_price_log_path, parse_dates=["start_time"])
            print(f"✅ 已讀取 key_price_log，共 {len(key_price_df)} 筆")
        except Exception as e:
            print(f"⚠️ 無法讀取 key_price_log: {e}")

    # 篩選日期範圍
    if kbar_df is not None:
        kbar_df = kbar_df.loc[
                (kbar_df['file_date'] >= start_date) &
                (kbar_df['file_date'] <= end_date)
                 ]
        print(f"✅ 已讀取 kbar_csv，共 {len(kbar_df)} 筆")

    if kbar_60m_df is not None:
        if 'file_date' in kbar_60m_df.columns:
            kbar_60m_df = kbar_60m_df.loc[
                    (kbar_60m_df['file_date'] >= start_date) &
                    (kbar_60m_df['file_date'] <= end_date)
                     ]
        print(f"✅ 篩選後 60分K線筆數: {len(kbar_60m_df)}")

    if trade_df is not None:
        trade_df = trade_df.loc[
                  (trade_df["file_date"] >= start_date) &
                  (trade_df["file_date"] <= end_date)
                  ]
        print(f"✅ 已讀取 trade_csv，共 {len(trade_df)} 筆")
    if ss_df is not None:
        ss_df = ss_df.loc[
            (ss_df["file_date"] >= start_date) &
            (ss_df["file_date"] <= end_date)
        ]
        print(f"✅ 篩選後 ss_log 筆數: {len(ss_df)}")
    if key_price_df is not None:
        key_price_df = key_price_df.loc[
            (key_price_df["file_date"] >= start_date) &
            (key_price_df["file_date"] <= end_date)
        ]
        print(f"✅ 篩選後 key_price_log 筆數: {len(key_price_df)}")

    # ==================== 偵測可用欄位 ====================
    available_fields = {
        # K線基本欄位
        'ohlc': all(col in kbar_df.columns for col in ['open', 'high', 'low', 'close']),
        'ma5': 'ma5' in kbar_df.columns,
        'ma10': 'ma10' in kbar_df.columns,
        'ma20': 'ma20' in kbar_df.columns,
        'bb_upper': 'bb_upper' in kbar_df.columns,
        'bb_lower': 'bb_lower' in kbar_df.columns,
        'ema12': 'ema12' in kbar_df.columns,
        'ema26': 'ema26' in kbar_df.columns,

        # KD 指標
        'K': 'K' in kbar_df.columns,
        'D': 'D' in kbar_df.columns,
        'RSI': 'RSI' in kbar_df.columns,

        # Range Filter
        'rf_filter': 'rf_filter' in kbar_df.columns,
        'rf_high_band': 'rf_high_band' in kbar_df.columns,
        'rf_low_band': 'rf_low_band' in kbar_df.columns,
        'rf_long_signal': 'rf_long_signal' in kbar_df.columns,
        'rf_short_signal': 'rf_short_signal' in kbar_df.columns,
        'rf_upward': 'rf_upward' in kbar_df.columns,
        'rf_downward': 'rf_downward' in kbar_df.columns,
    }

    # 判斷需要幾個子圖
    has_kd_subplot = show_kd and (available_fields['K'] or available_fields['D'] or available_fields['RSI'])
    has_rf_subplot = show_rf_signals and (available_fields['rf_upward'] or available_fields['rf_downward'] or
                      available_fields['rf_long_signal'] or available_fields['rf_short_signal'])

    print(f"\n🔍 欄位偵測結果:")
    print(f"   - K線圖: {'✓' if available_fields['ohlc'] else '✗'}")
    print(f"   - 60分K線疊加: {'✓' if (kbar_60m_df is not None and show_60m_overlay) else '✗'}")
    print(f"   - MA5: {'✓' if available_fields['ma5'] else '✗'}")
    print(f"   - MA10: {'✓' if available_fields['ma10'] else '✗'}")
    print(f"   - MA20: {'✓' if available_fields['ma20'] else '✗'}")
    print(f"   - KD指標子圖: {'✓' if has_kd_subplot else '✗'}")
    print(f"   - Range Filter子圖: {'✓' if has_rf_subplot else '✗'}")

    # ==================== 對齊交易時間 ====================
    def align_to_bar(tick_times, kbar_index, interval_minutes=5, label="right"):
        aligned_times = []
        if not isinstance(kbar_index, pd.DatetimeIndex):
            kbar_index = pd.to_datetime(kbar_index)

        for t in pd.to_datetime(tick_times):
            minutes_since_midnight = t.hour * 60 + t.minute
            if label == "right":
                next_minutes = ((minutes_since_midnight // interval_minutes) + 1) * interval_minutes
                bar_time_candidate = t.normalize() + pd.Timedelta(minutes=next_minutes)
            else:
                prev_minutes = (minutes_since_midnight // interval_minutes) * interval_minutes
                bar_time_candidate = t.normalize() + pd.Timedelta(minutes=prev_minutes)

            pos = kbar_index.searchsorted(bar_time_candidate, side="right") - 1
            if pos < 0:
                pos = 0
            aligned_times.append(kbar_index[pos])

        return aligned_times

    # ==================== 處理交易資料 ====================
    if trade_df is not None:
        trade_df["bar_time"] = align_to_bar(trade_df["tick_time"], kbar_df.index, interval_minutes, label="right")

        # 修正交易配對
        fixed_trades = []
        for _, row in trade_df.iterrows():
            action = str(row.get("action", "")).upper() if pd.notna(row.get("action", None)) else ""
            signal = 1 if action == "BUY" else -1 if action == "SELL" else 0
            if signal != 0 or action.startswith("TAKE_PROFIT") or action in ["STOP_LOSS", "CHANGE_TREND"]:
                fixed_trades.append(row.to_dict())

        trade_df = pd.DataFrame(fixed_trades)

        if len(trade_df) > 0:
            trade_df["tick_time"] = pd.to_datetime(trade_df["tick_time"], errors="coerce")
            trade_df["bar_time"] = pd.to_datetime(trade_df["bar_time"], errors="coerce")

            # 分類進場與出場
            entry_trades = trade_df[trade_df["action"].isin(["BUY", "SELL"])].copy()
            exit_trades = trade_df[
                trade_df["action"].astype(str).str.startswith("TAKE_PROFIT") |
                trade_df["action"].isin(["STOP_LOSS", "CHANGE_TREND"])
            ].copy()

            # 解析價格
            def resolve_price(df, preferred_cols):
                for col in preferred_cols:
                    if col in df.columns:
                        return df[col]
                return pd.Series([pd.NA] * len(df), index=df.index)

            if len(entry_trades) > 0:
                entry_trades["bar_price"] = resolve_price(entry_trades, ["price", "bar_price", "close"])
                entry_trades = entry_trades.dropna(subset=["bar_price", "bar_time"])

            if len(exit_trades) > 0:
                exit_trades["bar_price"] = resolve_price(exit_trades, ["exit_price", "price", "bar_price", "close"])
                exit_trades = exit_trades.dropna(subset=["bar_price", "bar_time"])
        else:
            entry_trades = pd.DataFrame()
            exit_trades = pd.DataFrame()
    else:
        entry_trades = pd.DataFrame()
        exit_trades = pd.DataFrame()

    # ==================== 偵測 KD 交叉 ====================
    golden_times, golden_prices = [], []
    dead_times, dead_prices = [], []

    if show_kd and available_fields['K'] and available_fields['D']:
        for i in range(1, len(kbar_df)):
            prev_bar = kbar_df.iloc[i - 1]
            bar = kbar_df.iloc[i]
            bar_start = kbar_df.index[i]

            if pd.notna(prev_bar["K"]) and pd.notna(prev_bar["D"]) and pd.notna(bar["K"]) and pd.notna(bar["D"]):
                if prev_bar["K"] < prev_bar["D"] and bar["K"] > bar["D"]:
                    golden_times.append(bar_start)
                    golden_prices.append(bar["K"])
                if prev_bar["K"] > prev_bar["D"] and bar["K"] < bar["D"]:
                    dead_times.append(bar_start)
                    dead_prices.append(bar["K"])

    # ==================== Range Filter 信號 ====================
    rf_long_times, rf_long_prices = [], []
    rf_short_times, rf_short_prices = [], []

    if show_rf_signals and available_fields['rf_long_signal']:
        mask_long = kbar_df["rf_long_signal"].astype(bool)
        rf_long_times = kbar_df.index[mask_long].tolist()
        rf_long_prices = kbar_df["close"][mask_long].tolist() if "close" in kbar_df.columns else [None] * mask_long.sum()

    if show_rf_signals and available_fields['rf_short_signal']:
        mask_short = kbar_df["rf_short_signal"].astype(bool)
        rf_short_times = kbar_df.index[mask_short].tolist()
        rf_short_prices = kbar_df["close"][mask_short].tolist() if "close" in kbar_df.columns else [None] * mask_short.sum()

    # ==================== 建立子圖 ====================
    subplot_count = 1 + (1 if has_kd_subplot else 0) + (1 if has_rf_subplot else 0)

    if subplot_count == 1:
        row_heights = [1.0]
        subplot_titles = ["KBar with Entry/Exit Trades"]
    elif subplot_count == 2:
        row_heights = [0.65, 0.35]
        if has_kd_subplot:
            subplot_titles = ["KBar with Entry/Exit Trades", "KD Indicator"]
        else:
            subplot_titles = ["KBar with Entry/Exit Trades", "Range Filter Signals"]
    else:
        row_heights = [0.5, 0.25, 0.25]
        subplot_titles = ["KBar with Entry/Exit Trades", "KD Indicator", "Range Filter Signals"]

    fig = make_subplots(
        rows=subplot_count,
        cols=1,
        shared_xaxes=True,
        row_heights=row_heights,
        vertical_spacing=0.05,
        subplot_titles=subplot_titles
    )

    # ==================== 繪製主圖 (K線) ====================
    current_row = 1

    # 先繪製60分K線（在背景層）
    if kbar_60m_df is not None and show_60m_overlay and len(kbar_60m_df) > 0:
        if all(col in kbar_60m_df.columns for col in ['open', 'high', 'low', 'close']):
            fig.add_trace(
                go.Candlestick(
                    x=kbar_60m_df.index,
                    open=kbar_60m_df["open"],
                    high=kbar_60m_df["high"],
                    low=kbar_60m_df["low"],
                    close=kbar_60m_df["close"],
                    name="60分K",
                    increasing=dict(line=dict(color='rgba(100,180,100,0.5)', width=2), fillcolor='rgba(100,180,100,0.3)'),
                    decreasing=dict(line=dict(color='rgba(180,100,100,0.5)', width=2), fillcolor='rgba(180,100,100,0.3)'),
                    opacity=0.9,
                    hovertext=[f"60分K<br>O: {int(o)}<br>H: {int(h)}<br>L: {int(l)}<br>C: {int(c)}"
                              for o, h, l, c in zip(kbar_60m_df["open"], kbar_60m_df["high"],
                                                    kbar_60m_df["low"], kbar_60m_df["close"])],
                    hoverinfo="text+x"
                ),
                row=current_row,
                col=1
            )

    # 再繪製5分K線（在前景層）
    if available_fields['ohlc']:
        fig.add_trace(
            go.Candlestick(
                x=kbar_df.index,
                open=kbar_df["open"],
                high=kbar_df["high"],
                low=kbar_df["low"],
                close=kbar_df["close"],
                name="5分K",
                hovertext=[f"5分K<br>O: {int(o)}<br>H: {int(h)}<br>L: {int(l)}<br>C: {int(c)}"
                          for o, h, l, c in zip(kbar_df["open"], kbar_df["high"],
                                                kbar_df["low"], kbar_df["close"])],
                hoverinfo="text+x"
            ),
            row=current_row,
            col=1
        )


    # 技術指標線
    if show_ma and available_fields['ma5']:
        fig.add_trace(go.Scatter(x=kbar_df.index, y=kbar_df["ma5"], mode="lines", name="MA5",
                                line=dict(width=1.5, color="blue")), row=current_row, col=1)

    if show_ma and available_fields['ma10']:
        fig.add_trace(go.Scatter(x=kbar_df.index, y=kbar_df["ma10"], mode="lines", name="MA10",
                                line=dict(width=1.5, color="green")), row=current_row, col=1)

    if show_ma and available_fields['ma20']:
        fig.add_trace(go.Scatter(x=kbar_df.index, y=kbar_df["ma20"], mode="lines", name="MA20",
                                line=dict(width=1.5, color="orange")), row=current_row, col=1)

    if show_bb and available_fields['bb_upper']:
        fig.add_trace(go.Scatter(x=kbar_df.index, y=kbar_df["bb_upper"], mode="lines", name="BB Upper",
                                line=dict(width=1, color="gray")), row=current_row, col=1)

    if show_bb and available_fields['bb_lower']:
        fig.add_trace(go.Scatter(x=kbar_df.index, y=kbar_df["bb_lower"], mode="lines", name="BB Lower",
                                line=dict(width=1, color="gray")), row=current_row, col=1)

    if show_ema and available_fields['ema12']:
        fig.add_trace(go.Scatter(x=kbar_df.index, y=kbar_df["ema12"], mode="lines", name="EMA12",
                                line=dict(color="yellow", width=1, dash="dot")), row=current_row, col=1)

    if show_ema and available_fields['ema26']:
        fig.add_trace(go.Scatter(x=kbar_df.index, y=kbar_df["ema26"], mode="lines", name="EMA26",
                                line=dict(color="cyan", width=1, dash="dot")), row=current_row, col=1)

    # Range Filter 線條
    if show_rf_lines and available_fields['rf_filter']:
        fig.add_trace(go.Scatter(x=kbar_df.index, y=kbar_df["rf_filter"], mode="lines", name="RF Filter",
                                line=dict(color="purple", width=2)), row=current_row, col=1)

    if show_rf_lines and available_fields['rf_high_band']:
        fig.add_trace(go.Scatter(x=kbar_df.index, y=kbar_df["rf_high_band"], mode="lines", name="RF High Band",
                                showlegend=False, line=dict(color="rgba(128,128,128,0.5)")), row=current_row, col=1)

    if show_rf_lines and available_fields['rf_low_band']:
        fig.add_trace(go.Scatter(x=kbar_df.index, y=kbar_df["rf_low_band"], mode="lines", fill="tonexty",
                                fillcolor="rgba(128,128,128,0.3)", name="RF Low Band", showlegend=False,
                                line=dict(color="rgba(128,128,128,0.5)")), row=current_row, col=1)

    # ==================== 繪製關鍵價格點 ====================
    if ss_df is not None and len(ss_df) > 0:
        # 繪製 stress
        if 'stress' in ss_df.columns:
            stress_df = ss_df.dropna(subset=['stress'])
            if len(stress_df) > 0:
                fig.add_trace(go.Scatter(
                    x=stress_df["start_time"],
                    y=stress_df["stress"],
                    mode="markers",
                    marker=dict(symbol="square", color="cyan", size=5, line=dict(color="darkblue", width=1)),
                    name="stress"
                ), row=current_row, col=1)
        # 繪製 support
        if 'support' in ss_df.columns:
            support_df = ss_df.dropna(subset=['support'])
            if len(support_df) > 0:
                fig.add_trace(go.Scatter(
                    x=support_df["start_time"],
                    y=support_df["support"],
                    mode="markers",
                    marker=dict(symbol="diamond", color="magenta", size=5, line=dict(color="darkmagenta", width=1)),
                    name="support"
                ), row=current_row, col=1)



    if key_price_df is not None and len(key_price_df) > 0:
        # 繪製 enter_price
        if 'enter_price' in key_price_df.columns:
            enter_df = key_price_df.dropna(subset=['enter_price'])
            if len(enter_df) > 0:
                fig.add_trace(go.Scatter(
                    x=enter_df["start_time"],
                    y=enter_df["enter_price"],
                    mode="markers",
                    marker=dict(symbol="square", color="cyan", size=10, line=dict(color="darkblue", width=1)),
                    name="Enter Price"
                ), row=current_row, col=1)

        # 繪製 reset_price
        if 'reset_price' in key_price_df.columns:
            reset_df = key_price_df.dropna(subset=['reset_price'])
            if len(reset_df) > 0:
                fig.add_trace(go.Scatter(
                    x=reset_df["start_time"],
                    y=reset_df["reset_price"],
                    mode="markers",
                    marker=dict(symbol="diamond", color="magenta", size=10, line=dict(color="darkmagenta", width=1)),
                    name="Reset Price"
                ), row=current_row, col=1)

        if 'exit_price' in key_price_df.columns:
            exit_df = key_price_df.dropna(subset=['exit_price'])
            if len(exit_df) > 0:
                fig.add_trace(go.Scatter(
                    x=exit_df["start_time"],
                    y=exit_df["exit_price"],
                    mode="markers",
                    marker=dict(symbol="diamond", color="orange", size=10, line=dict(color="orange", width=1)),
                    name="Exit Price"
                ), row=current_row, col=1)
    # Range Filter 信號點
    if rf_long_times:
        fig.add_trace(go.Scatter(x=rf_long_times, y=rf_long_prices, mode="markers",
                                marker=dict(symbol="triangle-up", color="darkgreen", size=10),
                                name="RF Long Signal"), row=current_row, col=1)

    if rf_short_times:
        fig.add_trace(go.Scatter(x=rf_short_times, y=rf_short_prices, mode="markers",
                                marker=dict(symbol="triangle-down", color="darkred", size=10),
                                name="RF Short Signal"), row=current_row, col=1)

    # 進場點
    if len(entry_trades) > 0:
        buy_trades = entry_trades[entry_trades["action"] == "BUY"]
        sell_trades = entry_trades[entry_trades["action"] == "SELL"]

        if not buy_trades.empty:
            fig.add_trace(go.Scatter(
                x=buy_trades["bar_time"],
                y=buy_trades["bar_price"],
                mode="markers+text",
                marker=dict(symbol="triangle-up", color="green", size=12),
                text=[f"Buy {p:.2f}" for p in buy_trades["bar_price"].astype(float)],
                textposition="top center",
                name="Buy Entry"
            ), row=current_row, col=1)

        if not sell_trades.empty:
            fig.add_trace(go.Scatter(
                x=sell_trades["bar_time"],
                y=sell_trades["bar_price"],
                mode="markers+text",
                marker=dict(symbol="triangle-down", color="red", size=12),
                text=[f"Sell {p:.2f}" for p in sell_trades["bar_price"].astype(float)],
                textposition="bottom center",
                name="Sell Entry"
            ), row=current_row, col=1)

    # 出場點
    if len(exit_trades) > 0:
        take_profit_trades = exit_trades[exit_trades["action"].astype(str).str.startswith("TAKE_PROFIT")]
        stop_loss_trades = exit_trades[exit_trades["action"] == "STOP_LOSS"]
        change_trend_trades = exit_trades[exit_trades["action"] == "CHANGE_TREND"]

        if not take_profit_trades.empty:
            fig.add_trace(go.Scatter(
                x=take_profit_trades["bar_time"],
                y=take_profit_trades["bar_price"],
                mode="markers+text",
                marker=dict(symbol="circle", color="blue", size=10),
                text=[f"TP {p:.2f}" for p in take_profit_trades["bar_price"].astype(float)],
                textposition="top center",
                name="Take Profit"
            ), row=current_row, col=1)

        if not stop_loss_trades.empty:
            fig.add_trace(go.Scatter(
                x=stop_loss_trades["bar_time"],
                y=stop_loss_trades["bar_price"],
                mode="markers+text",
                marker=dict(symbol="x", color="purple", size=10),
                text=[f"SL {p:.2f}" for p in stop_loss_trades["bar_price"].astype(float)],
                textposition="bottom center",
                name="Stop Loss"
            ), row=current_row, col=1)

        if not change_trend_trades.empty:
            fig.add_trace(go.Scatter(
                x=change_trend_trades["bar_time"],
                y=change_trend_trades["bar_price"],
                mode="markers+text",
                marker=dict(symbol="diamond", color="orange", size=10),
                text=[f"CT {p:.2f}" for p in change_trend_trades["bar_price"].astype(float)],
                textposition="top center",
                name="Trend Change"
            ), row=current_row, col=1)

    # ==================== 繪製 KD 指標子圖 ====================
    if has_kd_subplot:
        current_row += 1

        if available_fields['K']:
            fig.add_trace(go.Scatter(x=kbar_df.index, y=kbar_df["K"], mode="lines", name="K",
                                    line=dict(color="blue")), row=current_row, col=1)

        if available_fields['D']:
            fig.add_trace(go.Scatter(x=kbar_df.index, y=kbar_df["D"], mode="lines", name="D",
                                    line=dict(color="orange")), row=current_row, col=1)

        if available_fields['RSI']:
            fig.add_trace(go.Scatter(x=kbar_df.index, y=kbar_df["RSI"], mode="lines", name="RSI",
                                    line=dict(color="purple", dash="dash")), row=current_row, col=1)

        # KD 交叉點
        if golden_times:
            fig.add_trace(go.Scatter(x=golden_times, y=golden_prices, mode="markers",
                                    marker=dict(symbol="star", color="gold", size=12),
                                    name="KD Golden Cross"), row=current_row, col=1)

        if dead_times:
            fig.add_trace(go.Scatter(x=dead_times, y=dead_prices, mode="markers",
                                    marker=dict(symbol="star", color="black", size=12),
                                    name="KD Dead Cross"), row=current_row, col=1)

        fig.update_yaxes(title_text="KD Value", row=current_row, col=1)

    # ==================== 繪製 Range Filter 子圖 ====================
    if has_rf_subplot:
        current_row += 1

        # RF 方向指標
        if available_fields['rf_upward']:
            rf_upward_values = kbar_df["rf_upward"].fillna(0).apply(lambda x: 1 if x == 1 else None).tolist()
            fig.add_trace(go.Scatter(x=kbar_df.index, y=rf_upward_values, mode="markers",
                                    marker=dict(color="green", size=8, symbol="triangle-up"),
                                    name="RF Upward"), row=current_row, col=1)

        if available_fields['rf_downward']:
            rf_downward_values = kbar_df["rf_downward"].fillna(0).apply(lambda x: -1 if x == 1 else None).tolist()
            fig.add_trace(go.Scatter(x=kbar_df.index, y=rf_downward_values, mode="markers",
                                    marker=dict(color="red", size=8, symbol="triangle-down"),
                                    name="RF Downward"), row=current_row, col=1)

        # RF 信號標記
        if available_fields['rf_long_signal']:
            rf_long_signal_values = [0.5 if v else None for v in kbar_df["rf_long_signal"].fillna(False)]
            fig.add_trace(go.Scatter(x=kbar_df.index, y=rf_long_signal_values, mode="markers",
                                    marker=dict(color="darkgreen", size=10, symbol="circle"),
                                    name="RF Long Signal"), row=current_row, col=1)

        if available_fields['rf_short_signal']:
            rf_short_signal_values = [-0.5 if v else None for v in kbar_df["rf_short_signal"].fillna(False)]
            fig.add_trace(go.Scatter(x=kbar_df.index, y=rf_short_signal_values, mode="markers",
                                    marker=dict(color="darkred", size=10, symbol="circle"),
                                    name="RF Short Signal"), row=current_row, col=1)

        fig.update_yaxes(title_text="RF Signals", range=[-1.5, 1.5], row=current_row, col=1)

    # ==================== 圖表設定 ====================
    fig.update_layout(
        title=f"KBar Analysis ({start_date} to {end_date})",
        xaxis_title="Time" if subplot_count == 1 else None,
        yaxis_title="Price",
        xaxis_rangeslider_visible=False,
        width=1400,
        height=300 + 350 * subplot_count,
        showlegend=True,
        hovermode='x unified',
        xaxis=dict(
            hoverformat='%Y%m%d<br>%H:%M'
        ),
        yaxis=dict(
            separatethousands=False,
            hoverformat=".f",
            tickformat=".f"
        ),
    )

    # 隱藏所有子圖的 X 軸日期時間顯示在 hover 上
    for i in range(1, subplot_count + 1):
        fig.update_xaxes(showspikes=True, spikemode='across', spikesnap='cursor',
                        hoverformat='%Y%m%d<br>%H:%M', row=i, col=1)
        fig.update_yaxes(showspikes=True, spikemode='across', row=i, col=1)

    # 設定最下方子圖的 X 軸標題
    fig.update_xaxes(title_text="Time", row=subplot_count, col=1)

    # ==================== 輸出 ====================
    if output_html:
        fig.write_html(output_html)
        print(f"\n✅ 已輸出圖表至: {output_html}")
    else:
        fig.show()

    return fig

##繪圖N

In [ ]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from typing import Optional, List, Tuple


# def convert_5m_to_60m(kbar_5m_df):
#     """
#     將5分鐘K線轉換為60分鐘K線

#     Parameters:
#     -----------
#     kbar_5m_df : pd.DataFrame
#         5分鐘K線資料，index為start_time

#     Returns:
#     --------
#     kbar_60m_df : pd.DataFrame
#         60分鐘K線資料
#     """
#     if kbar_5m_df is None or len(kbar_5m_df) == 0:
#         return None

#     # 確保必要欄位存在
#     required_cols = ['open', 'high', 'low', 'close']
#     if not all(col in kbar_5m_df.columns for col in required_cols):
#         print(f"⚠️ 缺少必要欄位，無法轉換")
#         return None

#     try:
#         # 複製資料避免修改原始資料
#         df = kbar_5m_df.copy()

#         # 確保索引是DatetimeIndex
#         if not isinstance(df.index, pd.DatetimeIndex):
#             df.index = pd.to_datetime(df.index)

#         # 重採樣為60分鐘
#         kbar_60m = pd.DataFrame()
#         kbar_60m['open'] = df['open'].resample('60min', label='right', closed='right').first()
#         kbar_60m['high'] = df['high'].resample('60min', label='right', closed='right').max()
#         kbar_60m['low'] = df['low'].resample('60min', label='right', closed='right').min()
#         kbar_60m['close'] = df['close'].resample('60min', label='right', closed='right').last()

#         # 如果有成交量欄位也一併轉換
#         if 'volume' in df.columns:
#             kbar_60m['volume'] = df['volume'].resample('60min', label='right', closed='right').sum()

#         # 如果有file_date欄位
#         if 'file_date' in df.columns:
#             kbar_60m['file_date'] = df['file_date'].resample('60min', label='right', closed='right').first()

#         # 移除空值（沒有資料的時間段）
#         kbar_60m = kbar_60m.dropna(subset=['open', 'high', 'low', 'close'])

#         return kbar_60m

#     except Exception as e:
#         print(f"⚠️ 轉換60分K線時發生錯誤: {e}")
#         return None

# def convert_5m_to_60m_tw_tx(
#     df_5m: pd.DataFrame,
# ):
#     """
#     台指專用 5m → 60m
#     日盤：08:45 開，60K 收在 HH:45
#     夜盤：15:00 開，60K 收在 HH:00（隔日 05:00）
#     """

#     df = df_5m.copy()

#     if not isinstance(df.index, pd.DatetimeIndex):
#         df.index = pd.to_datetime(df.index)

#     df = df.sort_index()

#     # ===== 定義時段 =====
#     t = df.index.time

#     day_mask = (t >= pd.to_datetime("08:45").time()) & (t <= pd.to_datetime("13:45").time())
#     night_mask = (t >= pd.to_datetime("15:00").time()) | (t <= pd.to_datetime("05:00").time())

#     day_df = df[day_mask]
#     night_df = df[night_mask]

#     def resample_60(df, offset=None):
#         r = df.resample(
#             '60min',
#             label='right',
#             closed='right',
#             offset=offset
#         )
#         out = pd.DataFrame({
#             'open': r['open'].first(),
#             'high': r['high'].max(),
#             'low': r['low'].min(),
#             'close': r['close'].last(),
#         })
#         if 'volume' in df.columns:
#             out['volume'] = r['volume'].sum()
#         return out.dropna()

#     # 日盤 → 收在 HH:45
#     k60_day = resample_60(day_df, offset='45min')

#     # 夜盤 → 整點收
#     k60_night = resample_60(night_df, offset='0min')

#     # 合併
#     k60 = pd.concat([k60_day, k60_night]).sort_index()

#     return k60

def plot_kbar_analysis(
    start_date: str,
    end_date: str,
    kbar_csv_path: str=kbars_csv,
    trade_csv_path: str=trade_log_csv,
    interval_minutes: int = 5,
    ss_log_path: Optional[str] = ss_log_csv,
    key_price_log_path: Optional[str] = key_price_log_csv,
    kbar_60m_csv_path: Optional[str] = None,
    output_html: Optional[str] = None,
    show_ma: bool = True,
    show_bb: bool = False,
    show_ema: bool = False,
    show_rf_lines: bool = False,
    show_kd: bool = False,
    show_rf_signals: bool = False,
    show_60m_overlay: bool = True,
):
    """
    繪製 K 線圖表，包含交易點位、技術指標、關鍵價格等，自動偵測可用欄位
    新增：可疊加60分鐘K線，若無60分K線資料則自動從5分K轉換

    Parameters:
    -----------
    kbar_csv_path : str
        K線資料 CSV 檔案路徑
    trade_csv_path : str
        交易紀錄 CSV 檔案路徑
    start_date : str
        起始日期，格式: "YYYY-MM-DD"
    end_date : str
        結束日期，格式: "YYYY-MM-DD"
    interval_minutes : int
        K線時間間隔（分鐘），預設 5
    key_price_log_path : str, optional
        關鍵價格日誌 CSV 檔案路徑
    kbar_60m_csv_path : str, optional
        60分鐘K線資料 CSV 檔案路徑（若無則自動從5分K轉換）
    show_60m_overlay : bool
        是否顯示60分鐘K線疊加，預設 True
    output_html : str, optional
        輸出 HTML 檔案路徑，若為 None 則直接顯示

    Returns:
    --------
    fig : plotly.graph_objects.Figure
        繪製的圖表物件
    """

    # ==================== 讀取資料 ====================
    if end_date is None:
        end_date = start_date
    print("📊 讀取資料中...")


    kbar_df = None
    if kbar_csv_path:
        try:
            kbar_df = pd.read_csv(kbar_csv_path, parse_dates=["start_time"])
            kbar_df = kbar_df.set_index("start_time").sort_index()
            print(kbar_df.columns)
            print(kbar_df.head())
        except Exception as e:
            print(f"⚠️ 無法讀取 kbar_csv {e}")
    print(kbar_df)
    # 讀取60分鐘K線
    kbar_60m_df = None
    if show_60m_overlay:
        if kbar_60m_csv_path:
            try:
                kbar_60m_df = pd.read_csv(kbar_60m_csv_path, parse_dates=["start_time"])
                kbar_60m_df = kbar_60m_df.set_index("start_time").sort_index()
                print(f"✅ 已讀取 60分K線，共 {len(kbar_60m_df)} 筆")
            except Exception as e:
                print(f"⚠️ 無法讀取 60分K線: {e}")
                print(f"📊 將自動從5分K線轉換為60分K線...")
                kbar_60m_df = None

        # 如果沒有60分K線資料，從5分K線轉換
        if kbar_60m_df is None and kbar_df is not None:
            print(f"📊 正在從5分K線轉換為60分K線...")
            kbar_60m_df = convert_5m_to_60m(kbar_df)
            if kbar_60m_df is not None:
                print(f"✅ 轉換完成，共 {len(kbar_60m_df)} 筆60分K線")

    # 讀取交易紀錄
    trade_df = None
    if trade_csv_path:
        try:
            trade_df = pd.read_csv(trade_csv_path, parse_dates=["tick_time"])
            print(trade_df.columns)
            print(trade_df.head())
        except Exception as e:
            print(f"⚠️ 無法讀取 trade_csv: {e}")


    # 讀取關鍵價格日誌
    ss_df = None
    if ss_log_path:
        try:
            ss_df = pd.read_csv(ss_log_path, parse_dates=["start_time"])
            print(f"✅ 已讀取 ss_log，共 {len(ss_df)} 筆")
            print(ss_df.columns)
            print(ss_df.head())
        except Exception as e:
            print(f"⚠️ 無法讀取 ss_log: {e}")

    key_price_df = None
    if key_price_log_path:
        try:
            key_price_df = pd.read_csv(key_price_log_path, parse_dates=["start_time"])
            print(f"✅ 已讀取 key_price_log，共 {len(key_price_df)} 筆")
        except Exception as e:
            print(f"⚠️ 無法讀取 key_price_log: {e}")

    # 篩選日期範圍
    if kbar_df is not None:
        kbar_df = kbar_df.loc[
                (kbar_df['file_date'] >= start_date) &
                (kbar_df['file_date'] <= end_date)
                 ]
        print(f"✅ 已讀取 kbar_csv，共 {len(kbar_df)} 筆")

    if kbar_60m_df is not None:
        if 'file_date' in kbar_60m_df.columns:
            kbar_60m_df = kbar_60m_df.loc[
                    (kbar_60m_df['file_date'] >= start_date) &
                    (kbar_60m_df['file_date'] <= end_date)
                     ]
        print(f"✅ 篩選後 60分K線筆數: {len(kbar_60m_df)}")

    if trade_df is not None:
        trade_df = trade_df.loc[
                  (trade_df["file_date"] >= start_date) &
                  (trade_df["file_date"] <= end_date)
                  ]
        print(f"✅ 已讀取 trade_csv，共 {len(trade_df)} 筆")
    if ss_df is not None:
        ss_df = ss_df.loc[
            (ss_df["file_date"] >= start_date) &
            (ss_df["file_date"] <= end_date)
        ]
        print(f"✅ 篩選後 ss_log 筆數: {len(ss_df)}")
    if key_price_df is not None:
        key_price_df = key_price_df.loc[
            (key_price_df["file_date"] >= start_date) &
            (key_price_df["file_date"] <= end_date)
        ]
        print(f"✅ 篩選後 key_price_log 筆數: {len(key_price_df)}")

    # ==================== 偵測可用欄位 ====================
    available_fields = {
        # K線基本欄位
        'ohlc': all(col in kbar_df.columns for col in ['open', 'high', 'low', 'close']),
        'ma5': 'ma5' in kbar_df.columns,
        'ma10': 'ma10' in kbar_df.columns,
        'ma20': 'ma20' in kbar_df.columns,
        'bb_upper': 'bb_upper' in kbar_df.columns,
        'bb_lower': 'bb_lower' in kbar_df.columns,
        'ema12': 'ema12' in kbar_df.columns,
        'ema26': 'ema26' in kbar_df.columns,

        # KD 指標
        'K': 'K' in kbar_df.columns,
        'D': 'D' in kbar_df.columns,
        'RSI': 'RSI' in kbar_df.columns,

        # Range Filter
        'rf_filter': 'rf_filter' in kbar_df.columns,
        'rf_high_band': 'rf_high_band' in kbar_df.columns,
        'rf_low_band': 'rf_low_band' in kbar_df.columns,
        'rf_long_signal': 'rf_long_signal' in kbar_df.columns,
        'rf_short_signal': 'rf_short_signal' in kbar_df.columns,
        'rf_upward': 'rf_upward' in kbar_df.columns,
        'rf_downward': 'rf_downward' in kbar_df.columns,
    }

    # 判斷需要幾個子圖
    has_kd_subplot = show_kd and (available_fields['K'] or available_fields['D'] or available_fields['RSI'])
    has_rf_subplot = show_rf_signals and (available_fields['rf_upward'] or available_fields['rf_downward'] or
                      available_fields['rf_long_signal'] or available_fields['rf_short_signal'])

    print(f"\n🔍 欄位偵測結果:")
    print(f"   - K線圖: {'✓' if available_fields['ohlc'] else '✗'}")
    print(f"   - 60分K線疊加: {'✓' if (kbar_60m_df is not None and show_60m_overlay) else '✗'}")
    print(f"   - MA5: {'✓' if available_fields['ma5'] else '✗'}")
    print(f"   - MA10: {'✓' if available_fields['ma10'] else '✗'}")
    print(f"   - MA20: {'✓' if available_fields['ma20'] else '✗'}")
    print(f"   - KD指標子圖: {'✓' if has_kd_subplot else '✗'}")
    print(f"   - Range Filter子圖: {'✓' if has_rf_subplot else '✗'}")

    # ==================== 對齊交易時間 ====================
    def align_to_bar(tick_times, kbar_index, interval_minutes=5, label="right"):
        aligned_times = []
        if not isinstance(kbar_index, pd.DatetimeIndex):
            kbar_index = pd.to_datetime(kbar_index)

        for t in pd.to_datetime(tick_times):
            minutes_since_midnight = t.hour * 60 + t.minute
            if label == "right":
                next_minutes = ((minutes_since_midnight // interval_minutes) + 1) * interval_minutes
                bar_time_candidate = t.normalize() + pd.Timedelta(minutes=next_minutes)
            else:
                prev_minutes = (minutes_since_midnight // interval_minutes) * interval_minutes
                bar_time_candidate = t.normalize() + pd.Timedelta(minutes=prev_minutes)

            pos = kbar_index.searchsorted(bar_time_candidate, side="right") - 1
            if pos < 0:
                pos = 0
            aligned_times.append(kbar_index[pos])

        return aligned_times

    # ==================== 處理交易資料 ====================
    if trade_df is not None:
        trade_df["bar_time"] = align_to_bar(trade_df["tick_time"], kbar_df.index, interval_minutes, label="right")

        # 修正交易配對
        fixed_trades = []
        for _, row in trade_df.iterrows():
            action = str(row.get("action", "")).upper() if pd.notna(row.get("action", None)) else ""
            signal = 1 if action == "BUY" else -1 if action == "SELL" else 0
            if signal != 0 or action.startswith("TAKE_PROFIT") or action in ["STOP_LOSS", "CHANGE_TREND"]:
                fixed_trades.append(row.to_dict())

        trade_df = pd.DataFrame(fixed_trades)

        if len(trade_df) > 0:
            trade_df["tick_time"] = pd.to_datetime(trade_df["tick_time"], errors="coerce")
            trade_df["bar_time"] = pd.to_datetime(trade_df["bar_time"], errors="coerce")

            # 分類進場與出場
            entry_trades = trade_df[trade_df["action"].isin(["BUY", "SELL"])].copy()
            exit_trades = trade_df[
                trade_df["action"].astype(str).str.startswith("TAKE_PROFIT") |
                trade_df["action"].isin(["STOP_LOSS", "CHANGE_TREND"])
            ].copy()

            # 解析價格
            def resolve_price(df, preferred_cols):
                for col in preferred_cols:
                    if col in df.columns:
                        return df[col]
                return pd.Series([pd.NA] * len(df), index=df.index)

            if len(entry_trades) > 0:
                entry_trades["bar_price"] = resolve_price(entry_trades, ["price", "bar_price", "close"])
                entry_trades = entry_trades.dropna(subset=["bar_price", "bar_time"])

            if len(exit_trades) > 0:
                exit_trades["bar_price"] = resolve_price(exit_trades, ["exit_price", "price", "bar_price", "close"])
                exit_trades = exit_trades.dropna(subset=["bar_price", "bar_time"])
        else:
            entry_trades = pd.DataFrame()
            exit_trades = pd.DataFrame()
    else:
        entry_trades = pd.DataFrame()
        exit_trades = pd.DataFrame()

    # ==================== 偵測 KD 交叉 ====================
    golden_times, golden_prices = [], []
    dead_times, dead_prices = [], []

    if show_kd and available_fields['K'] and available_fields['D']:
        for i in range(1, len(kbar_df)):
            prev_bar = kbar_df.iloc[i - 1]
            bar = kbar_df.iloc[i]
            bar_start = kbar_df.index[i]

            if pd.notna(prev_bar["K"]) and pd.notna(prev_bar["D"]) and pd.notna(bar["K"]) and pd.notna(bar["D"]):
                if prev_bar["K"] < prev_bar["D"] and bar["K"] > bar["D"]:
                    golden_times.append(bar_start)
                    golden_prices.append(bar["K"])
                if prev_bar["K"] > prev_bar["D"] and bar["K"] < bar["D"]:
                    dead_times.append(bar_start)
                    dead_prices.append(bar["K"])

    # ==================== Range Filter 信號 ====================
    rf_long_times, rf_long_prices = [], []
    rf_short_times, rf_short_prices = [], []

    if show_rf_signals and available_fields['rf_long_signal']:
        mask_long = kbar_df["rf_long_signal"].astype(bool)
        rf_long_times = kbar_df.index[mask_long].tolist()
        rf_long_prices = kbar_df["close"][mask_long].tolist() if "close" in kbar_df.columns else [None] * mask_long.sum()

    if show_rf_signals and available_fields['rf_short_signal']:
        mask_short = kbar_df["rf_short_signal"].astype(bool)
        rf_short_times = kbar_df.index[mask_short].tolist()
        rf_short_prices = kbar_df["close"][mask_short].tolist() if "close" in kbar_df.columns else [None] * mask_short.sum()

    # ==================== 建立子圖 ====================
    subplot_count = 1 + (1 if has_kd_subplot else 0) + (1 if has_rf_subplot else 0)

    if subplot_count == 1:
        row_heights = [1.0]
        subplot_titles = ["KBar with Entry/Exit Trades"]
    elif subplot_count == 2:
        row_heights = [0.65, 0.35]
        if has_kd_subplot:
            subplot_titles = ["KBar with Entry/Exit Trades", "KD Indicator"]
        else:
            subplot_titles = ["KBar with Entry/Exit Trades", "Range Filter Signals"]
    else:
        row_heights = [0.5, 0.25, 0.25]
        subplot_titles = ["KBar with Entry/Exit Trades", "KD Indicator", "Range Filter Signals"]

    fig = make_subplots(
        rows=subplot_count,
        cols=1,
        shared_xaxes=True,
        row_heights=row_heights,
        vertical_spacing=0.05,
        subplot_titles=subplot_titles
    )

    # ==================== 繪製主圖 (K線) ====================
    current_row = 1

    # 先繪製60分K線（在背景層）
    if kbar_60m_df is not None and show_60m_overlay and len(kbar_60m_df) > 0:
        if all(col in kbar_60m_df.columns for col in ['open', 'high', 'low', 'close']):
            h_60 = [f"<br>Open: {o:.0f}<br>High: {h:.0f}<br>Low: {l:.0f}<br>Close: {c:.0f}"
                    for o, h, l, c in zip(kbar_60m_df["open"], kbar_60m_df["high"],
                                          kbar_60m_df["low"], kbar_60m_df["close"])]
            fig.add_trace(
                go.Candlestick(
                    x=kbar_60m_df.index,
                    open=kbar_60m_df["open"],
                    high=kbar_60m_df["high"],
                    low=kbar_60m_df["low"],
                    close=kbar_60m_df["close"],
                    name="60m K",
                    increasing=dict(line=dict(color='rgba(100,180,100,0.5)', width=2), fillcolor='rgba(100,180,100,0.3)'),
                    decreasing=dict(line=dict(color='rgba(180,100,100,0.5)', width=2), fillcolor='rgba(180,100,100,0.3)'),
                    opacity=0.9,
                    hovertext=h_60,
                    hoverinfo="text+name"
                ),
                row=current_row,
                col=1
            )

    # 再繪製5分K線（在前景層）
    if available_fields['ohlc']:
        h_5 = [f"<br>Open: {o:.0f}<br>High: {h:.0f}<br>Low: {l:.0f}<br>Close: {c:.0f}"
               for o, h, l, c in zip(kbar_df["open"], kbar_df["high"],
                                     kbar_df["low"], kbar_df["close"])]
        fig.add_trace(
            go.Candlestick(
                x=kbar_df.index,
                open=kbar_df["open"],
                high=kbar_df["high"],
                low=kbar_df["low"],
                close=kbar_df["close"],
                name="5m K",
                hovertext=h_5,
                hoverinfo="text+name"
            ),
            row=current_row,
            col=1
        )


    # 技術指標線
    if show_ma and available_fields['ma5']:
        fig.add_trace(go.Scatter(x=kbar_df.index, y=kbar_df["ma5"], mode="lines", name="MA5",
                                line=dict(width=1.5, color="blue")), row=current_row, col=1)

    if show_ma and available_fields['ma10']:
        fig.add_trace(go.Scatter(x=kbar_df.index, y=kbar_df["ma10"], mode="lines", name="MA10",
                                line=dict(width=1.5, color="green")), row=current_row, col=1)

    if show_ma and available_fields['ma20']:
        fig.add_trace(go.Scatter(x=kbar_df.index, y=kbar_df["ma20"], mode="lines", name="MA20",
                                line=dict(width=1.5, color="orange")), row=current_row, col=1)

    if show_bb and available_fields['bb_upper']:
        fig.add_trace(go.Scatter(x=kbar_df.index, y=kbar_df["bb_upper"], mode="lines", name="BB Upper",
                                line=dict(width=1, color="gray")), row=current_row, col=1)

    if show_bb and available_fields['bb_lower']:
        fig.add_trace(go.Scatter(x=kbar_df.index, y=kbar_df["bb_lower"], mode="lines", name="BB Lower",
                                line=dict(width=1, color="gray")), row=current_row, col=1)

    if show_ema and available_fields['ema12']:
        fig.add_trace(go.Scatter(x=kbar_df.index, y=kbar_df["ema12"], mode="lines", name="EMA12",
                                line=dict(color="yellow", width=1, dash="dot")), row=current_row, col=1)

    if show_ema and available_fields['ema26']:
        fig.add_trace(go.Scatter(x=kbar_df.index, y=kbar_df["ema26"], mode="lines", name="EMA26",
                                line=dict(color="cyan", width=1, dash="dot")), row=current_row, col=1)

    # Range Filter 線條
    if show_rf_lines and available_fields['rf_filter']:
        fig.add_trace(go.Scatter(x=kbar_df.index, y=kbar_df["rf_filter"], mode="lines", name="RF Filter",
                                line=dict(color="purple", width=2)), row=current_row, col=1)

    if show_rf_lines and available_fields['rf_high_band']:
        fig.add_trace(go.Scatter(x=kbar_df.index, y=kbar_df["rf_high_band"], mode="lines", name="RF High Band",
                                showlegend=False, line=dict(color="rgba(128,128,128,0.5)")), row=current_row, col=1)

    if show_rf_lines and available_fields['rf_low_band']:
        fig.add_trace(go.Scatter(x=kbar_df.index, y=kbar_df["rf_low_band"], mode="lines", fill="tonexty",
                                fillcolor="rgba(128,128,128,0.3)", name="RF Low Band", showlegend=False,
                                line=dict(color="rgba(128,128,128,0.5)")), row=current_row, col=1)

    # ==================== 繪製關鍵價格點 ====================
    if ss_df is not None and len(ss_df) > 0:
        # 繪製 stress
        if 'stress' in ss_df.columns:
            stress_df = ss_df.dropna(subset=['stress'])
            if len(stress_df) > 0:
                fig.add_trace(go.Scatter(
                    x=stress_df["start_time"],
                    y=stress_df["stress"],
                    mode="markers",
                    marker=dict(symbol="square", color="cyan", size=5, line=dict(color="darkblue", width=1)),
                    name="stress"
                ), row=current_row, col=1)
        # 繪製 support
        if 'support' in ss_df.columns:
            support_df = ss_df.dropna(subset=['support'])
            if len(support_df) > 0:
                fig.add_trace(go.Scatter(
                    x=support_df["start_time"],
                    y=support_df["support"],
                    mode="markers",
                    marker=dict(symbol="diamond", color="magenta", size=5, line=dict(color="darkmagenta", width=1)),
                    name="support"
                ), row=current_row, col=1)



    if key_price_df is not None and len(key_price_df) > 0:
        # 繪製 enter_price
        if 'enter_price' in key_price_df.columns:
            enter_df = key_price_df.dropna(subset=['enter_price'])
            if len(enter_df) > 0:
                fig.add_trace(go.Scatter(
                    x=enter_df["start_time"],
                    y=enter_df["enter_price"],
                    mode="markers",
                    marker=dict(symbol="square", color="cyan", size=10, line=dict(color="darkblue", width=1)),
                    name="Enter Price",
                    hovertemplate="<b>Enter Price</b><br>價格: %{y:.0f}<extra></extra>"
                ), row=current_row, col=1)

        # 繪製 reset_price
        if 'reset_price' in key_price_df.columns:
            reset_df = key_price_df.dropna(subset=['reset_price'])
            if len(reset_df) > 0:
                fig.add_trace(go.Scatter(
                    x=reset_df["start_time"],
                    y=reset_df["reset_price"],
                    mode="markers",
                    marker=dict(symbol="diamond", color="magenta", size=10, line=dict(color="darkmagenta", width=1)),
                    name="Reset Price",
                    hovertemplate="<b>Reset Price</b><br>價格: %{y:.0f}<extra></extra>"
                ), row=current_row, col=1)

        if 'exit_price' in key_price_df.columns:
            exit_df = key_price_df.dropna(subset=['exit_price'])
            if len(exit_df) > 0:
                fig.add_trace(go.Scatter(
                    x=exit_df["start_time"],
                    y=exit_df["exit_price"],
                    mode="markers",
                    marker=dict(symbol="diamond", color="orange", size=10, line=dict(color="orange", width=1)),
                    name="Exit Price",
                    hovertemplate="<b>Exit Price</b><br>價格: %{y:.0f}<extra></extra>"
                ), row=current_row, col=1)
    # Range Filter 信號點
    if rf_long_times:
        fig.add_trace(go.Scatter(x=rf_long_times, y=rf_long_prices, mode="markers",
                                marker=dict(symbol="triangle-up", color="darkgreen", size=10),
                                name="RF Long Signal"), row=current_row, col=1)

    if rf_short_times:
        fig.add_trace(go.Scatter(x=rf_short_times, y=rf_short_prices, mode="markers",
                                marker=dict(symbol="triangle-down", color="darkred", size=10),
                                name="RF Short Signal"), row=current_row, col=1)

    # 進場點
    if len(entry_trades) > 0:
        buy_trades = entry_trades[entry_trades["action"] == "BUY"]
        sell_trades = entry_trades[entry_trades["action"] == "SELL"]

        if not buy_trades.empty:
            fig.add_trace(go.Scatter(
                x=buy_trades["bar_time"],
                y=buy_trades["bar_price"],
                mode="markers",
                marker=dict(symbol="triangle-up", color="green", size=12),
                name="Buy Entry",
                hovertemplate="<b>🚀 買入進場</b><br>價格: %{y:.0f}<extra></extra>"
            ), row=current_row, col=1)

        if not sell_trades.empty:
            fig.add_trace(go.Scatter(
                x=sell_trades["bar_time"],
                y=sell_trades["bar_price"],
                mode="markers",
                marker=dict(symbol="triangle-down", color="red", size=12),
                name="Sell Entry",
                hovertemplate="<b>📉 賣出進場</b><br>價格: %{y:.0f}<extra></extra>"
            ), row=current_row, col=1)

    # 出場點
    if len(exit_trades) > 0:
        take_profit_trades = exit_trades[exit_trades["action"].astype(str).str.startswith("TAKE_PROFIT")]
        stop_loss_trades = exit_trades[exit_trades["action"] == "STOP_LOSS"]
        change_trend_trades = exit_trades[exit_trades["action"] == "CHANGE_TREND"]

        if not take_profit_trades.empty:
            fig.add_trace(go.Scatter(
                x=take_profit_trades["bar_time"],
                y=take_profit_trades["bar_price"],
                mode="markers",
                marker=dict(symbol="circle", color="blue", size=10),
                name="Take Profit",
                hovertemplate="<b>💰 獲利出場</b><br>價格: %{y:.0f}<extra></extra>"
            ), row=current_row, col=1)

        if not stop_loss_trades.empty:
            fig.add_trace(go.Scatter(
                x=stop_loss_trades["bar_time"],
                y=stop_loss_trades["bar_price"],
                mode="markers",
                marker=dict(symbol="x", color="purple", size=10),
                name="Stop Loss",
                hovertemplate="<b>🛑 停損出場</b><br>價格: %{y:.0f}<extra></extra>"
            ), row=current_row, col=1)

        if not change_trend_trades.empty:
            fig.add_trace(go.Scatter(
                x=change_trend_trades["bar_time"],
                y=change_trend_trades["bar_price"],
                mode="markers",
                marker=dict(symbol="diamond", color="orange", size=10),
                name="Trend Change",
                hovertemplate="<b>🔄 趨勢改變</b><br>價格: %{y:.0f}<extra></extra>"
            ), row=current_row, col=1)

    # ==================== 繪製 KD 指標子圖 ====================
    if has_kd_subplot:
        current_row += 1

        if available_fields['K']:
            fig.add_trace(go.Scatter(x=kbar_df.index, y=kbar_df["K"], mode="lines", name="K",
                                    line=dict(color="blue")), row=current_row, col=1)

        if available_fields['D']:
            fig.add_trace(go.Scatter(x=kbar_df.index, y=kbar_df["D"], mode="lines", name="D",
                                    line=dict(color="orange")), row=current_row, col=1)

        if available_fields['RSI']:
            fig.add_trace(go.Scatter(x=kbar_df.index, y=kbar_df["RSI"], mode="lines", name="RSI",
                                    line=dict(color="purple", dash="dash")), row=current_row, col=1)

        # KD 交叉點
        if golden_times:
            fig.add_trace(go.Scatter(x=golden_times, y=golden_prices, mode="markers",
                                    marker=dict(symbol="star", color="gold", size=12),
                                    name="KD Golden Cross"), row=current_row, col=1)

        if dead_times:
            fig.add_trace(go.Scatter(x=dead_times, y=dead_prices, mode="markers",
                                    marker=dict(symbol="star", color="black", size=12),
                                    name="KD Dead Cross"), row=current_row, col=1)

        fig.update_yaxes(title_text="KD Value", row=current_row, col=1)

    # ==================== 繪製 Range Filter 子圖 ====================
    if has_rf_subplot:
        current_row += 1

        # RF 方向指標
        if available_fields['rf_upward']:
            rf_upward_values = kbar_df["rf_upward"].fillna(0).apply(lambda x: 1 if x == 1 else None).tolist()
            fig.add_trace(go.Scatter(x=kbar_df.index, y=rf_upward_values, mode="markers",
                                    marker=dict(color="green", size=8, symbol="triangle-up"),
                                    name="RF Upward"), row=current_row, col=1)

        if available_fields['rf_downward']:
            rf_downward_values = kbar_df["rf_downward"].fillna(0).apply(lambda x: -1 if x == 1 else None).tolist()
            fig.add_trace(go.Scatter(x=kbar_df.index, y=rf_downward_values, mode="markers",
                                    marker=dict(color="red", size=8, symbol="triangle-down"),
                                    name="RF Downward"), row=current_row, col=1)

        # RF 信號標記
        if available_fields['rf_long_signal']:
            rf_long_signal_values = [0.5 if v else None for v in kbar_df["rf_long_signal"].fillna(False)]
            fig.add_trace(go.Scatter(x=kbar_df.index, y=rf_long_signal_values, mode="markers",
                                    marker=dict(color="darkgreen", size=10, symbol="circle"),
                                    name="RF Long Signal"), row=current_row, col=1)

        if available_fields['rf_short_signal']:
            rf_short_signal_values = [-0.5 if v else None for v in kbar_df["rf_short_signal"].fillna(False)]
            fig.add_trace(go.Scatter(x=kbar_df.index, y=rf_short_signal_values, mode="markers",
                                    marker=dict(color="darkred", size=10, symbol="circle"),
                                    name="RF Short Signal"), row=current_row, col=1)

        fig.update_yaxes(title_text="RF Signals", range=[-1.5, 1.5], row=current_row, col=1)

    # ==================== 圖表設定 ====================
    fig.update_layout(
        title=f"KBar Analysis ({start_date} to {end_date})",
        xaxis_title="Time" if subplot_count == 1 else None,
        yaxis_title="Price",
        xaxis_rangeslider_visible=False,
        width=1400,
        height=300 + 350 * subplot_count,
        showlegend=True,
        hovermode='x unified',
        xaxis=dict(
            hoverformat='%Y/%m/%d %H:%M'
        ),
        yaxis=dict(
            separatethousands=False,
            hoverformat=".f",
            tickformat=".f"
        ),
    )

    # 隱藏所有子圖的 X 軸日期時間顯示在 hover 上
    for i in range(1, subplot_count + 1):
        fig.update_xaxes(showspikes=True, spikemode='across', spikesnap='cursor',
                        hoverformat='%Y/%m/%d %H:%M', row=i, col=1)
        fig.update_yaxes(showspikes=True, spikemode='across', row=i, col=1)

    # 設定最下方子圖的 X 軸標題
    fig.update_xaxes(title_text="Time", row=subplot_count, col=1)

    # ==================== 輸出 ====================
    if output_html:
        fig.write_html(output_html)
        print(f"\n✅ 已輸出圖表至: {output_html}")
    else:
        fig.show()

    return fig

##有指標SIM

In [ ]:
import os
import pickle
import pandas as pd
from datetime import datetime


def run_simulation(
    start_date: str,
    end_date: str,
    folder_path: str =folder_path,
    interval_minutes: int = 5,
    kbars_csv: str = kbars_csv,
    tick_log_csv: str = tick_log_csv,
    trade_log_csv: str = trade_log_csv,
    key_price_log_csv: str = key_price_log_csv
):
    """
    主控模擬流程，依日期區間批次讀取 parquet 檔案，
    執行 KBar 模擬、輸出 K 線 / Tick / 交易紀錄 / 統計分析。

    Args:
        folder_path (str): parquet 檔所在資料夾
        start_date (str): 起始日期 (YYYY-MM-DD)
        end_date (str): 結束日期 (YYYY-MM-DD)
        interval_minutes (int): K 線週期 (預設 5 分)
        kbars_csv, tick_log_csv, trade_log_csv, key_price_log_csv: 各輸出檔名
    """
    start_dt = datetime.strptime(start_date, "%Y-%m-%d")
    end_dt = datetime.strptime(end_date, "%Y-%m-%d")

    # ====== 取得檔案清單 ======
    file_list = []
    for f in os.listdir(folder_path):
        if f.startswith("CLEAN_TICK_Daily_") and f.endswith(".parquet"):
            try:
                date_str = f.split("_Daily_")[1].replace(".parquet", "")
                file_date = datetime.strptime(date_str, "%Y_%m_%d")
                if start_dt <= file_date <= end_dt:
                    file_list.append(os.path.join(folder_path, f))
            except ValueError:
                continue

    file_list = sorted(file_list)
    print(f"找到 {len(file_list)} 個檔案\n")

    # ====== 狀態管理 ======
    sim = None
    start_index = 0
    last_kbars_count = 0

    if os.path.exists("sim_state.pkl"):
        with open("sim_state.pkl", "rb") as f:
            state = pickle.load(f)
        sim = state["sim"]
        last_kbars_count = state.get("kbars_count", 0)
        start_index = state["last_index"] + 1
        print(f"從第 {start_index+1} 檔繼續（已有 {len(sim.completed_bars)} 根K線）\n")
    else:
        print("未找到狀態檔，從頭開始\n")
        for csv_file in [kbars_csv, tick_log_csv, trade_log_csv, key_price_log_csv]:
            if os.path.exists(csv_file):
                os.remove(csv_file)

    # ====== 主迴圈 ======
    for i, file_path in enumerate(file_list[start_index:], start=start_index):
        file_name = os.path.basename(file_path)
        print(f"[{i+1}/{len(file_list)}] {file_name}")

        # 提取日期
        try:
            date_str = file_name.split("_Daily_")[1].replace(".parquet", "")
            file_date = datetime.strptime(date_str, "%Y_%m_%d").strftime("%Y-%m-%d")
        except:
            file_date = "未知日期"

        try:
            # 讀檔 + 轉K線
            ticks, km,km60 = process_rpt_file(file_path, f"{interval_minutes}min")
            print(f"  載入: {len(ticks)} 筆ticks, {len(km)} 根K線")

            if sim is None:
                print("  初始化模擬器...")
                km["file_date"] = file_date
                sim = KBarSimulator(
                    historical_kbars=km,
                    historical_kbars_60k=km60,
                    interval_minutes=interval_minutes,
                    debug=True,
                    rf_period=20,
                    rf_multiplier=3.5,
                    rsi_period=14
                )
                last_kbars_count = len(sim.completed_bars)
            else:
                ticks["file_date"] = file_date
                sim.run(ticks, is_first_file=False)

            # --- 儲存增量K線 ---
            new_kbars_count = len(sim.completed_bars)
            new_kbars = sim.completed_bars[last_kbars_count:]
            if new_kbars:
                kbar_df = pd.DataFrame(new_kbars)
                kbar_df["file_date"] = file_date
                kbar_df.to_csv(
                    kbars_csv, mode="a" if os.path.exists(kbars_csv) else "w",
                    index=False, encoding="utf-8-sig", header=not os.path.exists(kbars_csv)
                )
                print(f"  ✓ 儲存 {len(new_kbars)} 根K線到 CSV (日期: {file_date})")

            # --- 儲存 Tick Log ---
            if sim.tick_logs:
                tick_df = pd.DataFrame(sim.tick_logs)
                tick_df["file_date"] = file_date
                tick_df.to_csv(
                    tick_log_csv, mode="a" if os.path.exists(tick_log_csv) else "w",
                    index=False, encoding="utf-8-sig", header=not os.path.exists(tick_log_csv)
                )
                print(f"  ✓ 儲存 {len(sim.tick_logs)} 筆 ticks (日期: {file_date})")
                sim.tick_logs = []

            # --- 儲存 Trade Log ---
            if sim.trade_log:
                trade_df = pd.DataFrame(sim.trade_log)
                trade_df["file_date"] = file_date
                trade_df.to_csv(
                    trade_log_csv, mode="a" if os.path.exists(trade_log_csv) else "w",
                    index=False, encoding="utf-8-sig", header=not os.path.exists(trade_log_csv)
                )
                print(f"  ✓ 儲存 {len(sim.trade_log)} 筆交易 (日期: {file_date})")
                sim.trade_log = []

            # --- 儲存 Key Price Log ---
            if sim.ss_log:
                ss_df = pd.DataFrame(sim.ss_log)
                ss_df["file_date"] = file_date
                ss_df.to_csv(
                    ss_log_csv, mode="a" if os.path.exists(ss_log_csv) else "w",
                    index=False, encoding="utf-8-sig", header=not os.path.exists(ss_log_csv)
                )
                print(f"  ✓ 儲存 {len(sim.ss_log)} 筆壓力支撐 (日期: {file_date})")
                sim.ss_log = []
            if sim.key_price_log:
                key_df = pd.DataFrame(sim.key_price_log)
                key_df["file_date"] = file_date
                key_df.to_csv(
                    key_price_log_csv, mode="a" if os.path.exists(key_price_log_csv) else "w",
                    index=False, encoding="utf-8-sig", header=not os.path.exists(key_price_log_csv)
                )
                print(f"  ✓ 儲存 {len(sim.key_price_log)} 筆關鍵價格 (日期: {file_date})")
                sim.key_price_log = []

            # 更新狀態
            last_kbars_count = new_kbars_count
            state = {"sim": sim, "last_index": i, "kbars_count": last_kbars_count}
            with open("sim_state.pkl", "wb") as f:
                pickle.dump(state, f)
            print("  ✓ 進度已保存\n")

        except Exception as e:
            print(f"  ❌ 錯誤: {e}\n")
            import traceback
            traceback.print_exc()

    # ====== 最終統計 ======
    print("\n" + "=" * 60)
    print("所有檔案處理完成")
    print("=" * 60)

    if sim and len(sim.completed_bars) > 0:
        final_kbars = pd.DataFrame(sim.completed_bars)
        print(f"\n✓ 總共 {len(final_kbars)} 根K線")
        for col in ['RSI', 'ema12', 'ema26', 'K', 'D']:
            none_cnt = final_kbars[col].isna().sum()
            print(f"  {col}: {none_cnt} 缺失值 ({100 * none_cnt / len(final_kbars):.2f}%)")

        # 匯總交易資料
        if os.path.exists(trade_log_csv):
            trade_df = pd.read_csv(trade_log_csv)
            trade_df["date"] = trade_df["file_date"]
            if "PnL" in trade_df.columns:
                trade_df["pnl"] = trade_df["PnL"]
            elif "pnl" not in trade_df.columns:
                trade_df["pnl"] = 0.0

            # 每日統計
            daily_stats = (
                trade_df.groupby(["action", "date"])
                .agg(count=("action", "size"),
                    total_pnl=("pnl", "sum"),
                    max_pnl=("pnl", "max"),
                    min_pnl=("pnl", "min"))
                .reset_index()
            )

            # 依 action 分組後分別排序
            sorted_groups = []
            for action, subdf in daily_stats.groupby("action"):
                if action == "STOP_LOSS":
                    subdf = subdf.sort_values(["total_pnl","count"], ascending=[True,False])  # count 多→total_pnl 小
                else:
                    subdf = subdf.sort_values(["total_pnl","count"], ascending=[False, False])  # count 多→total_pnl 大
                sorted_groups.append(subdf.head(5))

            top5_by_action = pd.concat(sorted_groups, ignore_index=True)

            # 顯示前5名統計
            for action, subdf in top5_by_action.groupby("action"):
                print(f"\n=== {action} 出現次數最多的前5日 ===")
                print(subdf[["date", "count", "total_pnl", "max_pnl", "min_pnl"]].to_string(index=False))

            # 全體統計
            total_trades = len(trade_df[trade_df["action"] == "BUY"])
            total_pnl = trade_df["pnl"].sum()
            avg_pnl = total_pnl / total_trades if total_trades else 0
            win_trades = (trade_df["pnl"] > 0).sum()
            win_rate = win_trades / total_trades * 100 if total_trades else 0

            print("\n=== 全部交易整體統計 ===")
            print(f"總交易次數：{total_trades}")
            print(f"總獲利：{total_pnl:.2f}")
            print(f"平均每筆獲利：{avg_pnl:.2f}")
            print(f"勝率：{win_rate:.2f}%")

            overall_by_action = (
                trade_df.groupby("action")
                .agg(
                    trade_count=("action", "size"),
                    total_pnl=("pnl", "sum"),
                    avg_pnl=("pnl", "mean"),
                    win_rate=("pnl", lambda x: (x > 0).mean() * 100),
                )
                .reset_index()
                .sort_values("action")
            )

            print("\n=== 各 ACTION 統計 ===")
            print(overall_by_action.to_string(index=False, float_format=lambda x: f"{x:,.2f}"))

    # 清理狀態檔
    if os.path.exists("sim_state.pkl"):
        os.remove("sim_state.pkl")
        print("\n✓ 狀態檔已清理")



# ===== 使用範例 =====
# run_simulation("2025-05-22", "2025-05-24")


##無指標SIM

In [ ]:
import os
import pickle
import pandas as pd
from datetime import datetime

kbars_csv="kbars.csv"
tick_log_csv='tick_log.csv'
trade_log_csv="trade_log.csv"
ss_log_csv="ss_log.csv"
key_price_log_csv="key_price_log.csv"


def run_simulation(
    start_date: str,
    end_date: str,
    folder_path: str = folder_path,  # 假設外部有定義 folder_path
    interval_minutes: int = 5,
    kbars_csv: str = kbars_csv,
    tick_log_csv: str = tick_log_csv,
    trade_log_csv: str = trade_log_csv,
    key_price_log_csv: str = key_price_log_csv
):
    """
    主控模擬流程，依日期區間批次讀取 parquet 檔案，
    執行 KBar 模擬、輸出 K 線 / Tick / 交易紀錄 / 統計分析。
    """
    start_dt = datetime.strptime(start_date, "%Y-%m-%d")
    end_dt = datetime.strptime(end_date, "%Y-%m-%d")

    # ====== 取得檔案清單 ======
    file_list = []
    for f in os.listdir(folder_path):
        if f.startswith("CLEAN_TICK_Daily_") and f.endswith(".parquet"):
            try:
                date_str = f.split("_Daily_")[1].replace(".parquet", "")
                file_date = datetime.strptime(date_str, "%Y_%m_%d")
                if start_dt <= file_date <= end_dt:
                    file_list.append(os.path.join(folder_path, f))
            except ValueError:
                continue

    file_list = sorted(file_list)
    print(f"找到 {len(file_list)} 個檔案\n")

    # ====== 狀態管理 ======
    sim = None
    start_index = 0
    last_kbars_count = 0

    if os.path.exists("sim_state.pkl"):
        with open("sim_state.pkl", "rb") as f:
            state = pickle.load(f)
        sim = state["sim"]
        last_kbars_count = state.get("kbars_count", 0)
        start_index = state["last_index"] + 1
        print(f"從第 {start_index+1} 檔繼續（已有 {len(sim.completed_bars)} 根K線）\n")
    else:
        print("未找到狀態檔，從頭開始\n")
        for csv_file in [kbars_csv, tick_log_csv, trade_log_csv, key_price_log_csv]:
            if os.path.exists(csv_file):
                os.remove(csv_file)

    # ====== 主迴圈 ======
    for i, file_path in enumerate(file_list[start_index:], start=start_index):
        file_name = os.path.basename(file_path)
        print(f"[{i+1}/{len(file_list)}] {file_name}")

        # 提取日期
        try:
            date_str = file_name.split("_Daily_")[1].replace(".parquet", "")
            file_date = datetime.strptime(date_str, "%Y_%m_%d").strftime("%Y-%m-%d")
        except:
            file_date = "未知日期"

        try:
            # 讀檔 + 轉K線
            ticks, km,km60 = process_rpt_file(file_path, f"{interval_minutes}min")
            print(f"  載入: {len(ticks)} 筆ticks, {len(km)} 根K線")


            if sim is None:
                print("  初始化模擬器...")
                km["file_date"] = file_date
                # === 這裡是關鍵修正：只傳新版支援的參數 ===
                sim = KBarSimulator(
                    historical_kbars=km,
                    historical_kbars_60k=km60,
                    interval_minutes=interval_minutes,
                    debug=True,
                    max_position=1,              # 可自行調整
                    label="right",               # 或 "left"，依您需求
                    calc_technical_indicators=True
                )
                last_kbars_count = len(sim.completed_bars)
            else:
                ticks["file_date"] = file_date
                sim.run(ticks, is_first_file=False)

            # --- 儲存增量K線 ---
            new_kbars_count = len(sim.completed_bars)
            new_kbars = sim.completed_bars[last_kbars_count:]
            if new_kbars:
                kbar_df = pd.DataFrame(new_kbars)
                kbar_df["file_date"] = file_date
                header = not os.path.exists(kbars_csv)
                kbar_df.to_csv(
                    kbars_csv, mode="a", index=False, encoding="utf-8-sig", header=header
                )
                print(f"  ✓ 儲存 {len(new_kbars)} 根K線到 CSV (日期: {file_date})")

            # --- 儲存 Tick Log ---
            if sim.tick_logs:
                tick_df = pd.DataFrame(sim.tick_logs)
                tick_df["file_date"] = file_date
                header = not os.path.exists(tick_log_csv)
                tick_df.to_csv(
                    tick_log_csv, mode="a", index=False, encoding="utf-8-sig", header=header
                )
                print(f"  ✓ 儲存 {len(sim.tick_logs)} 筆 ticks (日期: {file_date})")
                sim.tick_logs = []

            # --- 儲存 Trade Log ---
            if sim.trade_log:
                trade_df = pd.DataFrame(sim.trade_log)
                print(trade_df)
                trade_df["file_date"] = file_date
                header = not os.path.exists(trade_log_csv)
                trade_df.to_csv(
                    trade_log_csv, mode="a", index=False, encoding="utf-8-sig", header=header
                )
                print(f"  ✓ 儲存 {len(sim.trade_log)} 筆交易 (日期: {file_date})")
                sim.trade_log = []

            # --- 儲存 Key Price Log ---
            if getattr(sim, "ss_log", None):
                ss_df = pd.DataFrame(sim.ss_log)
                ss_df["file_date"] = file_date
                header = not os.path.exists(ss_log_csv)
                ss_df.to_csv(
                    ss_log_csv, mode="a", index=False, encoding="utf-8-sig", header=header
                )
                print(f"  ✓ 儲存 {len(sim.ss_log)} 筆壓力支撐 (日期: {file_date})")
                sim.ss_log = []

            if getattr(sim, "key_price_log", None):
                key_df = pd.DataFrame(sim.key_price_log)
                key_df["file_date"] = file_date
                header = not os.path.exists(key_price_log_csv)
                key_df.to_csv(
                    key_price_log_csv, mode="a", index=False, encoding="utf-8-sig", header=header
                )
                print(f"  ✓ 儲存 {len(sim.key_price_log)} 筆關鍵價格 (日期: {file_date})")
                sim.key_price_log = []

            # 更新狀態
            last_kbars_count = new_kbars_count
            state = {"sim": sim, "last_index": i, "kbars_count": last_kbars_count}
            with open("sim_state.pkl", "wb") as f:
                pickle.dump(state, f)
            print("  ✓ 進度已保存\n")

        except Exception as e:
            print(f"  ❌ 錯誤: {e}\n")
            import traceback
            traceback.print_exc()

    # ====== 最終統計 ======
    print("\n" + "=" * 60)
    print("所有檔案處理完成")
    print("=" * 60)

    if sim and len(sim.completed_bars) > 0:
        final_kbars = pd.DataFrame(sim.completed_bars)
        print(f"\n✓ 總共 {len(final_kbars)} 根K線")


        # 匯總交易資料
        if os.path.exists(trade_log_csv):
            trade_df = pd.read_csv(trade_log_csv)
            trade_df["date"] = trade_df["file_date"]
            if "PnL" in trade_df.columns:
                trade_df["pnl"] = trade_df["PnL"]
            elif "pnl" not in trade_df.columns:
                trade_df["pnl"] = 0.0

            # 每日統計
            daily_stats = (
                trade_df.groupby(["action", "date"])
                .agg(count=("action", "size"),
                    total_pnl=("pnl", "sum"),
                    max_pnl=("pnl", "max"),
                    min_pnl=("pnl", "min"))
                .reset_index()
            )

            # 依 action 分組後分別排序
            sorted_groups = []
            for action, subdf in daily_stats.groupby("action"):
                if action == "STOP_LOSS":
                    subdf = subdf.sort_values(["total_pnl","count"], ascending=[True,False])  # count 多→total_pnl 小
                else:
                    subdf = subdf.sort_values(["total_pnl","count"], ascending=[False, False])  # count 多→total_pnl 大
                sorted_groups.append(subdf.head(5))

            top5_by_action = pd.concat(sorted_groups, ignore_index=True)

            # 顯示前5名統計
            for action, subdf in top5_by_action.groupby("action"):
                print(f"\n=== {action} 出現次數最多的前5日 ===")
                print(subdf[["date", "count", "total_pnl", "max_pnl", "min_pnl"]].to_string(index=False))

            # 全體統計
            total_trades = len(trade_df[trade_df["action"] == "BUY"])
            total_pnl = trade_df["pnl"].sum()
            avg_pnl = total_pnl / total_trades if total_trades else 0
            win_trades = (trade_df["pnl"] > 0).sum()
            win_rate = win_trades / total_trades * 100 if total_trades else 0

            print("\n=== 全部交易整體統計 ===")
            print(f"總交易次數：{total_trades}")
            print(f"總獲利：{total_pnl:.2f}")
            print(f"平均每筆獲利：{avg_pnl:.2f}")
            print(f"勝率：{win_rate:.2f}%")

            overall_by_action = (
                trade_df.groupby("action")
                .agg(
                    trade_count=("action", "size"),
                    total_pnl=("pnl", "sum"),
                    avg_pnl=("pnl", "mean"),
                    win_rate=("pnl", lambda x: (x > 0).mean() * 100),
                )
                .reset_index()
                .sort_values("action")
            )

            print("\n=== 各 ACTION 統計 ===")
            print(overall_by_action.to_string(index=False, float_format=lambda x: f"{x:,.2f}"))

    # 清理狀態檔
    if os.path.exists("sim_state.pkl"):
        os.remove("sim_state.pkl")
        print("\n✓ 狀態檔已清理")


##無指標SIM_N

In [ ]:
import os
import pickle
import pandas as pd
from datetime import datetime



def save_to_csv(data, csv_path: str, file_date: str, data_type: str = "records"):

    if not data:
        return

    # 轉換為 DataFrame
    if isinstance(data, list):
        df = pd.DataFrame(data)
    else:
        df = data.copy()

    # 添加日期欄位
    df["file_date"] = file_date

    # 判斷是否需要寫入表頭
    header = not os.path.exists(csv_path)

    # 寫入 CSV
    df.to_csv(
        csv_path,
        mode="a",
        index=False,
        encoding="utf-8-sig",
        header=header
    )

    print(f"  ✓ 儲存 {len(df)} 筆{data_type}到 CSV (日期: {file_date})")


def run_simulation(
    start_date: str,
    end_date: str,
    folder_path: str = folder_path,  # 假設外部有定義 folder_path
    interval_minutes: int = 5,
    kbars_csv: str = kbars_csv,
    tick_log_csv: str = tick_log_csv,
    trade_log_csv: str = trade_log_csv,
    key_price_log_csv: str = key_price_log_csv
):
    """
    主控模擬流程，依日期區間批次讀取 parquet 檔案，
    執行 KBar 模擬、輸出 K 線 / Tick / 交易紀錄 / 統計分析。
    """
    start_dt = datetime.strptime(start_date, "%Y-%m-%d")
    end_dt = datetime.strptime(end_date, "%Y-%m-%d")

    # ====== 取得檔案清單 ======
    file_list = []
    for f in os.listdir(folder_path):
        if f.startswith("CLEAN_TICK_Daily_") and f.endswith(".parquet"):
            try:
                date_str = f.split("_Daily_")[1].replace(".parquet", "")
                file_date = datetime.strptime(date_str, "%Y_%m_%d")
                if start_dt <= file_date <= end_dt:
                    file_list.append(os.path.join(folder_path, f))
            except ValueError:
                continue

    file_list = sorted(file_list)
    print(f"找到 {len(file_list)} 個檔案\n")

    # ====== 狀態管理 ======
    sim = None
    start_index = 0
    last_kbars_count = 0

    if os.path.exists("sim_state.pkl"):
        with open("sim_state.pkl", "rb") as f:
            state = pickle.load(f)
        sim = state["sim"]
        last_kbars_count = state.get("kbars_count", 0)
        start_index = state["last_index"] + 1
        print(f"從第 {start_index+1} 檔繼續（已有 {len(sim.completed_bars)} 根K線）\n")
    else:
        print("未找到狀態檔，從頭開始\n")
        for csv_file in [kbars_csv, tick_log_csv, trade_log_csv, ss_log_csv, key_price_log_csv]:
            if os.path.exists(csv_file):
                os.remove(csv_file)

    # ====== 主迴圈 ======
    for i, file_path in enumerate(file_list[start_index:], start=start_index):
        file_name = os.path.basename(file_path)
        print(f"[{i+1}/{len(file_list)}] {file_name}")

        # 提取日期
        try:
            date_str = file_name.split("_Daily_")[1].replace(".parquet", "")
            file_date = datetime.strptime(date_str, "%Y_%m_%d").strftime("%Y-%m-%d")
        except:
            file_date = "未知日期"

        try:
            # 讀檔 + 轉K線
            ticks, km, km60 = process_rpt_file(file_path, f"{interval_minutes}min")
            print(f"  載入: {len(ticks)} 筆ticks, {len(km)} 根K線")

            if sim is None:
                print("  初始化模擬器...")
                km["file_date"] = file_date
                sim = KBarSimulator(
                    historical_kbars=km,
                    historical_kbars_60k=km60,
                    interval_minutes=interval_minutes,
                    debug=True,
                    max_position=1,
                    label="right",
                    calc_technical_indicators=True
                )
                last_kbars_count = len(sim.completed_bars)
            else:
                ticks["file_date"] = file_date
                sim.run(ticks, is_first_file=False)

            # ====== 使用統一方法儲存各類數據 ======

            # 儲存增量K線
            new_kbars_count = len(sim.completed_bars)
            new_kbars = sim.completed_bars[last_kbars_count:]
            save_to_csv(new_kbars, kbars_csv, file_date, "根K線")

            # 儲存 Tick Log
            save_to_csv(sim.tick_logs, tick_log_csv, file_date, "筆 ticks")
            sim.tick_logs = []

            # 儲存 Trade Log
            if sim.trade_log:
                print(pd.DataFrame(sim.trade_log))
            save_to_csv(sim.trade_log, trade_log_csv, file_date, "筆交易")
            sim.trade_log = []

            # 儲存壓力支撐 Log
            if hasattr(sim, "ss_log"):
                save_to_csv(sim.ss_log, ss_log_csv, file_date, "筆壓力支撐")
                sim.ss_log = []

            # 儲存關鍵價格 Log
            if hasattr(sim, "key_price_log"):
                save_to_csv(sim.key_price_log, key_price_log_csv, file_date, "筆關鍵價格")
                sim.key_price_log = []

            # 更新狀態
            last_kbars_count = new_kbars_count
            state = {"sim": sim, "last_index": i, "kbars_count": last_kbars_count}
            with open("sim_state.pkl", "wb") as f:
                pickle.dump(state, f)
            print("  ✓ 進度已保存\n")

        except Exception as e:
            print(f"  ❌ 錯誤: {e}\n")
            import traceback
            traceback.print_exc()

    # ====== 最終統計 ======
    print("\n" + "=" * 60)
    print("所有檔案處理完成")
    print("=" * 60)

    if sim and len(sim.completed_bars) > 0:
        final_kbars = pd.DataFrame(sim.completed_bars)
        print(f"\n✓ 總共 {len(final_kbars)} 根K線")

        # 匯總交易資料
        if os.path.exists(trade_log_csv):
            trade_df = pd.read_csv(trade_log_csv)
            trade_df["date"] = trade_df["file_date"]
            if "PnL" in trade_df.columns:
                trade_df["pnl"] = trade_df["PnL"]
            elif "pnl" not in trade_df.columns:
                trade_df["pnl"] = 0.0

            # 每日統計
            daily_stats = (
                trade_df.groupby(["action", "date"])
                .agg(count=("action", "size"),
                    total_pnl=("pnl", "sum"),
                    max_pnl=("pnl", "max"),
                    min_pnl=("pnl", "min"))
                .reset_index()
            )

            # 依 action 分組後分別排序
            sorted_groups = []
            for action, subdf in daily_stats.groupby("action"):
                if action == "STOP_LOSS":
                    subdf = subdf.sort_values(["total_pnl", "count"], ascending=[True, False])
                else:
                    subdf = subdf.sort_values(["total_pnl", "count"], ascending=[False, False])
                sorted_groups.append(subdf.head(5))

            top5_by_action = pd.concat(sorted_groups, ignore_index=True)

            # 顯示前5名統計
            for action, subdf in top5_by_action.groupby("action"):
                print(f"\n=== {action} 出現次數最多的前5日 ===")
                print(subdf[["date", "count", "total_pnl", "max_pnl", "min_pnl"]].to_string(index=False))

            # 全體統計
            total_trades = len(trade_df[trade_df["action"] == "BUY"])
            total_pnl = trade_df["pnl"].sum()
            avg_pnl = total_pnl / total_trades if total_trades else 0
            win_trades = (trade_df["pnl"] > 0).sum()
            win_rate = win_trades / total_trades * 100 if total_trades else 0

            print("\n=== 全部交易整體統計 ===")
            print(f"總交易次數：{total_trades}")
            print(f"總獲利：{total_pnl:.2f}")
            print(f"平均每筆獲利：{avg_pnl:.2f}")
            print(f"勝率：{win_rate:.2f}%")

            overall_by_action = (
                trade_df.groupby("action")
                .agg(
                    trade_count=("action", "size"),
                    total_pnl=("pnl", "sum"),
                    avg_pnl=("pnl", "mean"),
                    win_rate=("pnl", lambda x: (x > 0).mean() * 100),
                )
                .reset_index()
                .sort_values("action")
            )

            print("\n=== 各 ACTION 統計 ===")
            print(overall_by_action.to_string(index=False, float_format=lambda x: f"{x:,.2f}"))

    # 清理狀態檔
    if os.path.exists("sim_state.pkl"):
        os.remove("sim_state.pkl")
        print("\n✓ 狀態檔已清理")
    return sim

#主程式

##有指標

In [ ]:
import pandas as pd
from datetime import timedelta
import math
import time


class KBarSimulator:
    def __init__(self, historical_kbars,historical_kbars_60k, interval_minutes=5, debug=True, label="right",
                bb_period=20, bb_std=2, kd_period=14, kd_d_period=3,
                rsi_period=14, max_position=1,
                rf_period=20, rf_multiplier=3.5,
                calc_technical_indicators=False):
        self.calc_technical_indicators = calc_technical_indicators
        self.interval = timedelta(minutes=interval_minutes)
        self.debug = debug
        self.label = label
        self.bb_period = bb_period
        self.bb_std = bb_std
        self.kd_period = kd_period
        self.kd_d_period = kd_d_period
        self.rsi_period = rsi_period
        self.completed_bars = []
        self.tick_logs = []
        self.current_bar = None
        self.current_bar_trade_info = {
            "max_buy_price": None,
            "min_sell_price": None,
            "stop_loss_count": 0
        }

        # === 交易控管 ===
        self.position = 0
        self.entry_price = None
        self.exit_price = None
        self.entry_time = None
        self.trade_log = []
        self.max_position = max_position
        self.max_pnl_pct = 0
        self.max_pnl = 0


        # === BB 狀態初始值 ===
        self.last_bb_state = 0
        self.golden_cross_history = []
        self.dead_cross_history = []
        self.fail_history = []
        self.interval_high = None
        self.interval_low = None
        self.rollback_rate = 0
        self.rf_state=0


        self.bar_stream=[]
        self.up_stream=[]
        self.down_stream=[]
        self.bar_pattern=[]
        self.inertia=[]
        self.fail_stream=[]
        # self.key_entry_price2 = None
        # self.key_exit_price2 = None
        # self.key_reset_price2 = None
        self.support_price=None
        self.stress_price=None
        self.ss_log=[]
        self.day_highs=[]
        self.day_lows=[]


        self.pre_tick_price = None
        self.min_between_gold_cross = None
        self.max_between_dead_cross = None
        self.key_price_log=[]
        self.key_entry_price = None
        self.key_exit_price = None
        self.key_reset_price = None

        self.trend_g1 = 0
        self.trend_g2 = 0
        self.trend_d1 = 0
        self.trend_d2 = 0
        self.trend_upper_1 = 0
        self.trend_upper_2 = 0
        self.trend_lower_1 = 0
        self.trend_lower_2 = 0
        self.trend_center_1 = 0
        self.trend_center_2 = 0
        self.state_ma20_low = 0
        self.state_ma20_high = 0
        self.k_state = 0

        # === Range Filter parameters ===
        self.rf_period = rf_period
        self.rf_multiplier = rf_multiplier
        self.rf_direction = 0.0
        self.rf_condition_ini = 0

        # 初始化 combined_check 及 completed_bars
        self.update_combined_check(historical_kbars)
        self._initialize_completed_bars(historical_kbars)

    # ------------------ MA Calculation Functions ------------------
    def _calculate_ma(self, closes, period):
        """
        Calculate Simple Moving Average
        Args:
            closes: List of closing prices
            period: MA period
        Returns:
            MA value
        """
        if len(closes) < period:
            return sum(closes) / len(closes) if closes else None
        return sum(closes[-period:]) / period

    # ------------------ EMA Calculation Functions ------------------
    def _calculate_ema(self, closes, period):
        """
        Calculate EMA (Exponential Moving Average)
        Args:
            closes: List of closing prices
            period: EMA period
        Returns:
            EMA value
        """
        if len(closes) == 0:
            return None

        alpha = 2.0 / (period + 1)
        ema = closes[0]

        for i in range(1, len(closes)):
            ema = alpha * closes[i] + (1 - alpha) * ema

        return ema

    # ------------------ RSI Calculation Functions ------------------
    def _calculate_rsi(self, closes):
        return None

    # ------------------ Range Filter Functions ------------------
    def _range_size(self, prices, qty, n):
        """
        Calculate range size using EMA of price changes
        """
        if len(prices) < 2:
            return 0.0

        abs_changes = [abs(prices[i] - prices[i-1]) for i in range(1, len(prices))]
        if not abs_changes:
            return 0.0

        alpha = 2.0 / (n + 1)
        avrng = abs_changes[0]
        for change in abs_changes[1:]:
            avrng = alpha * change + (1 - alpha) * avrng

        wper = (n * 2) - 1
        alpha_ac = 2.0 / (wper + 1)
        ac = avrng * qty

        return ac

    def _range_filter(self, current_price, range_size, previous_filter=None):
        """
        Calculate range filter values
        Returns: (high_band, low_band, filter_value)
        """
        if previous_filter is None:
            filter_value = current_price
        else:
            filter_value = previous_filter

            if current_price - range_size > filter_value:
                filter_value = current_price - range_size
            elif current_price + range_size < filter_value:
                filter_value = current_price + range_size

        high_band = filter_value + range_size
        low_band = filter_value - range_size

        return high_band, low_band, filter_value

    def _calculate_range_filter_signals(self, price, high_band, low_band, filter_value):
        """
        計算 Range Filter buy/sell signals
        Returns: (long_condition, short_condition, upward, downward)
        """
        if len(self.completed_bars) > 0:
            previous_filter = self.completed_bars[-1].get("rf_filter")
        else:
            previous_filter = filter_value

        if filter_value > previous_filter:
            self.rf_direction = 1
        elif filter_value < previous_filter:
            self.rf_direction = -1

        upward = 1 if self.rf_direction == 1 else 0
        downward = 1 if self.rf_direction == -1 else 0

        prev_price = self.pre_tick_price if self.pre_tick_price is not None else price

        long_cond = (price > filter_value and price > prev_price and upward > 0) or \
                  (price > filter_value and price < prev_price and upward > 0)

        short_cond = (price < filter_value and price < prev_price and downward > 0) or \
                    (price < filter_value and price > prev_price and downward > 0)

        if long_cond:
            new_condition_ini = 1
        elif short_cond:
            new_condition_ini = -1
        else:
            new_condition_ini = self.rf_condition_ini

        long_condition = long_cond and self.rf_condition_ini == -1
        short_condition = short_cond and self.rf_condition_ini == 1

        self.rf_condition_ini = new_condition_ini

        return long_condition, short_condition, upward, downward

    # ------------------ 初始化計算 ------------------
    def _initialize_completed_bars(self, historical_kbars):
        closes = historical_kbars["close"].tolist()
        highs = historical_kbars["high"].tolist()
        lows = historical_kbars["low"].tolist()
        k_list = []

        for i in range(len(historical_kbars)):
            # MA5, MA10, MA20
            ma5 = self._calculate_ma(closes[:i + 1], 5)
            ma10 = self._calculate_ma(closes[:i + 1], 10)
            ma20 = self._calculate_ma(closes[:i + 1], 20)

            # EMA12 and EMA26
            ema12 = self._calculate_ema(closes[:i + 1], 12)
            ema26 = self._calculate_ema(closes[:i + 1], 26)

            # BB
            if i + 1 >= self.bb_period:
                window = closes[i - self.bb_period + 1:i + 1]
                bb_ma = sum(window) / self.bb_period
                std = math.sqrt(sum((c - bb_ma) ** 2 for c in window) / self.bb_period)
            else:
                window = closes[:i + 1]
                bb_ma = sum(window) / len(window)
                std = math.sqrt(sum((c - bb_ma) ** 2 for c in window) / len(window))
            upper = bb_ma + self.bb_std * std
            lower = bb_ma - self.bb_std * std

            # KD
            if i + 1 >= self.kd_period:
                low_min = min(lows[i - self.kd_period + 1:i + 1])
                high_max = max(highs[i - self.kd_period + 1:i + 1])
            else:
                low_min = min(lows[:i + 1])
                high_max = max(highs[:i + 1])
            k = 100 * (closes[i] - low_min) / (high_max - low_min) if high_max != low_min else 50
            if i >= self.kd_d_period - 1:
                d = sum(k_list[-(self.kd_d_period-1):] + [k]) / self.kd_d_period
            else:
                d = k
            k_list.append(k)

            # RSI calculation
            rsi = self._calculate_rsi(closes[:i + 1])

            # Range Filter calculation
            price_window = closes[:i + 1]
            range_size = self._range_size(price_window, self.rf_multiplier, self.rf_period)

            if i > 0 and len(self.completed_bars) > 0:
                previous_filter = self.completed_bars[-1].get("rf_filter")
            else:
                previous_filter = None

            h_band, l_band, filt = self._range_filter(closes[i], range_size, previous_filter)

            # Calculate Range Filter signals
            rf_long, rf_short, rf_upward, rf_downward = self._calculate_range_filter_signals(
                closes[i], h_band, l_band, filt
            )

            self.completed_bars.append({
                "start_time": historical_kbars.iloc[i]["start_time"],
                "open": historical_kbars.iloc[i]["open"],
                "high": historical_kbars.iloc[i]["high"],
                "low": historical_kbars.iloc[i]["low"],
                "close": historical_kbars.iloc[i]["close"],
                "volume": historical_kbars.iloc[i]["volume"],
                "ma5": ma5,
                "ma10": ma10,
                "ma20": ma20,
                "ema12": ema12,
                "ema26": ema26,
                "bb_upper": upper,
                "bb_lower": lower,
                "K": k,
                "D": d,
                "RSI": rsi,
                "BB_STATE": 0,
                "state_ma20_low": 0,
                "state_ma20_high": 0,
                "min_between_gold_cross1": None,
                "min_between_gold_cross2": None,
                "max_between_dead_cross1": None,
                "max_between_dead_cross2": None,
                "rf_high_band": h_band,
                "rf_low_band": l_band,
                "rf_filter": filt,
                "rf_upward": rf_upward,
                "rf_downward": rf_downward,
                "rf_long_signal": rf_long,
                "rf_short_signal": rf_short
            })

    # ------------------ 更新檢查表並初始化技術指標 ------------------
    def update_combined_check(self, historical_kbars):
        try:
            check_kbars = pd.read_csv('check_kbars.csv')
            combined_check = pd.concat([historical_kbars, check_kbars], ignore_index=True)
        except FileNotFoundError:
            combined_check = historical_kbars.copy()

        combined_check["start_time"] = pd.to_datetime(combined_check["start_time"])
        combined_check = combined_check.sort_values("start_time").reset_index(drop=True)

        closes, highs, lows, k_list = [], [], [], []

        for i, row in combined_check.iterrows():
            close, high, low = row["close"], row["high"], row["low"]
            closes.append(close)
            highs.append(high)
            lows.append(low)

            # MA5, MA10, MA20
            ma5 = self._calculate_ma(closes, 5)
            ma10 = self._calculate_ma(closes, 10)
            ma20 = self._calculate_ma(closes, 20)

            # EMA12 and EMA26
            ema12 = self._calculate_ema(closes, 12)
            ema26 = self._calculate_ema(closes, 26)

            # BB
            if i + 1 >= self.bb_period:
                window = closes[-self.bb_period:]
                bb_ma = sum(window)/self.bb_period
                std = math.sqrt(sum((c-bb_ma)**2 for c in window)/self.bb_period)
            else:
                bb_ma = sum(closes)/len(closes)
                std = math.sqrt(sum((c-bb_ma)**2 for c in closes)/len(closes))
            upper = bb_ma + self.bb_std*std
            lower = bb_ma - self.bb_std*std

            # KD
            if i + 1 >= self.kd_period:
                low_min = min(lows[-self.kd_period:])
                high_max = max(highs[-self.kd_period:])
            else:
                low_min = min(lows)
                high_max = max(highs)
            k = 100*(close-low_min)/(high_max-low_min) if high_max!=low_min else 50
            d = sum(k_list[-(self.kd_d_period-1):]+[k])/self.kd_d_period if i>=self.kd_d_period-1 else k
            k_list.append(k)

            # RSI calculation
            rsi = self._calculate_rsi(closes)

            # Range Filter
            range_size = self._range_size(closes, self.rf_multiplier, self.rf_period)

            if i > 0 and "rf_filter" in combined_check.columns:
                previous_filter = combined_check.loc[i-1, "rf_filter"]
            else:
                previous_filter = None

            h_band, l_band, filt = self._range_filter(close, range_size, previous_filter)

            combined_check.loc[i,"ma5"]=ma5
            combined_check.loc[i,"ma10"]=ma10
            combined_check.loc[i,"ma20"]=ma20
            combined_check.loc[i,"ema12"]=ema12
            combined_check.loc[i,"ema26"]=ema26
            combined_check.loc[i,"bb_upper"]=upper
            combined_check.loc[i,"bb_lower"]=lower
            combined_check.loc[i,"K"]=k
            combined_check.loc[i,"D"]=d
            combined_check.loc[i,"RSI"]=rsi
            combined_check.loc[i,"rf_high_band"]=h_band
            combined_check.loc[i,"rf_low_band"]=l_band
            combined_check.loc[i,"rf_filter"]=filt

        self.check_kbars = combined_check.set_index("start_time")

    # ------------------ 計算 MA/BB/KD/RSI/RF ------------------
    def _calculate_ma_bb_kd_rsi_rf(self, current_close, current_high, current_low):
        if not self.calc_technical_indicators:
                return (None, None, None, None, None,
                        None, None, None, None, None,
                        None, None, None, False, False, 0, 0)
        closes = [bar["close"] for bar in self.completed_bars[-(self.bb_period-1):]] + [current_close]
        highs = [bar["high"] for bar in self.completed_bars[-(self.kd_period-1):]] + [current_high]
        lows = [bar["low"] for bar in self.completed_bars[-(self.kd_period-1):]] + [current_low]

        # MA5, MA10, MA20
        ma5 = self._calculate_ma(closes, 5)
        ma10 = self._calculate_ma(closes, 10)
        ma20 = self._calculate_ma(closes, 20)

        # EMA12 and EMA26 calculation
        ema12 = self._calculate_ema(closes, 12)
        ema26 = self._calculate_ema(closes, 26)

        # BB calculation
        bb_ma = sum(closes[-self.bb_period:])/len(closes[-self.bb_period:])
        std = math.sqrt(sum((c-bb_ma)**2 for c in closes[-self.bb_period:])/len(closes[-self.bb_period:]))
        upper = bb_ma + self.bb_std*std
        lower = bb_ma - self.bb_std*std

        # KD calculation
        low_min = min(lows[-self.kd_period:])
        high_max = max(highs[-self.kd_period:])
        k = 100*(current_close-low_min)/(high_max-low_min) if high_max!=low_min else 50
        d = sum([bar["K"] for bar in self.completed_bars[-(self.kd_d_period-1):]]+[k])/self.kd_d_period if len(self.completed_bars)>=self.kd_d_period-1 else k

        # RSI calculation
        rsi = self._calculate_rsi(closes)

        # Range Filter calculation
        price_window = closes
        range_size = self._range_size(price_window, self.rf_multiplier, self.rf_period)

        if len(self.completed_bars) > 0:
            previous_filter = self.completed_bars[-1].get("rf_filter")
        else:
            previous_filter = None

        h_band, l_band, filt = self._range_filter(current_close, range_size, previous_filter)

        # Calculate Range Filter signals
        rf_long, rf_short, rf_upward, rf_downward = self._calculate_range_filter_signals(
            current_close, h_band, l_band, filt
        )

        return ma5, ma10, ma20, ema12, ema26, upper, lower, k, d, rsi, h_band, l_band, filt, rf_long, rf_short, rf_upward, rf_downward

    # ------------------ 交易策略 ------------------

    def _entry_signal(self, bar):
        if self.position != 0 or len(self.completed_bars) == 0:
            return 0

        prev_bar = self.completed_bars[-1]
        # prev_bar2 = self.completed_bars[-2]

        def safe_check(name, expr):
            try:
                result = expr()
                if result is None or (isinstance(result, float) and pd.isna(result)):
                    print(f"[NA/NONE] {name} = {result}")
                    return False
                return result
            except Exception as e:
                return False



        # KD + RSI 多單
        kd_rsi_long = (
            safe_check("pre_tick_price < prev_bar[GOLDEN_LAST_1_LOW]", lambda: prev_bar["close"] <= self.key_entry_price) and
            safe_check("pre_tick_price < prev_bar[GOLDEN_LAST_1_LOW]", lambda: self.pre_tick_price < self.key_entry_price) and
            safe_check("bar[close] >= prev_bar[rf_filter]", lambda: bar["close"] >= self.key_entry_price)
        )

        if kd_rsi_long:
            # self.key_exit_price = self.key_reset_price
            return 1
        # if self.key_reset_price is not None:
        #     if ((bar["low"]<self.key_reset_price) or
        #         (bar["high"] >self.key_entry_price)):
        #         self.key_entry_price = None
        #         self.key_reset_price  = None
        #         self.key_exit_price  = None
        return 0

    # ------------------ 出場判斷 ------------------
    def _exit_signal(self, pnl, bar):
        if self.position==0 or self.entry_price is None:
            return None
        pnl_pct = pnl/self.entry_price*100
        self.max_pnl_pct = max(getattr(self,"max_pnl_pct",pnl_pct),pnl_pct)
        self.max_pnl=max(getattr(self,"max_pnl",pnl),pnl)

        if self.position==1 and bar["close"]<self.key_reset_price:
            if pnl<0:
                return "STOP_LOSS"
            else:
                return "TAKE_PROFIT_0"


        if self.max_pnl >= (self.key_exit_price-self.entry_price)*2 and pnl <= self.max_pnl*0.8 :
            return "TAKE_PROFIT_2"
        if self.max_pnl >= (self.key_exit_price-self.entry_price)*1 and pnl <= self.max_pnl*0.2 :#and bar["close"]<bar["ma20"]:
            return "TAKE_PROFIT_1"



        return None

    # ------------------ 持倉控管 ------------------
    def _manage_position(self, bar):
        tick_time = bar["tick_time"]
        current_hour = tick_time.hour
        current_minute = tick_time.minute
        current_time = current_hour * 100 + current_minute

        restricted_periods = [
            (430, 900),
            (1300, 1515)
        ]
        force_close_times = [430, 1300]

        in_restricted_period = False
        for start, end in restricted_periods:
            if start <= current_time <= end:
                in_restricted_period = True
                break

        # Forced exit at 04:30 or 13:00
        # if current_time in force_close_times:
        if in_restricted_period:
            if self.position != 0:
                pnl = (bar["close"] - self.entry_price) * self.position
                self.exit_price = bar["close"]
                holding_time = (tick_time - self.entry_time).total_seconds()

                self.trade_log.append({
                    "tick_time": tick_time,
                    "action": "FORCE_CLOSE",
                    "entry_price": self.entry_price,
                    "exit_price": self.exit_price,
                    "PnL": pnl,
                    "holding_time_sec": holding_time
                })
                self.position = 0
                self.max_pnl_pct = 0
                self.max_pnl=0
                self.entry_price = None
                self.exit_price = None
                self.entry_time = None
                if self.debug:
                    print(f"[TRADE] {tick_time} 強制平倉 @ {self.exit_price}  PnL={pnl:.2f} 持倉秒數={holding_time:.2f}")



            # return


        # if in_restricted_period:
            self.key_exit_price = None
            self.key_reset_price = None
            self.key_entry_price = None
            self.bar_stream=[]
            self.up_stream=[]
            self.down_stream=[]
            self.inertia=[]
            self.bar_pattern=[]

        # Entry logic
        elif self.position == 0:
            signal = self._entry_signal(bar)
            if signal != 0 and abs(self.position) < self.max_position:
                # if self.current_bar_trade_info["stop_loss_count"] >= 2:
                #     return

                self.position = signal
                self.entry_price = bar["close"]
                self.entry_time = bar["tick_time"]
                self.max_pnl_pct = 0
                self.trade_log.append({
                    "tick_time": bar["tick_time"],
                    "action": "BUY" if signal == 1 else "SELL",
                    "price": bar["close"],
                    'K': bar['K'],
                    'D': bar['D'],
                    'RSI': bar['RSI'],
                    'ma5': bar['ma5'],
                    'ma10': bar['ma10'],
                    'ma20': bar['ma20'],
                    'ema12': bar['ema12'],
                    'ema26': bar['ema26'],
                    'k_state': self.k_state,
                    'last_bb_state': self.last_bb_state,
                    'rollback_rate': self.rollback_rate,
                    "PnL": 0
                })
                if self.debug:
                    print(f"[TRADE] {bar['tick_time']} 進場 {'多單' if signal == 1 else '空單'} @ {bar['close']}")

        elif self.position != 0:
            # Exit logic
            pnl = (bar["close"] - self.entry_price) * self.position
            exit_action = self._exit_signal(pnl, bar)
            if exit_action:
                self.exit_price = bar["close"]
                holding_time = (bar["tick_time"] - self.entry_time).total_seconds()

                self.trade_log.append({
                    "tick_time": bar["tick_time"],
                    "action": exit_action,
                    "entry_price": self.entry_price,
                    "exit_price": self.exit_price,
                    "PnL": pnl,
                    "holding_time_sec": holding_time
                })

                if self.debug:
                    print(f"[TRADE] {bar['tick_time']} 平倉 ({exit_action}) @ {self.exit_price}  PnL={pnl:.0f} 持倉秒數={holding_time:.0f}")

                self.position = 0
                self.max_pnl_pct = 0
                self.max_pnl=0
                self.entry_price = None
                self.exit_price = None
                self.entry_time = None
                self.key_exit_price = None
                self.key_reset_price = None
                self.key_entry_price = None
                # self.key_exit_price2 = None
                # self.key_reset_price2 = None
                # self.key_entry_price2 = None

    # ------------------ Tick 處理 ------------------
    def process_tick(self, tick):
        tick_time = pd.to_datetime(tick["time"])
        price = tick["price"]
        volume = tick["volume"]
        bar_time = self._get_bar_time(tick_time)

        if self.current_bar is None or bar_time != self.current_bar["start_time"]:
            if self.current_bar is not None:
                self._finalize_bar(self.current_bar)
            self.current_bar = {
                "start_time": bar_time,
                "open": price,
                "high": price,
                "low": price,
                "close": price,
                "volume": volume,
                "ma5": None,
                "ma10": None,
                "ma20": None,
                "ema12": None,
                "ema26": None,
                "bb_upper": None,
                "bb_lower": None,
                "K": None,
                "D": None,
                "RSI": None,
                "BB_STATE": self.last_bb_state,
                "state_ma20_low": self.state_ma20_low,
                "state_ma20_high": self.state_ma20_high,
                "min_between_gold_cross1": None,
                "min_between_gold_cross2": None,
                "max_between_dead_cross1": None,
                "max_between_dead_cross2": None,
                "rf_high_band": None,
                "rf_low_band": None,
                "rf_filter": None,
                "rf_upward": None,
                "rf_downward": None,
                "rf_long_signal": False,
                "rf_short_signal": False
            }

            self.current_bar_trade_info = {
                "max_buy_price": None,
                "min_sell_price": None,
                "stop_loss_count": 0
            }
        else:
            self.current_bar["high"] = max(self.current_bar["high"], price)
            self.current_bar["low"] = min(self.current_bar["low"], price)
            self.current_bar["close"] = price
            self.current_bar["volume"] += volume

        ma5, ma10, ma20, ema12, ema26, ub, lb, k, d, rsi, rf_h_band, rf_l_band, rf_filt, rf_long, rf_short, rf_upward, rf_downward = self._calculate_ma_bb_kd_rsi_rf(
            self.current_bar["close"], self.current_bar["high"], self.current_bar["low"]
        )

        temp_bar = {
            "tick_time": tick_time,
            "high": self.current_bar["high"],
            "low": self.current_bar["low"],
            "close": price,
            "K": k,
            "D": d,
            "RSI": rsi,
            "ma5": ma5,
            "ma10": ma10,
            "ma20": ma20,
            "ema12": ema12,
            "ema26": ema26,
            "bar_bb_upper": ub,
            "bar_bb_lower": lb,
            "rf_high_band": rf_h_band,
            "rf_low_band": rf_l_band,
            "rf_filter": rf_filt,
            "rf_long_signal": rf_long,
            "rf_short_signal": rf_short,
            "rf_upward": rf_upward,
            "rf_downward": rf_downward
        }
        self._manage_position(temp_bar)

        self.tick_logs.append({
            "tick_time": tick_time,
            "price": price,
            "volume": volume,
            "bar_time": bar_time,
            "bar_open": self.current_bar["open"],
            "bar_high": self.current_bar["high"],
            "bar_low": self.current_bar["low"],
            "bar_close": self.current_bar["close"],
            "bar_volume": self.current_bar["volume"],
            "bar_ma5": ma5,
            "bar_ma10": ma10,
            "bar_ma20": ma20,
            "bar_ema12": ema12,
            "bar_ema26": ema26,
            "bar_bb_upper": ub,
            "bar_bb_lower": lb,
            "bar_K": k,
            "bar_D": d,
            "bar_RSI": rsi,
            "BB_STATE": self.last_bb_state,
            "state_ma20_low": self.state_ma20_low,
            "state_ma20_high": self.state_ma20_high,
            "rf_high_band": rf_h_band,
            "rf_low_band": rf_l_band,
            "rf_filter": rf_filt,
            "rf_upward": rf_upward,
            "rf_downward": rf_downward,
            "rf_long_signal": rf_long,
            "rf_short_signal": rf_short
        })
        self.pre_tick_price = price

    # ------------------ 完成 K 棒 ------------------
    def _finalize_bar(self, bar):
        last_tick_data = self.tick_logs[-1] if self.tick_logs else {}
        bar["ma5"] = last_tick_data.get("bar_ma5")
        bar["ma10"] = last_tick_data.get("bar_ma10")
        bar["ma20"] = last_tick_data.get("bar_ma20")
        bar["ema12"] = last_tick_data.get("bar_ema12")
        bar["ema26"] = last_tick_data.get("bar_ema26")
        bar["bb_upper"] = last_tick_data.get("bar_bb_upper")
        bar["bb_lower"] = last_tick_data.get("bar_bb_lower")
        bar["K"] = last_tick_data.get("bar_K")
        bar["D"] = last_tick_data.get("bar_D")
        bar["RSI"] = last_tick_data.get("bar_RSI")

        # Update Range Filter values
        bar["rf_high_band"] = last_tick_data.get("rf_high_band")
        bar["rf_low_band"] = last_tick_data.get("rf_low_band")
        bar["rf_filter"] = last_tick_data.get("rf_filter")
        bar["rf_upward"] = last_tick_data.get("rf_upward")
        bar["rf_downward"] = last_tick_data.get("rf_downward")
        bar["rf_long_signal"] = last_tick_data.get("rf_long_signal", False)
        bar["rf_short_signal"] = last_tick_data.get("rf_short_signal", False)



        # bar_time = bar["start_time"]
        # bar_time_hour = bar_time.hour
        # bar_time_minute = bar_time.minute
        # bar_time = bar_time_hour * 100 + bar_time_minute
        # force_clear_times = [850, 1505]
        # if bar_time in force_clear_times:
        #     self.bar_stream=[]
        #     self.bar_pattern=[]
        #     self.fail_stream=[]
        #     self.key_entry_price = None
        #     self.key_reset_price  = None
        #     self.key_exit_price  = None


        if not self.day_highs or bar["high"] > self.day_highs[-1]["price"]:
            if self.day_highs and self.day_highs[-1]["time"]==self.completed_bars[-1]["start_time"]:
                self.day_highs[-1]={"time": bar["start_time"], "price": bar["high"]}
            else:
                self.day_highs.append({"time": bar["start_time"], "price": bar["high"]})

        if not self.day_lows or bar["low"] < self.day_lows[-1]["price"]:
            if self.day_lows and self.day_lows[-1]["time"]==self.completed_bars[-1]["start_time"]:
                self.day_lows[-1]={"time": bar["start_time"], "price": bar["low"]}
            else:
                self.day_lows.append({"time": bar["start_time"], "price": bar["low"]})



        if len(self.completed_bars) >= 2:
            bar1 = self.completed_bars[-1]
            bar2 = self.completed_bars[-2]

            bar_data = {
                "time": bar["start_time"],
                "open": bar["open"],
                "high": bar["high"],
                "low": bar["low"],
                "close": bar["close"],
                "ma20": bar["ma20"]
            }
            bar1_data = {
                "time": bar1["start_time"],
                "open": bar1["open"],
                "high": bar1["high"],
                "low": bar1["low"],
                "close": bar1["close"],
                "ma20": bar1["ma20"]
            }
            # prev_bar2_data = {
            #     "time": prev_bar2["start_time"],
            #     "open": prev_bar2["open"],
            #     "high": prev_bar2["high"],
            #     "low": prev_bar2["low"],
            #     "close": prev_bar2["close"],
            #     "ma20": prev_bar2["ma20"]
            # }

            def flush_stream(stream, fail_type):
                if not stream:
                    return

                self.bar_stream.append({
                    "start_time": stream[0]["time"],
                    "end_time": stream[-1]["time"],
                    "FAIL": fail_type,
                    "open": stream[0]["open"],
                    "high": stream[0]["high"],
                    "low": stream[0]["low"],
                    "close": stream[0]["close"],
                    "ma20": stream[0]["ma20"],
                    "interval_high": max(d["high"] for d in stream),
                    "interval_low": min(d["low"] for d in stream),
                    "end_open": stream[-2]["open"],
                    "end_high": stream[-2]["high"],
                    "end_low": stream[-2]["low"],
                    "end_close": stream[-2]["close"],
                    "power": abs(stream[0]["close"] - stream[-2]["open"]) ,
                    "range":max(d["high"] for d in stream)-min(d["low"] for d in stream),
                    'speed':len(stream),
                    "reverse_open": stream[-1]["open"],
                    "reverse_high": stream[-1]["high"],
                    "reverse_low": stream[-1]["low"],
                    "reverse_close": stream[-1]["close"]
                })
                print(f"{fail_type},start_{stream[0]["time"]},end_{stream[-1]["time"]}_ilow{min(d["low"] for d in stream)}_ihigh{max(d["high"] for d in stream)}")
                print(len(self.bar_stream))
                if len(self.bar_stream)>=3:
                    s3, s2, s1 = self.bar_stream[-3:]
                    time_match = s3['end_time'] == s1['start_time']

                    if fail_type == 'NEW_LOW':
                        interval_cond = s2['interval_high'] < s3['interval_high']
                    else:
                        interval_cond = s2['interval_low'] > s3['interval_low']

                    if time_match and interval_cond:

                        merged = {
                            "start_time": s3["start_time"],
                            "end_time": s1["end_time"],
                            "FAIL": fail_type,

                            "open": s3["open"],
                            "high": s3["high"],
                            "low": s3["low"],
                            "close": s1["close"],

                            "interval_high": max(
                                s1["interval_high"],
                                s2["interval_high"],
                                s3["interval_high"]
                            ),
                            "interval_low": min(
                                s1["interval_low"],
                                s2["interval_low"],
                                s3["interval_low"]
                            ),

                            # 倒數第二段
                            "end_open": s3["end_open"],
                            "end_high": s3["end_high"],
                            "end_low": s3["end_low"],
                            "end_close": s3["end_close"],

                            # 動能與速度
                            "power": abs(s1["close"] - s3["open"]),
                            "range": (
                                max(s1["interval_high"], s2["interval_high"], s3["interval_high"])
                                - min(s1["interval_low"], s2["interval_low"], s3["interval_low"])
                            ),
                            "speed": s1["speed"] + s2["speed"] + s3["speed"],

                            # 反轉K（最新）
                            "reverse_open": s3["reverse_open"],
                            "reverse_high": s3["reverse_high"],
                            "reverse_low": s3["reverse_low"],
                            "reverse_close": s3["reverse_close"],
                        }

                        # 移除舊的三段，加入新的一段
                        self.bar_stream = self.bar_stream[:-3]
                        self.bar_stream.append(merged)
                        print(f'吞噬合併{fail_type}：start_time_{merged['start_time']}_end_time{merged['end_time']}')
                        print(len(self.bar_stream))
                # if len(self.bar_stream)>=5 and fail_type=='NEW_HIGH':
                #     s5,s4,s3, s2, s1 = self.bar_stream[-5:]
                #     print(f"********S5{s5["start_time"]}_S3{s3["start_time"]}_S1{s1["start_time"]}")
            if bar["close"] < bar["open"] and bar["low"] < bar1["low"]:
                if self.up_stream != []:
                    self.up_stream.append(bar_data)
                flush_stream(self.up_stream, "NEW_HIGH")
                self.up_stream.clear()
                if self.down_stream == []:
                    self.down_stream.append(bar1_data)
                self.down_stream.append(bar_data)

            elif bar["close"] > bar["open"] and bar["high"] > bar1["high"]:
                if self.down_stream != []:
                    self.down_stream.append(bar_data)
                flush_stream(self.down_stream, "NEW_LOW")
                self.down_stream.clear()
                if self.up_stream == []:
                    self.up_stream.append(bar1_data)
                self.up_stream.append(bar_data)

            else:
                (self.up_stream if self.up_stream else self.down_stream).append(bar_data)

        inertia_type = None

        if len(self.bar_stream) >= 5:
            streams = self.bar_stream[-5:]
            stream1, stream2, stream3, stream4, stream5 = streams[::-1]
            # self.bar_stream = self.bar_stream[-4:]  # []


            if (
                (stream1["FAIL"] == 'NEW_HIGH') and
                # (stream1["start_time"] == stream3["end_time"]) and
                (stream1["interval_high"] < stream3["interval_high"]) and
                (stream3["interval_high"] <= stream5["interval_high"]) #and
                # (stream2["interval_low"] > stream4["interval_low"]) #and
                # (stream3["interval_low"] < stream5["interval_low"]) and
                # (stream3["low"] < stream5["low"]) #and
                # (stream3["interval_low"] > stream2["interval_low"]) #and
                # (stream2["range"] < stream4["range"])
            ):
                inertia_type='減速下跌'

            else:
                inertia_type='非減速下跌'

        if inertia_type is not None:
            inertia = {
                "start_time": streams[0]["start_time"],
                "end_time": streams[-1]["end_time"],
                "type": inertia_type,
                "power": abs(streams[-1]["close"] - streams[0]["open"]),
                "range": max(d["high"] for d in streams) - min(d["low"] for d in streams),
                "speed": sum(d["speed"] for d in streams),
                "center_high": streams[-2]["high"],
                "center_low": streams[-2]["interval_low"],
                "start_high": streams[0]["high"],
                "start_low": streams[0]["low"],
                "end_high": streams[-1]["end_high"],
                "end_low": streams[-1]["end_low"],
                "inertia_low": min(d["low"] for d in streams)
            }
            self.inertia.append(inertia)
            print(inertia["start_time"],inertia["end_time"], inertia["type"])

        pattern_type=None
        # new_pattern={}
        if len(self.completed_bars) >= 2:
            bar1 = self.completed_bars[-1]
            bar2 = self.completed_bars[-2]

            if len(self.inertia) >= 1:
                inertia1 = self.inertia[-1]
                # pattern1=self.bar_pattern[-1]
                # pattern2=self.bar_pattern[-2]
                if (
                    # self.position==0 and

                    (inertia1["type"] in ['減速下跌'] ) #and
                    # (pattern1["type"] in ['雙跌'] ) and
                    # (inertia1["interval_low"] < pattern1["low"])
                    # (pattern2["type"] == '標準底' ) and
                    # (pattern2["right_power"] >= pattern1["right_power"])

                    ) :
                    # if self.position==0:
                    self.key_entry_price = inertia1["end_high"]
                    self.key_reset_price  = inertia1["center_low"]
                    self.key_exit_price  = inertia1["center_high"]-inertia1["center_low"]+inertia1["end_high"]
                    # action=f'{pattern1["type"]}_設定'
                    action='設定'
                    print(f"{action}_key_entry{self.key_entry_price}_key_reset{self.key_reset_price}__key_exit{self.key_exit_price}")


                    new_key_log = {
                        "start_time": bar["start_time"],
                        'action': action,
                        "enter_price": self.key_entry_price,
                        "reset_price": self.key_reset_price,
                        "exit_price": self.key_exit_price
                    }

                    self.key_price_log.append(new_key_log)

        # if  self.key_reset_price is not None:
        #     if self.position==0:
        #         if (
        #         (bar["close"]<self.key_reset_price) or
        #         (bar["close"] >self.key_entry_price)
        #           ):
        #           self.key_entry_price = None
        #           self.key_reset_price  = None
        #           self.key_exit_price  = None

        # Store completed bar
        print(f"pnl{self.max_pnl}")
        self.completed_bars.append(bar.copy())

    def _get_bar_time(self, tick_time):
        interval_min=self.interval.seconds//60
        if self.label=="right":
            minute_offset=(tick_time.minute//interval_min+1)*interval_min
            return tick_time.replace(minute=0,second=0,microsecond=0)+timedelta(minutes=minute_offset)
        else:
            return tick_time-timedelta(minutes=tick_time.minute%interval_min,
                                      seconds=tick_time.second,microseconds=tick_time.microsecond)

    # ------------------ 運行 ------------------
    def run(self, tick_data, is_first_file=False):
        """
        處理 tick 數據並生成 K 線

        Args:
            tick_data: DataFrame of ticks
            is_first_file: 是否是第一個檔案（影響初始化邏輯）
        """
        start_time = time.time()

        if not is_first_file and len(self.completed_bars) > 0:
            print(f"  [續跑] 已有 {len(self.completed_bars)} 根K線")

        # 處理所有 ticks
        for idx, tick in tick_data.iterrows():
            try:
                self.process_tick(tick)
            except Exception as e:
                print(f"  ⚠ Tick {idx} 處理失敗: {e}")
                raise e

        # 完成最後一根 K 線
        if self.current_bar:
            try:
                self._finalize_bar(self.current_bar)
                self.current_bar = None
            except Exception as e:
                print(f"  ⚠ 最終K線處理失敗: {e}")

        elapsed_time = time.time() - start_time

        print(f"  [完成] 處理 {len(tick_data)} 筆ticks，耗時 {elapsed_time:.2f}s")
        print(f"  [結果] 現有 {len(self.completed_bars)} 根K線")
        self.bar_stream=[]

        return {
            "kbars": pd.DataFrame(self.completed_bars),
            "tick_logs": pd.DataFrame(self.tick_logs) if self.tick_logs else None
        }

##無指標

In [ ]:
import pandas as pd
from datetime import timedelta
import math
import time


class KBarSimulator:
    def __init__(self, historical_kbars,historical_kbars60, interval_minutes=5, debug=True, label="right",
                 max_position=1, calc_technical_indicators=True):
        """
        初始化 KBarSimulator

        Args:
            historical_kbars: 歷史K線 DataFrame (必須包含 start_time, open, high, low, close, volume)
            interval_minutes: K線週期 (分鐘)
            debug: 是否輸出除錯訊息
            label: "right" (右標籤) 或 "left"
            max_position: 最大持倉數 (預設1)
            calc_technical_indicators: 是否計算技術指標 (MA/EMA)
        """
        self.calc_technical_indicators = calc_technical_indicators
        self.interval = timedelta(minutes=interval_minutes)
        self.debug = debug
        self.label = label

        self.completed_bars = []
        self.tick_logs = []
        self.current_bar = None

        # === 交易控管 ===
        self.position = 0
        self.entry_price = None
        self.exit_price = None
        self.entry_time = None
        self.trade_log = []
        self.max_position = max_position
        self.max_pnl_pct = 0
        self.max_pnl = 0

        # === 自定義結構波與慣性相關變數 ===
        self.bar_stream = []
        self.up_stream = []
        self.down_stream = []
        self.inertia = []

        self.pre_tick_price = None

        self.key_entry_price = None
        self.key_reset_price = None
        self.key_exit_price = None
        self.key_price_log = []

        self.day_highs = []
        self.day_lows = []

        # 初始化歷史完成的K棒（只計算 MA5/10/20 與 EMA12/26）
        self._initialize_completed_bars(historical_kbars)

    # ------------------ MA / EMA 計算 ------------------
    def _calculate_ma(self, closes, period):
        if len(closes) < period:
            return sum(closes) / len(closes) if closes else None
        return sum(closes[-period:]) / period

    def _calculate_ema(self, closes, period):
        if len(closes) == 0:
            return None
        alpha = 2.0 / (period + 1)
        ema = closes[0]
        for price in closes[1:]:
            ema = alpha * price + (1 - alpha) * ema
        return ema

    # ------------------ 初始化歷史K棒 ------------------
    def _initialize_completed_bars(self, historical_kbars):
        closes = historical_kbars["close"].tolist()
        for i in range(len(historical_kbars)):
            ma5 = self._calculate_ma(closes[:i + 1], 5)
            ma10 = self._calculate_ma(closes[:i + 1], 10)
            ma20 = self._calculate_ma(closes[:i + 1], 20)
            ema12 = self._calculate_ema(closes[:i + 1], 12)
            ema26 = self._calculate_ema(closes[:i + 1], 26)

            self.completed_bars.append({
                "start_time": historical_kbars.iloc[i]["start_time"],
                "open": historical_kbars.iloc[i]["open"],
                "high": historical_kbars.iloc[i]["high"],
                "low": historical_kbars.iloc[i]["low"],
                "close": historical_kbars.iloc[i]["close"],
                "volume": historical_kbars.iloc[i]["volume"],
                "ma5": ma5,
                "ma10": ma10,
                "ma20": ma20,
                "ema12": ema12,
                "ema26": ema26,
            })

    # ------------------ 計算當前 bar 的 MA/EMA ------------------
    def _calculate_ma_ema(self, current_close):
        if not self.calc_technical_indicators:
            return None, None, None, None, None

        # 取足夠長度的收盤價來計算 MA20
        closes = [bar["close"] for bar in self.completed_bars[-30:]] + [current_close]

        ma5 = self._calculate_ma(closes, 5)
        ma10 = self._calculate_ma(closes, 10)
        ma20 = self._calculate_ma(closes, 20)
        ema12 = self._calculate_ema(closes, 12)
        ema26 = self._calculate_ema(closes, 26)

        return ma5, ma10, ma20, ema12, ema26

    # ------------------ 進場訊號 ------------------
    def _entry_signal(self, bar):
        if self.position != 0:
            return 0

        # 進場條件：價格突破 key_entry_price（由慣性減速下跌形態設定）
        if self.key_entry_price is not None and bar["close"] >= self.key_entry_price:
            return 1  # 多單

        return 0

    # ------------------ 出場訊號 ------------------
    def _exit_signal(self, pnl, bar):
        if self.position == 0 or self.entry_price is None:
            return None

        self.max_pnl = max(self.max_pnl, pnl)
        self.max_pnl_pct = max(self.max_pnl_pct, pnl / self.entry_price * 100)

        # 停損：跌破 reset 價
        if self.position == 1 and self.key_reset_price is not None and bar["close"] < self.key_reset_price:
            return "STOP_LOSS" if pnl < 0 else "TAKE_PROFIT_0"

        # 動態停利
        if self.key_exit_price is not None:
            target = (self.key_exit_price - self.entry_price) * self.position
            if self.max_pnl >= target * 2 and pnl <= self.max_pnl * 0.8:
                return "TAKE_PROFIT_2"
            if self.max_pnl >= target and pnl <= self.max_pnl * 0.2:
                return "TAKE_PROFIT_1"

        return None

    # ------------------ 持倉管理 ------------------
    def _manage_position(self, bar):
        tick_time = bar["tick_time"]
        current_time = tick_time.hour * 100 + tick_time.minute
        restricted_periods = [(430, 900), (1300, 1515)]

        in_restricted = any(start <= current_time <= end for start, end in restricted_periods)

        # 強制平倉時段
        if in_restricted and self.position != 0:
            pnl = (bar["close"] - self.entry_price) * self.position
            holding_time = (tick_time - self.entry_time).total_seconds()
            self.trade_log.append({
                "tick_time": tick_time,
                "action": "FORCE_CLOSE",
                "entry_price": self.entry_price,
                "exit_price": bar["close"],
                "PnL": pnl,
                "holding_time_sec": holding_time
            })
            if self.debug:
                print(f"[TRADE] {tick_time} 強制平倉 @ {bar['close']} PnL={pnl:.2f}")
            self._reset_position()
            self.bar_stream = []
            self.up_stream = []
            self.down_stream = []
            self.inertia = []
            return

        # 進場
        if self.position == 0:
            signal = self._entry_signal(bar)
            if signal != 0 and abs(self.position) < self.max_position:
                self.position = signal
                self.entry_price = bar["close"]
                self.entry_time = tick_time
                self.max_pnl = self.max_pnl_pct = 0
                self.trade_log.append({
                    "tick_time": tick_time,
                    "action": "BUY" if signal == 1 else "SELL",
                    "price": bar["close"],
                    "PnL": 0
                })
                if self.debug:
                    print(f"[TRADE] {tick_time} 進場 {'多單' if signal == 1 else '空單'} @ {bar['close']}")

        # 出場
        else:
            pnl = (bar["close"] - self.entry_price) * self.position
            action = self._exit_signal(pnl, bar)
            if action:
                holding_time = (tick_time - self.entry_time).total_seconds()
                self.trade_log.append({
                    "tick_time": tick_time,
                    "action": action,
                    "entry_price": self.entry_price,
                    "exit_price": bar["close"],
                    "PnL": pnl,
                    "holding_time_sec": holding_time
                })
                if self.debug:
                    print(f"[TRADE] {tick_time} 平倉 ({action}) @ {bar['close']} PnL={pnl:.0f}")
                self._reset_position()

    def _reset_position(self):
        self.position = 0
        self.entry_price = self.exit_price = self.entry_time = None
        self.max_pnl = self.max_pnl_pct = 0
        self.key_entry_price = self.key_reset_price = self.key_exit_price = None





    # ------------------ Tick 處理 ------------------
    def process_tick(self, tick):
        tick_time = pd.to_datetime(tick["time"])
        price = tick["price"]
        volume = tick["volume"]
        bar_time = self._get_bar_time(tick_time)

        # 新K棒開始
        if self.current_bar is None or bar_time != self.current_bar["start_time"]:
            if self.current_bar is not None:
                self._finalize_bar(self.current_bar)
            self.current_bar = {
                "start_time": bar_time,
                "open": price,
                "high": price,
                "low": price,
                "close": price,
                "volume": volume,
                "ma5": None, "ma10": None, "ma20": None,
                "ema12": None, "ema26": None,
            }

        # 更新當前K棒
        self.current_bar["high"] = max(self.current_bar["high"], price)
        self.current_bar["low"] = min(self.current_bar["low"], price)
        self.current_bar["close"] = price
        self.current_bar["volume"] += volume

        # 計算即時指標
        ma5, ma10, ma20, ema12, ema26 = self._calculate_ma_ema(price)

        temp_bar = {
            "tick_time": tick_time,
            "close": price,
            "high": self.current_bar["high"],
            "low": self.current_bar["low"],
            "ma5": ma5, "ma10": ma10, "ma20": ma20,
            "ema12": ema12, "ema26": ema26,
        }
        self._manage_position(temp_bar)

        # 記錄 tick log
        self.tick_logs.append({
            "tick_time": tick_time,
            "price": price,
            "volume": volume,
            "bar_time": bar_time,
            "bar_open": self.current_bar["open"],
            "bar_high": self.current_bar["high"],
            "bar_low": self.current_bar["low"],
            "bar_close": price,
            "bar_volume": self.current_bar["volume"],
            "bar_ma5": ma5, "bar_ma10": ma10, "bar_ma20": ma20,
            "bar_ema12": ema12, "bar_ema26": ema26,
        })
        self.pre_tick_price = price

    # ------------------ 完成一根K棒 ------------------
    def _finalize_bar(self, bar):
        last_tick = self.tick_logs[-1] if self.tick_logs else {}
        bar["ma5"] = last_tick.get("bar_ma5")
        bar["ma10"] = last_tick.get("bar_ma10")
        bar["ma20"] = last_tick.get("bar_ma20")
        bar["ema12"] = last_tick.get("bar_ema12")
        bar["ema26"] = last_tick.get("bar_ema26")


        if not self.day_highs or bar["high"] > self.day_highs[-1]["price"]:
            if self.day_highs and self.day_highs[-1]["time"]==self.completed_bars[-1]["start_time"]:
                self.day_highs[-1]={"time": bar["start_time"], "price": bar["high"]}
            else:
                self.day_highs.append({"time": bar["start_time"], "price": bar["high"]})

        if not self.day_lows or bar["low"] < self.day_lows[-1]["price"]:
            if self.day_lows and self.day_lows[-1]["time"]==self.completed_bars[-1]["start_time"]:
                self.day_lows[-1]={"time": bar["start_time"], "price": bar["low"]}
            else:
                self.day_lows.append({"time": bar["start_time"], "price": bar["low"]})



        if len(self.completed_bars) >= 2:
            bar1 = self.completed_bars[-1]
            bar2 = self.completed_bars[-2]

            bar_data = {
                "time": bar["start_time"],
                "open": bar["open"],
                "high": bar["high"],
                "low": bar["low"],
                "close": bar["close"],
                "ma20": bar["ma20"]
            }
            bar1_data = {
                "time": bar1["start_time"],
                "open": bar1["open"],
                "high": bar1["high"],
                "low": bar1["low"],
                "close": bar1["close"],
                "ma20": bar1["ma20"]
            }


            def flush_stream(stream, fail_type):
                if not stream:
                    return

                self.bar_stream.append({
                    "start_time": stream[0]["time"],
                    "end_time": stream[-1]["time"],
                    "FAIL": fail_type,
                    "open": stream[0]["open"],
                    "high": stream[0]["high"],
                    "low": stream[0]["low"],
                    "close": stream[0]["close"],
                    "ma20": stream[0]["ma20"],
                    "interval_high": max(d["high"] for d in stream),
                    "interval_low": min(d["low"] for d in stream),
                    "end_open": stream[-2]["open"],
                    "end_high": stream[-2]["high"],
                    "end_low": stream[-2]["low"],
                    "end_close": stream[-2]["close"],
                    "power": abs(stream[0]["close"] - stream[-2]["open"]) ,
                    "range":max(d["high"] for d in stream)-min(d["low"] for d in stream),
                    'speed':len(stream),
                    "reverse_open": stream[-1]["open"],
                    "reverse_high": stream[-1]["high"],
                    "reverse_low": stream[-1]["low"],
                    "reverse_close": stream[-1]["close"]
                })
                print(f"{fail_type},start_{stream[0]["time"]},end_{stream[-1]["time"]}_ilow{min(d["low"] for d in stream)}_ihigh{max(d["high"] for d in stream)}")
                print(len(self.bar_stream))
                if len(self.bar_stream)>=3:
                    s3, s2, s1 = self.bar_stream[-3:]
                    time_match = s3['end_time'] == s1['start_time']

                    if fail_type == 'NEW_LOW':
                        interval_cond = s2['interval_high'] < s3['interval_high']
                    else:
                        interval_cond = s2['interval_low'] > s3['interval_low']

                    if time_match and interval_cond:

                        merged = {
                            "start_time": s3["start_time"],
                            "end_time": s1["end_time"],
                            "FAIL": fail_type,

                            "open": s3["open"],
                            "high": s3["high"],
                            "low": s3["low"],
                            "close": s1["close"],

                            "interval_high": max(
                                s1["interval_high"],
                                s2["interval_high"],
                                s3["interval_high"]
                            ),
                            "interval_low": min(
                                s1["interval_low"],
                                s2["interval_low"],
                                s3["interval_low"]
                            ),

                            # 倒數第二段
                            "end_open": s3["end_open"],
                            "end_high": s3["end_high"],
                            "end_low": s3["end_low"],
                            "end_close": s3["end_close"],

                            # 動能與速度
                            "power": abs(s1["close"] - s3["open"]),
                            "range": (
                                max(s1["interval_high"], s2["interval_high"], s3["interval_high"])
                                - min(s1["interval_low"], s2["interval_low"], s3["interval_low"])
                            ),
                            "speed": s1["speed"] + s2["speed"] + s3["speed"],

                            # 反轉K（最新）
                            "reverse_open": s3["reverse_open"],
                            "reverse_high": s3["reverse_high"],
                            "reverse_low": s3["reverse_low"],
                            "reverse_close": s3["reverse_close"],
                        }

                        # 移除舊的三段，加入新的一段
                        self.bar_stream = self.bar_stream[:-3]
                        self.bar_stream.append(merged)
                        print(f'吞噬合併{fail_type}：start_time_{merged['start_time']}_end_time{merged['end_time']}')
                        print(len(self.bar_stream))
                # if len(self.bar_stream)>=5 and fail_type=='NEW_HIGH':
                #     s5,s4,s3, s2, s1 = self.bar_stream[-5:]
                #     print(f"********S5{s5["start_time"]}_S3{s3["start_time"]}_S1{s1["start_time"]}")
            if bar["close"] < bar["open"] and bar["low"] < bar1["low"]:
                if self.up_stream != []:
                    self.up_stream.append(bar_data)
                flush_stream(self.up_stream, "NEW_HIGH")
                self.up_stream.clear()
                if self.down_stream == []:
                    self.down_stream.append(bar1_data)
                self.down_stream.append(bar_data)

            elif bar["close"] > bar["open"] and bar["high"] > bar1["high"]:
                if self.down_stream != []:
                    self.down_stream.append(bar_data)
                flush_stream(self.down_stream, "NEW_LOW")
                self.down_stream.clear()
                if self.up_stream == []:
                    self.up_stream.append(bar1_data)
                self.up_stream.append(bar_data)

            else:
                (self.up_stream if self.up_stream else self.down_stream).append(bar_data)

        inertia_type = None

        if len(self.bar_stream) >= 5:
            streams = self.bar_stream[-5:]
            stream1, stream2, stream3, stream4, stream5 = streams[::-1]
            # self.bar_stream = self.bar_stream[-4:]  # []


            if (
                (stream1["FAIL"] == 'NEW_HIGH') and
                # (stream1["start_time"] == stream3["end_time"]) and
                (stream1["interval_high"] < stream3["interval_high"]) and
                (stream3["interval_high"] <= stream5["interval_high"]) #and
                # (stream2["interval_low"] > stream4["interval_low"]) #and
                # (stream3["interval_low"] < stream5["interval_low"]) and
                # (stream3["low"] < stream5["low"]) #and
                # (stream3["interval_low"] > stream2["interval_low"]) #and
                # (stream2["range"] < stream4["range"])
            ):
                inertia_type='減速下跌'

            else:
                inertia_type='非減速下跌'

        if inertia_type is not None:
            inertia = {
                "start_time": streams[0]["start_time"],
                "end_time": streams[-1]["end_time"],
                "type": inertia_type,
                "power": abs(streams[-1]["close"] - streams[0]["open"]),
                "range": max(d["high"] for d in streams) - min(d["low"] for d in streams),
                "speed": sum(d["speed"] for d in streams),
                "center_high": streams[-2]["high"],
                "center_low": streams[-2]["interval_low"],
                "start_high": streams[0]["high"],
                "start_low": streams[0]["low"],
                "end_high": streams[-1]["end_high"],
                "end_low": streams[-1]["end_low"],
                "inertia_low": min(d["low"] for d in streams)
            }
            self.inertia.append(inertia)
            print(inertia["start_time"],inertia["end_time"], inertia["type"])

        pattern_type=None
        # new_pattern={}
        if len(self.completed_bars) >= 2:
            bar1 = self.completed_bars[-1]
            bar2 = self.completed_bars[-2]

            if len(self.inertia) >= 1:
                inertia1 = self.inertia[-1]
                # pattern1=self.bar_pattern[-1]
                # pattern2=self.bar_pattern[-2]
                if (
                    # self.position==0 and

                    (inertia1["type"] in ['減速下跌'] ) #and
                    # (pattern1["type"] in ['雙跌'] ) and
                    # (inertia1["interval_low"] < pattern1["low"])
                    # (pattern2["type"] == '標準底' ) and
                    # (pattern2["right_power"] >= pattern1["right_power"])

                    ) :
                    # if self.position==0:
                    self.key_entry_price = inertia1["end_high"]
                    self.key_reset_price  = inertia1["center_low"]
                    self.key_exit_price  = inertia1["center_high"]-inertia1["center_low"]+inertia1["end_high"]
                    # action=f'{pattern1["type"]}_設定'
                    action='設定'
                    print(f"{action}_key_entry{self.key_entry_price}_key_reset{self.key_reset_price}__key_exit{self.key_exit_price}")


                    new_key_log = {
                        "start_time": bar["start_time"],
                        'action': action,
                        "enter_price": self.key_entry_price,
                        "reset_price": self.key_reset_price,
                        "exit_price": self.key_exit_price
                    }

                    self.key_price_log.append(new_key_log)

        if  self.key_reset_price is not None:
            if self.position==0:
                if (
                (bar["close"]<self.key_reset_price) or
                (bar["close"] >self.key_entry_price)
                  ):
                  self.key_entry_price = None
                  self.key_reset_price  = None
                  self.key_exit_price  = None
            # elif self.bar_stream!=[]:
            #     if (
            #       self.bar_stream[-1]['interval_low']>self.key_reset_price
            #       ):
            #         self.key_reset_price=stream1['interval_low']
            #         print(f"調高key_reset_price={self.key_reset_price}")
        # Store completed bar
        print(f"pnl{self.max_pnl}")
        self.completed_bars.append(bar.copy())


    # ------------------ 取得K棒時間 ------------------
    def _get_bar_time(self, tick_time):
        interval_min = self.interval.seconds // 60
        if self.label == "right":
            minute_offset = (tick_time.minute // interval_min + 1) * interval_min
            return tick_time.replace(minute=0, second=0, microsecond=0) + timedelta(minutes=minute_offset)
        else:
            return tick_time - timedelta(minutes=tick_time.minute % interval_min,
                                        seconds=tick_time.second,
                                        microseconds=tick_time.microsecond)

    # ------------------ 主運行函數 ------------------
    def run(self, tick_data, is_first_file=False):
        start_time = time.time()

        for _, tick in tick_data.iterrows():
            self.process_tick(tick)

        if self.current_bar:
            self._finalize_bar(self.current_bar)
            self.current_bar = None

        elapsed = time.time() - start_time
        print(f"[完成] 處理 {len(tick_data)} 筆 tick，耗時 {elapsed:.2f}s，生成 {len(self.completed_bars)} 根K線")

        return {
            "kbars": pd.DataFrame(self.completed_bars),
            "tick_logs": pd.DataFrame(self.tick_logs) if self.tick_logs else None,
            "trade_log": pd.DataFrame(self.trade_log) if self.trade_log else None
        }

##無指標+60K

In [ ]:
import pandas as pd
from datetime import timedelta
import math
import time


class KBarSimulator:
    def __init__(self, historical_kbars,historical_kbars_60k, interval_minutes=5, debug=True, label="right",
                 max_position=1, calc_technical_indicators=False):
        """
        初始化 KBarSimulator

        Args:
            historical_kbars: 歷史K線 DataFrame (必須包含 start_time, open, high, low, close, volume)
            interval_minutes: K線週期 (分鐘)
            debug: 是否輸出除錯訊息
            label: "right" (右標籤) 或 "left"
            max_position: 最大持倉數 (預設1)
            calc_technical_indicators: 是否計算技術指標 (MA/EMA)
        """
        self.calc_technical_indicators = calc_technical_indicators
        self.interval = timedelta(minutes=interval_minutes)
        self.interval_60k = timedelta(minutes=60)
        self.debug = debug
        self.label = label

        self.completed_bars = []
        self.completed_bars_60k = []
        self.tick_logs = []
        self.current_bar = None
        self.current_bar_60k = None

        # === 交易控管 ===
        self.position = 0
        self.entry_price = None
        self.exit_price = None
        self.entry_time = None
        self.trade_log = []
        self.max_position = max_position
        self.max_pnl_pct = 0
        self.max_pnl = 0

        # === 自定義結構波與慣性相關變數 ===
        self.bar_stream = []
        self.up_stream = []
        self.down_stream = []
        self.inertia = []

        self.pre_tick_price = None

        self.key_entry_price = None
        self.key_reset_price = None
        self.key_exit_price = None
        self.key_price_log = []

        self.day_highs = []
        self.day_lows = []

        # === 扣抵值追蹤 ===
        self.deduction_values = {
            'ma5': [],
            'ma10': [],
            'ma20': [],
            'ema12': [],
            'ema26': []
        }

        # 初始化歷史完成的K棒（只計算 MA5/10/20 與 EMA12/26）
        self._initialize_completed_bars(historical_kbars)
        self._initialize_completed_bars_60k(historical_kbars_60k)

    def log_trade(
        self,
        tick_time,
        action,
        side=None,
        price=None,
        entry_price=None,
        exit_price=None,
        key_entry_price=None,
        key_reset_price=None,
        key_exit_price=None,
        pnl=0,
        holding_time=None
    ):
        self.trade_log.append({
            "tick_time": tick_time,
            "action": action,
            "side": side,
            "price": price,
            "entry_price": entry_price,
            "exit_price": exit_price,
            "PnL": pnl,
            "holding_time_sec": holding_time,
            "key_entry_price": key_entry_price,
            "key_reset_price": key_reset_price,
            "key_exit_price": key_exit_price
        })


    # ------------------ MA / EMA 計算 ------------------
    def _calculate_ma(self, closes, period):
        if len(closes) < period:
            return sum(closes) / len(closes) if closes else None
        return sum(closes[-period:]) / period

    def _calculate_ema(self, closes, period):
        if len(closes) == 0:
            return None
        alpha = 2.0 / (period + 1)
        ema = closes[0]
        for price in closes[1:]:
            ema = alpha * price + (1 - alpha) * ema
        return ema

    def _get_deduction_value(self, closes, period, indicator_type='ma'):
        """
        計算扣抵值：當新K棒加入時，會被移出計算的舊值

        Args:
            closes: 收盤價列表
            period: 週期
            indicator_type: 'ma' 或 'ema'

        Returns:
            扣抵值（被移出的舊K棒收盤價）
        """
        if len(closes) >= period:
            # 對於MA，扣抵值就是period根K棒之前的收盤價
            deduction_value = closes[-period] if len(closes) > period else closes[0]
            return deduction_value
        return None

    # ------------------ 初始化歷史K棒 ------------------
    def _initialize_completed_bars(self, historical_kbars):
        closes = historical_kbars["close"].tolist()
        for i in range(len(historical_kbars)):
            ma5 = self._calculate_ma(closes[:i + 1], 5)
            ma10 = self._calculate_ma(closes[:i + 1], 10)
            ma20 = self._calculate_ma(closes[:i + 1], 20)
            ema12 = self._calculate_ema(closes[:i + 1], 12)
            ema26 = self._calculate_ema(closes[:i + 1], 26)

            # 計算扣抵值
            deduct_ma5 = self._get_deduction_value(closes[:i + 1], 5, 'ma')
            deduct_ma10 = self._get_deduction_value(closes[:i + 1], 10, 'ma')
            deduct_ma20 = self._get_deduction_value(closes[:i + 1], 20, 'ma')
            deduct_ema12 = self._get_deduction_value(closes[:i + 1], 12, 'ema')
            deduct_ema26 = self._get_deduction_value(closes[:i + 1], 26, 'ema')

            self.completed_bars.append({
                "start_time": historical_kbars.iloc[i]["start_time"],
                "open": historical_kbars.iloc[i]["open"],
                "high": historical_kbars.iloc[i]["high"],
                "low": historical_kbars.iloc[i]["low"],
                "close": historical_kbars.iloc[i]["close"],
                "volume": historical_kbars.iloc[i]["volume"],
                "ma5": ma5,
                "ma10": ma10,
                "ma20": ma20,
                "ema12": ema12,
                "ema26": ema26,
                "deduct_ma5": deduct_ma5,
                "deduct_ma10": deduct_ma10,
                "deduct_ma20": deduct_ma20,
                "deduct_ema12": deduct_ema12,
                "deduct_ema26": deduct_ema26,
            })

    def _initialize_completed_bars_60k(self, historical_kbars_60k):
        """初始化60分鐘K線歷史資料"""
        closes = historical_kbars_60k["close"].tolist()
        for i in range(len(historical_kbars_60k)):
            self.completed_bars_60k.append({
                "start_time": historical_kbars_60k.iloc[i]["start_time"],
                "open": historical_kbars_60k.iloc[i]["open"],
                "high": historical_kbars_60k.iloc[i]["high"],
                "low": historical_kbars_60k.iloc[i]["low"],
                "close": historical_kbars_60k.iloc[i]["close"],
                "volume": historical_kbars_60k.iloc[i]["volume"],
            })

    # ------------------ 計算當前 bar 的 MA/EMA ------------------
    def _calculate_ma_ema(self, current_close):
        if not self.calc_technical_indicators:
            return None, None, None, None, None, None, None, None, None, None

        # 取足夠長度的收盤價來計算 MA20
        closes = [bar["close"] for bar in self.completed_bars[-30:]] + [current_close]

        ma5 = self._calculate_ma(closes, 5)
        ma10 = self._calculate_ma(closes, 10)
        ma20 = self._calculate_ma(closes, 20)
        ema12 = self._calculate_ema(closes, 12)
        ema26 = self._calculate_ema(closes, 26)

        # 計算扣抵值
        deduct_ma5 = self._get_deduction_value(closes, 5, 'ma')
        deduct_ma10 = self._get_deduction_value(closes, 10, 'ma')
        deduct_ma20 = self._get_deduction_value(closes, 20, 'ma')
        deduct_ema12 = self._get_deduction_value(closes, 12, 'ema')
        deduct_ema26 = self._get_deduction_value(closes, 26, 'ema')

        return ma5, ma10, ma20, ema12, ema26, deduct_ma5, deduct_ma10, deduct_ma20, deduct_ema12, deduct_ema26

    # ------------------ 進場訊號 ------------------
    def _entry_signal(self, bar):
        if self.position != 0:
            return 0
        bar_60k = self.completed_bars_60k[-1]
        prev_bar = self.completed_bars[-1]
        prev_bar2 = self.completed_bars[-2]

        def safe_check(name, expr):
            try:
                result = expr()
                if result is None or (isinstance(result, float) and pd.isna(result)):
                    print(f"[NA/NONE] {name} = {result}")
                    return False
                return result
            except Exception as e:
                return False



        # KD + RSI 多單
        if_long = (
            # safe_check("", lambda: prev_bar["low"]>bar["low"]) and
            # safe_check("", lambda: prev_bar["low"]>prev_bar2["low"]) and
            # safe_check("", lambda: prev_bar["high"] < prev_bar2["high"]) and
            # safe_check("", lambda: prev_bar2["high"] < bar["close"]) and
            # safe_check("", lambda: bar_60k["close"] < bar_60k["open"]) and
            # safe_check("", lambda: bar["close"] < self.key_exit_price) and
            safe_check("", lambda: bar["close"] >= self.key_entry_price) and
            safe_check("", lambda: bar["close"] >= bar["ma5"]) #and
            # safe_check("", lambda: bar["ma5"] >= bar["ma10"]) and
            # safe_check("", lambda: bar["ma10"] >= bar["ma20"]) #and
        )

        # 進場條件：價格突破 key_entry_price（由慣性減速下跌形態設定）
        if if_long:
            print(f'符合條件：MA5:{bar["ma5"]} MA10:{ bar["ma10"]} MA20:{bar["ma20"]} PRE_MA20:{prev_bar["ma20"]}')
            # 顯示扣抵值資訊
            if bar.get("deduct_ma5"):
                print(f'扣抵值：MA5:{bar["deduct_ma5"]:.2f} MA10:{bar["deduct_ma10"]:.2f} MA20:{bar["deduct_ma20"]:.2f}')
            return 1  # 多單

        return 0

    # ------------------ 出場訊號 ------------------
    def _exit_signal(self, pnl, bar):
        if self.position == 0 or self.entry_price is None:
            return None

        self.max_pnl = max(self.max_pnl, pnl)
        self.max_pnl_pct = max(self.max_pnl_pct, pnl / self.entry_price * 100)

        # 停損：跌破 reset 價
        if self.position == 1 and self.key_reset_price is not None and ((bar["close"] < self.key_reset_price)):
            return "STOP_LOSS" if pnl < 0 else "TAKE_PROFIT_0"

        # 動態停利
        if self.key_exit_price is not None:

            target = (self.key_exit_price - self.entry_price) * self.position
            if self.max_pnl >= target and bar["close"] < self.key_exit_price and bar["ma5"] <bar["ma10"] :
                return "TAKE_PROFIT_3"

            if self.max_pnl_pct>0.75 and pnl <= self.max_pnl * 0.8:
                return "TAKE_PROFIT_2"
            if self.max_pnl_pct>0.15 and pnl <= self.max_pnl * 0.2:
                return "TAKE_PROFIT_1"

        return None

    # ------------------ 持倉管理 ------------------
    def _manage_position(self, bar):
        tick_time = bar["tick_time"]
        current_time = tick_time.hour * 100 + tick_time.minute
        restricted_periods = [(430, 900), (1300, 1515)]

        in_restricted = any(start <= current_time <= end for start, end in restricted_periods)

        # 強制平倉時段
        if in_restricted and self.position != 0:
            pnl = (bar["close"] - self.entry_price) * self.position
            holding_time = (tick_time - self.entry_time).total_seconds()
            self.log_trade(
                tick_time=tick_time,
                action="FORCE_CLOSE",
                side=self.position,
                entry_price=self.entry_price,
                exit_price=bar["close"],
                pnl=pnl,
                holding_time=holding_time
            )

            if self.debug:
                print(f"[TRADE] {tick_time} 強制平倉 @ {bar['close']} PnL={pnl:.2f}")
            self._reset_position()
            self.bar_stream = []
            self.up_stream = []
            self.down_stream = []
            self.inertia = []
            return

        # 進場
        if not in_restricted and self.position == 0:
            signal = self._entry_signal(bar)
            if signal != 0 and abs(self.position) < self.max_position:
                self.position = signal
                self.entry_price = bar["close"]
                self.entry_time = tick_time
                self.max_pnl = self.max_pnl_pct = 0
                self.log_trade(
                    tick_time=tick_time,
                    action="BUY" if signal == 1 else "SELL",
                    key_entry_price= self.key_entry_price,
                    key_reset_price= self.key_reset_price,
                    key_exit_price= self.key_exit_price,
                    price=bar["close"],
                    pnl=0
                )

                if self.debug:
                    print(f"[TRADE] {tick_time} 進場 {'多單' if signal == 1 else '空單'} @ {bar['close']}")

        # 出場
        elif self.position != 0:
            pnl = (bar["close"] - self.entry_price) * self.position
            action = self._exit_signal(pnl, bar)
            if action:
                holding_time = (tick_time - self.entry_time).total_seconds()
                self.log_trade(
                    tick_time=tick_time,
                    action=action,
                    entry_price=self.entry_price,
                    exit_price=bar["close"],
                    pnl=pnl,
                    holding_time=holding_time
                )

                if self.debug:
                    print(f"[TRADE] {tick_time} 平倉 ({action}) @ {bar['close']} PnL={pnl:.0f}")
                self.position = 0
                if bar["close"] > self.key_exit_price :
                    self._reset_position()
                    print(f'{tick_time}_CLEAR_KEY_PRIVE')

    def _reset_position(self):
        self.position = 0
        self.entry_price = self.exit_price = self.entry_time = None
        self.max_pnl = self.max_pnl_pct = 0
        self.key_entry_price = self.key_reset_price = self.key_exit_price = None
        print('清除關鍵價格')

    # ------------------ Tick 處理 ------------------
    def process_tick(self, tick):
        tick_time = pd.to_datetime(tick["time"])
        price = tick["price"]
        volume = tick["volume"]
        bar_time = self._get_bar_time(tick_time)
        bar_time_60k = self._get_bar_time_60k(tick_time)

        # 新K棒開始
        if self.current_bar is None or bar_time != self.current_bar["start_time"]:
            if self.current_bar is not None:
                self._finalize_bar(self.current_bar)
            self.current_bar = {
                "start_time": bar_time,
                "open": price,
                "high": price,
                "low": price,
                "close": price,
                "volume": volume,
                "ma5": None, "ma10": None, "ma20": None,
                "ema12": None, "ema26": None,
                "deduct_ma5": None, "deduct_ma10": None, "deduct_ma20": None,
                "deduct_ema12": None, "deduct_ema26": None,
            }

        # 更新當前K棒
        self.current_bar["high"] = max(self.current_bar["high"], price)
        self.current_bar["low"] = min(self.current_bar["low"], price)
        self.current_bar["close"] = price
        self.current_bar["volume"] += volume

        # 計算即時指標（包含扣抵值）
        ma5, ma10, ma20, ema12, ema26, deduct_ma5, deduct_ma10, deduct_ma20, deduct_ema12, deduct_ema26 = self._calculate_ma_ema(price)

        # === 處理60K ===
        if self.current_bar_60k is None or bar_time_60k != self.current_bar_60k["start_time"]:
            if self.current_bar_60k is not None:
                self._finalize_bar_60k(self.current_bar_60k)
            self.current_bar_60k = {
                "start_time": bar_time_60k,
                "open": price,
                "high": price,
                "low": price,
                "close": price,
                "volume": volume,
            }

        # 更新當前60K棒
        self.current_bar_60k["high"] = max(self.current_bar_60k["high"], price)
        self.current_bar_60k["low"] = min(self.current_bar_60k["low"], price)
        self.current_bar_60k["close"] = price
        self.current_bar_60k["volume"] += volume

        temp_bar = {
            "tick_time": tick_time,
            "open": self.current_bar["open"],
            "close": price,
            "high": self.current_bar["high"],
            "low": self.current_bar["low"],
            "ma5": ma5, "ma10": ma10, "ma20": ma20,
            "ema12": ema12, "ema26": ema26,
            "deduct_ma5": deduct_ma5, "deduct_ma10": deduct_ma10, "deduct_ma20": deduct_ma20,
            "deduct_ema12": deduct_ema12, "deduct_ema26": deduct_ema26,
            "open_60K": self.current_bar_60k["open"],
            "high_60K": self.current_bar_60k["high"],
            "low_60K": self.current_bar_60k["low"],
        }
        self._manage_position(temp_bar)

        # 記錄 tick log
        self.tick_logs.append({
            "tick_time": tick_time,
            "price": price,
            "volume": volume,
            "bar_time": bar_time,
            "bar_open": self.current_bar["open"],
            "bar_high": self.current_bar["high"],
            "bar_low": self.current_bar["low"],
            "bar_close": price,
            "bar_volume": self.current_bar["volume"],
            "bar_ma5": ma5, "bar_ma10": ma10, "bar_ma20": ma20,
            "bar_ema12": ema12, "bar_ema26": ema26,
            "deduct_ma5": deduct_ma5, "deduct_ma10": deduct_ma10, "deduct_ma20": deduct_ma20,
            "deduct_ema12": deduct_ema12, "deduct_ema26": deduct_ema26,
            "bar_open_60K": self.current_bar_60k["open"],
        })
        self.pre_tick_price = price

    # ------------------ 完成一根K棒 ------------------
    def _finalize_bar_60k(self, bar):
        self.completed_bars_60k.append(bar.copy())

    def _finalize_bar(self, bar):
        last_tick = self.tick_logs[-1] if self.tick_logs else {}
        bar["ma5"] = last_tick.get("bar_ma5")
        bar["ma10"] = last_tick.get("bar_ma10")
        bar["ma20"] = last_tick.get("bar_ma20")
        bar["ema12"] = last_tick.get("bar_ema12")
        bar["ema26"] = last_tick.get("bar_ema26")
        bar["deduct_ma5"] = last_tick.get("deduct_ma5")
        bar["deduct_ma10"] = last_tick.get("deduct_ma10")
        bar["deduct_ma20"] = last_tick.get("deduct_ma20")
        bar["deduct_ema12"] = last_tick.get("deduct_ema12")
        bar["deduct_ema26"] = last_tick.get("deduct_ema26")
        bar["open_60K"] = last_tick.get("bar_open_60K")
        self.completed_bars.append(bar.copy())

        # 記錄扣抵值歷史
        if bar["deduct_ma5"] is not None:
            self.deduction_values['ma5'].append({
                'time': bar["start_time"],
                'value': bar["deduct_ma5"]
            })
        if bar["deduct_ma10"] is not None:
            self.deduction_values['ma10'].append({
                'time': bar["start_time"],
                'value': bar["deduct_ma10"]
            })
        if bar["deduct_ma20"] is not None:
            self.deduction_values['ma20'].append({
                'time': bar["start_time"],
                'value': bar["deduct_ma20"]
            })

        if len(self.completed_bars) >= 2:
            bar = self.completed_bars[-1]
            bar1 = self.completed_bars[-2]

            bar_data = {
                "time": bar["start_time"],
                "open": bar["open"],
                "high": bar["high"],
                "low": bar["low"],
                "close": bar["close"],
                "ma20": bar["ma20"]
            }
            bar1_data = {
                "time": bar1["start_time"],
                "open": bar1["open"],
                "high": bar1["high"],
                "low": bar1["low"],
                "close": bar1["close"],
                "ma20": bar1["ma20"]
            }

            def flush_stream(stream, fail_type):
                if not stream:
                    return

                self.bar_stream.append({
                    "start_time": stream[0]["time"],
                    "end_time": stream[-1]["time"],
                    "FAIL": fail_type,
                    "open": stream[0]["open"],
                    "high": stream[0]["high"],
                    "low": stream[0]["low"],
                    "close": stream[0]["close"],
                    "ma20": stream[0]["ma20"],
                    "interval_high": max(d["high"] for d in stream),
                    "interval_low": min(d["low"] for d in stream),
                    "end_open": stream[-2]["open"],
                    "end_high": stream[-2]["high"],
                    "end_low": stream[-2]["low"],
                    "end_close": stream[-2]["close"],
                    "power": abs(stream[0]["close"] - stream[-2]["open"]) ,
                    "range":max(d["high"] for d in stream)-min(d["low"] for d in stream),
                    'speed':len(stream),
                    "reverse_open": stream[-1]["open"],
                    "reverse_high": stream[-1]["high"],
                    "reverse_low": stream[-1]["low"],
                    "reverse_close": stream[-1]["close"],


                    "left_open": stream[-3]["open"],
                    "left_high": stream[-3]["high"],
                    "left_low": stream[-3]["low"],
                    "left_close": stream[-3]["close"],
                    "center_open": stream[-2]["open"],
                    "center_high": stream[-2]["high"],
                    "center_low": stream[-2]["low"],
                    "center_close": stream[-2]["close"],
                    "right_open": stream[-1]["open"],
                    "right_high": stream[-1]["high"],
                    "right_low": stream[-1]["low"],
                    "right_close": stream[-1]["close"]



                })
                print(f"{fail_type},start_{stream[0]['time']},end_{stream[-1]['time']}_ilow{min(d['low'] for d in stream)}_ihigh{max(d['high'] for d in stream)}")
                print(len(self.bar_stream))
                if len(self.bar_stream)>=3:
                    s3, s2, s1 = self.bar_stream[-3:]
                    time_match = s3['end_time'] >= s1['start_time']

                    if time_match:
                        merged = {
                            "start_time": s3["start_time"],
                            "end_time": s1["end_time"],
                            "FAIL": fail_type,

                            "open": s3["open"],
                            "high": s3["high"],
                            "low": s3["low"],
                            "close": s1["close"],

                            "interval_high": max(
                                s1["interval_high"],
                                s2["interval_high"],
                                s3["interval_high"]
                            ),
                            "interval_low": min(
                                s1["interval_low"],
                                s2["interval_low"],
                                s3["interval_low"]
                            ),

                            "end_open": s3["end_open"],
                            "end_high": s3["end_high"],
                            "end_low": s3["end_low"],
                            "end_close": s3["end_close"],

                            "power": abs(s1["close"] - s3["open"]),
                            "range": (
                                max(s1["interval_high"], s2["interval_high"], s3["interval_high"])
                                - min(s1["interval_low"], s2["interval_low"], s3["interval_low"])
                            ),
                            "speed": s1["speed"] + s2["speed"] + s3["speed"],

                            "reverse_open": s3["reverse_open"],
                            "reverse_high": s3["reverse_high"],
                            "reverse_low": s3["reverse_low"],
                            "reverse_close": s3["reverse_close"],

                            "left_open":s1["left_open"],
                            "left_high":s1["left_high"],
                            "left_low":s1["left_low"],
                            "left_close":s1["left_close"],
                            'center_open':s1['center_open'],
                            'center_high':s1['center_high'],
                            'center_low':s1['center_low'],
                            'center_close':s1['center_close'],
                            'right_open':s1['right_open'],
                            'right_high':s1['right_high'],
                            'right_low':s1['right_low'],
                            'right_close':s1['right_close']


                        }

                        self.bar_stream = self.bar_stream[:-3]
                        self.bar_stream.append(merged)
                        print(f'吞噬合併{fail_type}：start_time_{merged["start_time"]}_end_time{merged["end_time"]}')
                        print(len(self.bar_stream))

            if bar["close"] < bar["open"] and bar["low"] < bar1["low"]:
                if self.up_stream != []:
                    self.up_stream.append(bar_data)
                flush_stream(self.up_stream, "NEW_HIGH")
                self.up_stream.clear()
                if self.down_stream == []:
                    self.down_stream.append(bar1_data)
                self.down_stream.append(bar_data)

            elif bar["close"] > bar["open"] and bar["high"] > bar1["high"]:
                if self.down_stream != []:
                    self.down_stream.append(bar_data)
                flush_stream(self.down_stream, "NEW_LOW")
                self.down_stream.clear()
                if self.up_stream == []:
                    self.up_stream.append(bar1_data)
                self.up_stream.append(bar_data)

            else:
                (self.up_stream if self.up_stream else self.down_stream).append(bar_data)

        inertia_type = None

        if len(self.bar_stream) >= 5:
            streams = self.bar_stream[-5:]
            stream1, stream2, stream3, stream4, stream5 = streams[::-1]

            if (
                (stream1["FAIL"] == 'NEW_LOW') and
                (stream1["interval_low"] < stream3["interval_low"])
            ):
                inertia_type='減速下跌'
            else:
                inertia_type='非減速下跌'

        if inertia_type is not None:
            inertia = {
                "start_time": streams[0]["start_time"],
                "end_time": streams[-1]["end_time"],
                "type": inertia_type,
                "power": abs(streams[-1]["close"] - streams[0]["open"]),
                "range": max(d["high"] for d in streams) - min(d["low"] for d in streams),
                "speed": sum(d["speed"] for d in streams),
                "center_high": streams[-2]["high"],
                "center_low": streams[-2]["interval_low"],
                "start_high": streams[0]["high"],
                "start_low": streams[0]["low"],
                "end_high": streams[-1]["end_high"],
                "end_low": streams[-1]["end_low"],
                "inertia_low": min(d["low"] for d in streams),


                "left_open":streams[-1]["left_open"],
                "left_high":streams[-1]["left_high"],
                "left_low":streams[-1]["left_low"],
                "left_close":streams[-1]["left_close"],
                'center_open':streams[-1]['center_open'],
                'center_high':streams[-1]['center_high'],
                'center_low':streams[-1]['center_low'],
                'center_close':streams[-1]['center_close'],
                'right_open':streams[-1]['right_open'],
                'right_high':streams[-1]['right_high'],
                'right_low':streams[-1]['right_low'],
                'right_close':streams[-1]['right_close']


            }
            self.inertia.append(inertia)
            print(bar["start_time"],inertia["start_time"],inertia["end_time"], inertia["type"])

        if len(self.completed_bars) >= 2:
            if len(self.inertia) >= 1:
                inertia1 = self.inertia[-1]
                if inertia1["type"] in ['減速下跌']:
                    self.key_entry_price = inertia1["left_high"]
                    self.key_reset_price  = inertia1["center_low"]
                    self.key_exit_price  = inertia1["center_high"]-inertia1["center_low"]+inertia1["end_low"]
                    action='設定'
                    print(f"{action}_key_entry{self.key_entry_price}_key_reset{self.key_reset_price}__key_exit{self.key_exit_price}")

                    new_key_log = {
                        "start_time": bar["start_time"],
                        'action': action,
                        "enter_price": self.key_entry_price,
                        "reset_price": self.key_reset_price,
                        "exit_price": self.key_exit_price
                    }
                    self.key_price_log.append(new_key_log)

    # ------------------ 取得K棒時間 ------------------
    def _get_bar_time(self, tick_time):
        interval_min = self.interval.seconds // 60
        if self.label == "right":
            minute_offset = (tick_time.minute // interval_min + 1) * interval_min
            return tick_time.replace(minute=0, second=0, microsecond=0) + timedelta(minutes=minute_offset)
        else:
            return tick_time - timedelta(minutes=tick_time.minute % interval_min,
                                        seconds=tick_time.second,
                                        microseconds=tick_time.microsecond)

    def _get_bar_time_60k(self, tick_time):
        """取得60K棒時間（自動判斷早盤/夜盤）"""
        h = tick_time.hour
        m = tick_time.minute

        is_morning = (
            (h == 8 and m >= 45) or
            (9 <= h <= 12) or
            (h == 13 and m < 45)
        )

        is_night = (h >= 15 or h < 5)

        if is_morning:
            if self.label == "right":
                base = tick_time.replace(minute=45, second=0, microsecond=0)
                if tick_time >= base:
                    base += timedelta(hours=1)
                return base
            else:
                return tick_time.replace(minute=45, second=0, microsecond=0)

        if is_night:
            if self.label == "right":
                base = tick_time.replace(minute=0, second=0, microsecond=0)
                if tick_time >= base:
                    base += timedelta(hours=1)
                return base
            else:
                return tick_time.replace(minute=0, second=0, microsecond=0)

        return tick_time.replace(minute=0, second=0, microsecond=0)

    # ------------------ 主運行函數 ------------------
    def run(self, tick_data, is_first_file=False):
        start_time = time.time()

        for _, tick in tick_data.iterrows():
            self.process_tick(tick)

        if self.current_bar_60k:
            self._finalize_bar_60k(self.current_bar_60k)
            self.current_bar_60k = None
        if self.current_bar:
            self._finalize_bar(self.current_bar)
            self.current_bar = None

        elapsed = time.time() - start_time
        print(f"[完成] 處理 {len(tick_data)} 筆 tick，耗時 {elapsed:.2f}s，生成 {len(self.completed_bars)} 根K線")

        return {
            "kbars": pd.DataFrame(self.completed_bars),
            "tick_logs": pd.DataFrame(self.tick_logs) if self.tick_logs else None,
            "trade_log": pd.DataFrame(self.trade_log) if self.trade_log else None,
            "deduction_values": self.deduction_values
        }

#分析

###2025

In [ ]:
sim=run_simulation("2025-01-02", "2025-12-31")

In [ ]:
# === STOP_LOSS 出現次數最多的前5日 ===
#       date  count  total_pnl  max_pnl  min_pnl
# 2025-04-11      2       -219      -42     -177
# 2025-10-17      5       -145      -10      -55
# 2025-03-03      5       -137      -14      -61
# 2025-11-24      2       -114      -48      -66
# 2025-11-17      3       -106      -19      -55

In [ ]:
import pandas as pd

def pair_trades_from_csv(csv_path: str):
    df = pd.read_csv(csv_path, parse_dates=["tick_time"])
    df = df.sort_values("tick_time")

    paired_trades = []
    current_entry = None

    for _, row in df.iterrows():
        action = row["action"]

        # 進場
        if action in ("BUY", "SELL"):
            current_entry = row

        # 平倉
        elif action in (
            "STOP_LOSS",'FORCE_CLOSE',
            "TAKE_PROFIT_0",
            "TAKE_PROFIT_1",
            "TAKE_PROFIT_2",
        ):
            if current_entry is None:
                continue

            paired_trades.append({
                "entry_time": current_entry["tick_time"],
                "exit_time": row["tick_time"],
                "side": current_entry["action"],
                "entry_price": row["entry_price"],
                "exit_price": row["exit_price"],
                "exit_action": action,
                "pnl": row["PnL"],
                "holding_time_sec": row["holding_time_sec"],
                "key_entry_price": current_entry.get("key_entry_price"),
                "key_reset_price": current_entry.get("key_reset_price"),
                "key_exit_price": current_entry.get("key_exit_price"),
            })

            current_entry = None

    return pd.DataFrame(paired_trades)

df = pair_trades_from_csv("trade_log.csv")
df["key_diff"] = (df["key_entry_price"]- df["key_reset_price"])
df["key_diff_bin"] = pd.qcut(df["key_diff"], 10)
df.groupby("key_diff_bin", observed=True)["pnl"].agg(
    trades="count",
    win_rate=lambda x: (x > 0).mean(),
    avg_pnl="mean",
    total_pnl="sum"
)
# import matplotlib.pyplot as plt
# import seaborn as sns


# plt.figure(figsize=(10,6))

# # 直接用 signed_key_diff 對應 pnl
# sns.scatterplot(x="key_diff", y="pnl", data=df, alpha=0.6)

# plt.xlabel("Key Diff (key_entry - key_reset)")
# plt.ylabel("PnL")
# plt.title("PnL vs Key Diff (All Trades)")
# plt.grid(True)
# plt.show()

###2024

In [ ]:
sim=run_simulation("2024-01-02", "2024-12-31")

In [ ]:
# === STOP_LOSS 出現次數最多的前5日 ===
#       date  count  total_pnl  max_pnl  min_pnl
# 2024-08-06      2       -139      -40      -99
# 2024-07-22      3        -95      -21      -50
# 2024-09-04      2        -91      -21      -70
# 2024-11-18      2        -89      -24      -65
# 2024-10-29      3        -81      -21      -31

##2023

In [ ]:
sim=run_simulation("2023-01-02", "2023-12-31")

In [ ]:
# === STOP_LOSS 出現次數最多的前5日 ===
#       date  count  total_pnl  max_pnl  min_pnl
# 2023-10-24      2        -66       -7      -59
# 2023-07-10      3        -65      -11      -32
# 2023-03-14      4        -53       -4      -30
# 2023-07-20      3        -49      -10      -21
# 2023-11-01      2        -45      -11      -34

##2022

In [ ]:
sim=run_simulation("2022-01-02", "2022-12-31")

In [ ]:
# === STOP_LOSS 出現次數最多的前5日 ===
#       date  count  total_pnl  max_pnl  min_pnl
# 2022-06-28      5       -107      -13      -34
# 2022-03-14      3        -70      -19      -28
# 2022-07-11      3        -59      -13      -27
# 2022-03-25      4        -58       -6      -19
# 2022-11-16      1        -58      -58      -58

##2021

In [ ]:
sim=run_simulation("2021-01-02", "2021-12-31")

In [ ]:
# === STOP_LOSS 出現次數最多的前5日 ===
#       date  count  total_pnl  max_pnl  min_pnl
# 2021-05-14      3       -114       -5      -71
# 2021-05-20      2        -91      -30      -61
# 2021-05-18      2        -70      -29      -41
# 2021-05-25      2        -70      -12      -58
# 2021-10-05      5        -64       -9      -20

##2020

In [ ]:
sim=run_simulation("2020-01-02", "2020-12-31")

In [ ]:
# === STOP_LOSS 出現次數最多的前5日 ===
#       date  count  total_pnl  max_pnl  min_pnl
# 2020-03-13      1       -151     -151     -151
# 2020-03-20      2       -109      -22      -87
# 2020-03-10      2        -68      -21      -47
# 2020-03-18      2        -63      -30      -33
# 2020-03-27      2        -56       -7      -49

#特記案例

##未分類

###20250108

In [ ]:
run_simulation("2025-01-06", "2025-01-08")
fig = plot_kbar_analysis(start_date="2025-01-08", end_date=None)

找到 3 個檔案

從第 2 檔繼續（已有 228 根K線）

[2/3] CLEAN_TICK_Daily_2025_01_07.parquet
  載入: 83105 筆ticks, 228 根K線
  ❌ 錯誤: list index out of range

[3/3] CLEAN_TICK_Daily_2025_01_08.parquet


Traceback (most recent call last):
  File "/tmp/ipython-input-4094339611.py", line 119, in run_simulation
    sim.run(ticks, is_first_file=False)
  File "/tmp/ipython-input-1496396680.py", line 750, in run
    self.process_tick(tick)
  File "/tmp/ipython-input-1496396680.py", line 368, in process_tick
    self._finalize_bar(self.current_bar)
  File "/tmp/ipython-input-1496396680.py", line 624, in _finalize_bar
    flush_stream(self.down_stream, "NEW_LOW")
  File "/tmp/ipython-input-1496396680.py", line 530, in flush_stream
    "left_open": stream[-3]["open"],
                 ~~~~~~^^^^
IndexError: list index out of range


  載入: 85048 筆ticks, 228 根K線
NEW_LOW,start_2025-01-06 15:05:00,end_2025-01-07 15:05:00_ilow23618_ihigh23787
1
NEW_HIGH,start_2025-01-06 15:10:00,end_2025-01-07 15:30:00_ilow23629_ihigh23828
2
NEW_LOW,start_2025-01-07 15:25:00,end_2025-01-07 15:45:00_ilow23794_ihigh23828
3
NEW_HIGH,start_2025-01-07 15:40:00,end_2025-01-07 15:55:00_ilow23799_ihigh23823
4
NEW_LOW,start_2025-01-07 15:50:00,end_2025-01-07 16:05:00_ilow23808_ihigh23825
5
2025-01-07 16:05:00 2025-01-06 15:05:00 2025-01-07 16:05:00 非減速下跌
2025-01-07 16:10:00 2025-01-06 15:05:00 2025-01-07 16:05:00 非減速下跌
NEW_HIGH,start_2025-01-07 16:00:00,end_2025-01-07 16:15:00_ilow23806_ihigh23825
6
2025-01-07 16:15:00 2025-01-06 15:10:00 2025-01-07 16:15:00 非減速下跌
2025-01-07 16:20:00 2025-01-06 15:10:00 2025-01-07 16:15:00 非減速下跌
NEW_LOW,start_2025-01-07 16:10:00,end_2025-01-07 16:25:00_ilow23797_ihigh23824
7
2025-01-07 16:25:00 2025-01-07 15:25:00 2025-01-07 16:25:00 減速下跌
設定_key_entry23821_key_reset23797__key_exit23809
2025-01-07 16:30:00 2025-

###20251017

In [ ]:
run_simulation("2025-10-16", "2025-10-17")
fig = plot_kbar_analysis(start_date="2025-10-17", end_date=None)

找到 2 個檔案

未找到狀態檔，從頭開始

[1/2] CLEAN_TICK_Daily_2025_10_16.parquet
  載入: 118864 筆ticks, 228 根K線
  初始化模擬器...
  ✓ 進度已保存

[2/2] CLEAN_TICK_Daily_2025_10_17.parquet
  載入: 116280 筆ticks, 228 根K線
NEW_LOW,start_2025-10-16 13:45:00,end_2025-10-16 15:35:00_ilow27612_ihigh27834.0
1
NEW_HIGH,start_2025-10-16 15:30:00,end_2025-10-16 16:15:00_ilow27645_ihigh27805
2
NEW_LOW,start_2025-10-16 16:10:00,end_2025-10-16 16:45:00_ilow27706_ihigh27805
3
NEW_HIGH,start_2025-10-16 16:40:00,end_2025-10-16 17:05:00_ilow27728_ihigh27786
4
NEW_LOW,start_2025-10-16 17:00:00,end_2025-10-16 17:25:00_ilow27739_ihigh27779
5
2025-10-16 17:25:00 2025-10-16 13:45:00 2025-10-16 17:25:00 非減速下跌
NEW_HIGH,start_2025-10-16 17:20:00,end_2025-10-16 17:30:00_ilow27737_ihigh27778
6
2025-10-16 17:30:00 2025-10-16 15:30:00 2025-10-16 17:30:00 非減速下跌
2025-10-16 17:35:00 2025-10-16 15:30:00 2025-10-16 17:30:00 非減速下跌
2025-10-16 17:40:00 2025-10-16 15:30:00 2025-10-16 17:30:00 非減速下跌
2025-10-16 17:45:00 2025-10-16 15:30:00 2025-10-16 17:30:

###20251023

In [ ]:
run_simulation("2025-10-22", "2025-10-23")
fig = plot_kbar_analysis(start_date="2025-10-23", end_date=None)

###20251105

In [ ]:
run_simulation("2025-11-04", "2025-11-05")
fig = plot_kbar_analysis(start_date="2025-11-05", end_date=None)

找到 2 個檔案

未找到狀態檔，從頭開始

[1/2] CLEAN_TICK_Daily_2025_11_04.parquet
  載入: 90829 筆ticks, 228 根K線
  初始化模擬器...
  ✓ 進度已保存

[2/2] CLEAN_TICK_Daily_2025_11_05.parquet
  載入: 123921 筆ticks, 229 根K線
NEW_LOW,start_2025-11-04 13:45:00,end_2025-11-04 15:20:00_ilow28060_ihigh28144.0
1
NEW_HIGH,start_2025-11-04 15:15:00,end_2025-11-04 15:35:00_ilow28076_ihigh28149
2
NEW_LOW,start_2025-11-04 15:30:00,end_2025-11-04 15:40:00_ilow28127_ihigh28163
3
NEW_HIGH,start_2025-11-04 15:35:00,end_2025-11-04 16:00:00_ilow28127_ihigh28163
4
吞噬合併NEW_HIGH：start_time_2025-11-04 15:15:00_end_time2025-11-04 16:00:00
2
NEW_LOW,start_2025-11-04 15:55:00,end_2025-11-04 16:25:00_ilow28052_ihigh28156
3
NEW_HIGH,start_2025-11-04 16:20:00,end_2025-11-04 16:50:00_ilow28052_ihigh28139
4
NEW_LOW,start_2025-11-04 16:45:00,end_2025-11-04 17:05:00_ilow28096_ihigh28142
5
2025-11-04 17:05:00 2025-11-04 13:45:00 2025-11-04 17:05:00 非減速下跌
NEW_HIGH,start_2025-11-04 17:00:00,end_2025-11-04 17:10:00_ilow28089_ihigh28142
6
2025-11-04 17:10:00

##大漲

###20240807

漲多跌少+底分型>>BUY、跌多漲少+打臉頂分型>>BUY，漲多跌少+打臉底分型>>SELL、跌多漲少+頂分型>>SELL

In [ ]:
run_simulation("2024-08-06", "2024-08-07")
fig = plot_kbar_analysis(start_date="2024-08-07", end_date=None)

找到 2 個檔案

未找到狀態檔，從頭開始

[1/2] CLEAN_TICK_Daily_2024_08_06.parquet
  載入: 267138 筆ticks, 228 根K線
  初始化模擬器...
  ✓ 進度已保存

[2/2] CLEAN_TICK_Daily_2024_08_07.parquet
  載入: 210095 筆ticks, 228 根K線
NEW_LOW,start_2024-08-06 13:45:00,end_2024-08-06 15:10:00_ilow20222_ihigh20540.0
1
NEW_HIGH,start_2024-08-06 15:05:00,end_2024-08-06 15:40:00_ilow20222_ihigh20426
2
NEW_LOW,start_2024-08-06 15:35:00,end_2024-08-06 16:25:00_ilow20027_ihigh20424
3
NEW_HIGH,start_2024-08-06 16:20:00,end_2024-08-06 16:40:00_ilow20048_ihigh20190
4
NEW_LOW,start_2024-08-06 16:35:00,end_2024-08-06 16:45:00_ilow20085_ihigh20185
5
2024-08-06 16:45:00 2024-08-06 13:45:00 2024-08-06 16:45:00 非減速下跌
2024-08-06 16:50:00 2024-08-06 13:45:00 2024-08-06 16:45:00 非減速下跌
2024-08-06 16:55:00 2024-08-06 13:45:00 2024-08-06 16:45:00 非減速下跌
NEW_HIGH,start_2024-08-06 16:40:00,end_2024-08-06 17:00:00_ilow20085_ihigh20247
6
吞噬合併NEW_HIGH：start_time_2024-08-06 16:20:00_end_time2024-08-06 17:00:00
4
NEW_LOW,start_2024-08-06 16:55:00,end_2024-08-06 

###20240814

In [ ]:
run_simulation("2024-08-13", "2024-08-14")
fig = plot_kbar_analysis(start_date="2024-08-14", end_date=None)

###20241115(CHECK)

In [ ]:
run_simulation("2024-11-14", "2024-11-15")
fig = plot_kbar_analysis(start_date="2024-11-15", end_date=None)

找到 2 個檔案

未找到狀態檔，從頭開始

[1/2] CLEAN_TICK_Daily_2024_11_14.parquet
  載入: 118492 筆ticks, 228 根K線
  初始化模擬器...
  ✓ 進度已保存

[2/2] CLEAN_TICK_Daily_2024_11_15.parquet
  載入: 89663 筆ticks, 228 根K線
NEW_LOW,start_2024-11-14 13:45:00,end_2024-11-14 15:20:00_ilow22592_ihigh22727.0
1
NEW_HIGH,start_2024-11-14 15:15:00,end_2024-11-14 15:55:00_ilow22610_ihigh22708
2
NEW_LOW,start_2024-11-14 15:50:00,end_2024-11-14 16:10:00_ilow22643_ihigh22708
3
NEW_HIGH,start_2024-11-14 16:05:00,end_2024-11-14 16:25:00_ilow22643_ihigh22727
4
NEW_LOW,start_2024-11-14 16:20:00,end_2024-11-14 16:30:00_ilow22697_ihigh22733
5
2024-11-14 16:30:00 2024-11-14 13:45:00 2024-11-14 16:30:00 非減速下跌
2024-11-14 16:35:00 2024-11-14 13:45:00 2024-11-14 16:30:00 非減速下跌
NEW_HIGH,start_2024-11-14 16:25:00,end_2024-11-14 16:40:00_ilow22697_ihigh22750
6
吞噬合併NEW_HIGH：start_time_2024-11-14 16:05:00_end_time2024-11-14 16:40:00
4
NEW_LOW,start_2024-11-14 16:35:00,end_2024-11-14 17:00:00_ilow22677_ihigh22750
5
2024-11-14 17:00:00 2024-11-14 13:4

###20241122

In [ ]:
run_simulation("2024-11-21", "2024-11-22")
fig = plot_kbar_analysis(start_date="2024-11-22", end_date=None)

























































































































###20250410

In [ ]:
run_simulation("2025-04-09", "2025-04-10")
fig = plot_kbar_analysis(start_date="2025-04-10", end_date=None)

###20250409

In [ ]:
run_simulation("2025-04-08", "2025-04-09")
fig = plot_kbar_analysis( start_date="2025-04-09",  end_date=None  )

###20250408

In [ ]:
run_simulation("2025-04-07", "2025-04-08")
fig = plot_kbar_analysis(   start_date="2025-04-08",   end_date=None   )

找到 2 個檔案

未找到狀態檔，從頭開始

[1/2] CLEAN_TICK_Daily_2025_04_07.parquet
  載入: 64345 筆ticks, 224 根K線
  初始化模擬器...
  ✓ 進度已保存

[2/2] CLEAN_TICK_Daily_2025_04_08.parquet
  載入: 266328 筆ticks, 229 根K線
NEW_LOW,start_2025-04-07 13:45:00,end_2025-04-07 15:15:00_ilow17566_ihigh19204.0
1
NEW_HIGH,start_2025-04-07 15:10:00,end_2025-04-07 15:55:00_ilow17566_ihigh17922
2
NEW_LOW,start_2025-04-07 15:50:00,end_2025-04-07 16:05:00_ilow17828_ihigh17997
3
NEW_HIGH,start_2025-04-07 16:00:00,end_2025-04-07 16:15:00_ilow17828_ihigh18017
4
NEW_LOW,start_2025-04-07 16:10:00,end_2025-04-07 17:30:00_ilow17656_ihigh18017
5
2025-04-07 17:30:00 2025-04-07 13:45:00 2025-04-07 17:30:00 減速下跌
設定_key_entry17772_key_reset17662__key_exit17718
2025-04-07 17:35:00 2025-04-07 13:45:00 2025-04-07 17:30:00 減速下跌
設定_key_entry17772_key_reset17662__key_exit17718
NEW_HIGH,start_2025-04-07 17:25:00,end_2025-04-07 17:40:00_ilow17606_ihigh17735
6
2025-04-07 17:40:00 2025-04-07 15:10:00 2025-04-07 17:40:00 非減速下跌
2025-04-07 17:45:00 2025-04-07

###20250407

In [ ]:
run_simulation("2025-04-02", "2025-04-07")
fig = plot_kbar_analysis(    start_date="2025-04-07",   end_date=None   )

標準頂分型向下翻亞當失敗(成功前出現標準頂分型)不取消入場點設定，若翻亞當成功則取消

###20250401

In [ ]:
run_simulation("2025-03-30", "2025-04-01")
fig = plot_kbar_analysis(  start_date="2025-04-01",   end_date=None    )

###20250402

In [ ]:
run_simulation("2025-04-01", "2025-04-02")
fig = plot_kbar_analysis(    start_date="2025-04-02",   end_date=None   )

##大跌

###20240712

In [ ]:
run_simulation("2024-07-11", "2024-07-12")
fig = plot_kbar_analysis(start_date="2024-07-12", end_date=None)

###20240719

In [ ]:
run_simulation("2024-07-18", "2024-07-19")
fig = plot_kbar_analysis(start_date="2024-07-19", end_date=None)

找到 2 個檔案

未找到狀態檔，從頭開始

[1/2] CLEAN_TICK_Daily_2024_07_18.parquet
  載入: 154569 筆ticks, 228 根K線
  初始化模擬器...
  ✓ 進度已保存

[2/2] CLEAN_TICK_Daily_2024_07_19.parquet
  載入: 205250 筆ticks, 228 根K線
NEW_HIGH,start_2024-07-18 13:45:00,end_2024-07-18 15:15:00_ilow23380.0_ihigh23592
1
NEW_LOW,start_2024-07-18 15:10:00,end_2024-07-18 15:45:00_ilow23398_ihigh23578
2
NEW_HIGH,start_2024-07-18 15:40:00,end_2024-07-18 16:05:00_ilow23400_ihigh23472
3
NEW_LOW,start_2024-07-18 16:00:00,end_2024-07-18 16:20:00_ilow23417_ihigh23468
4
NEW_HIGH,start_2024-07-18 16:15:00,end_2024-07-18 16:55:00_ilow23422_ihigh23529
5
2024-07-18 16:55:00 2024-07-18 13:45:00 2024-07-18 16:55:00 非減速下跌
2024-07-18 17:00:00 2024-07-18 13:45:00 2024-07-18 16:55:00 非減速下跌
2024-07-18 17:05:00 2024-07-18 13:45:00 2024-07-18 16:55:00 非減速下跌
2024-07-18 17:10:00 2024-07-18 13:45:00 2024-07-18 16:55:00 非減速下跌
NEW_LOW,start_2024-07-18 16:50:00,end_2024-07-18 17:15:00_ilow23487_ihigh23525
6
2024-07-18 17:15:00 2024-07-18 15:10:00 2024-07-18 17:15:

###20240802

In [ ]:
run_simulation("2024-08-01", "2024-08-02")
fig = plot_kbar_analysis(start_date="2024-08-02", end_date=None)

###20240805

In [ ]:
run_simulation("2024-08-02", "2024-08-05")
fig = plot_kbar_analysis(start_date="2024-08-05",end_date=None)

###20240808

In [ ]:
run_simulation("2024-08-07", "2024-08-08")
fig = plot_kbar_analysis(start_date="2024-08-08", end_date=None)

###20240904

In [ ]:
run_simulation("2024-09-03", "2024-09-04")
fig = plot_kbar_analysis(start_date="2024-09-04", end_date=None)


###20240909

In [ ]:
run_simulation("2024-09-06", "2024-09-09")
fig = plot_kbar_analysis(start_date="2024-09-09", end_date=None)

###20241001

In [ ]:
run_simulation("2024-09-30", "2024-10-01")
fig = plot_kbar_analysis(start_date="2024-10-01", end_date=None)

找到 2 個檔案

未找到狀態檔，從頭開始

[1/2] CLEAN_TICK_Daily_2024_09_30.parquet
  載入: 121866 筆ticks, 228 根K線
  初始化模擬器...
  ✓ 進度已保存

[2/2] CLEAN_TICK_Daily_2024_10_01.parquet
  載入: 95272 筆ticks, 228 根K線
NEW_HIGH,start_2024-09-30 13:45:00,end_2024-09-30 15:20:00_ilow22321.0_ihigh22435
1
NEW_LOW,start_2024-09-30 15:15:00,end_2024-09-30 15:35:00_ilow22355_ihigh22435
2
NEW_HIGH,start_2024-09-30 15:30:00,end_2024-09-30 15:40:00_ilow22348_ihigh22398
3
NEW_LOW,start_2024-09-30 15:35:00,end_2024-09-30 15:45:00_ilow22348_ihigh22406
4
吞噬合併NEW_LOW：start_time_2024-09-30 15:15:00_end_time2024-09-30 15:45:00
2
NEW_HIGH,start_2024-09-30 15:40:00,end_2024-09-30 16:00:00_ilow22348_ihigh22417
3
NEW_LOW,start_2024-09-30 15:55:00,end_2024-09-30 16:30:00_ilow22278_ihigh22417
4
NEW_HIGH,start_2024-09-30 16:25:00,end_2024-09-30 16:50:00_ilow22282_ihigh22363
5
2024-09-30 16:50:00 2024-09-30 13:45:00 2024-09-30 16:50:00 非減速下跌
2024-09-30 16:55:00 2024-09-30 13:45:00 2024-09-30 16:50:00 非減速下跌
NEW_LOW,start_2024-09-30 16:45:00,e

In [ ]:
run_simulation("2024-09-30", "2024-10-01")
fig = plot_kbar_analysis(start_date="2024-10-01", end_date=None)

###20241002

In [ ]:
run_simulation("2024-10-01", "2024-10-02")
fig = plot_kbar_analysis(start_date="2024-10-02", end_date=None)

###20241003

In [ ]:
run_simulation("2024-10-02", "2024-10-03")
fig = plot_kbar_analysis(start_date="2024-10-03", end_date=None)

###20241004

In [ ]:
run_simulation("2024-10-03", "2024-10-04")
fig = plot_kbar_analysis(start_date="2024-10-04", end_date=None)

###20241105(CHECK)

In [ ]:
run_simulation("2024-11-04", "2024-11-05")
fig = plot_kbar_analysis(start_date="2024-11-05", end_date=None)

In [ ]:
run_simulation("2024-11-04", "2024-11-05")
fig = plot_kbar_analysis(start_date="2024-11-05", end_date=None)

找到 2 個檔案

未找到狀態檔，從頭開始

[1/2] CLEAN_TICK_Daily_2024_11_04.parquet
  載入: 85901 筆ticks, 228 根K線
  初始化模擬器...
  ✓ 進度已保存

[2/2] CLEAN_TICK_Daily_2024_11_05.parquet
  載入: 88970 筆ticks, 228 根K線
NEW_HIGH,start_2024-11-04 13:45:00,end_2024-11-04 15:25:00_ilow23007.0_ihigh23088
1
NEW_LOW,start_2024-11-04 15:20:00,end_2024-11-04 15:40:00_ilow23014_ihigh23078
2
NEW_HIGH,start_2024-11-04 15:35:00,end_2024-11-04 15:50:00_ilow23002_ihigh23043
3
NEW_LOW,start_2024-11-04 15:45:00,end_2024-11-04 16:15:00_ilow22982_ihigh23041
4
NEW_HIGH,start_2024-11-04 16:10:00,end_2024-11-04 16:30:00_ilow22985_ihigh23039
5
2024-11-04 16:30:00 2024-11-04 13:45:00 2024-11-04 16:30:00 非減速下跌
2024-11-04 16:35:00 2024-11-04 13:45:00 2024-11-04 16:30:00 非減速下跌
2024-11-04 16:40:00 2024-11-04 13:45:00 2024-11-04 16:30:00 非減速下跌
2024-11-04 16:45:00 2024-11-04 13:45:00 2024-11-04 16:30:00 非減速下跌
NEW_LOW,start_2024-11-04 16:25:00,end_2024-11-04 16:50:00_ilow22962_ihigh23034
6
2024-11-04 16:50:00 2024-11-04 15:20:00 2024-11-04 16:50:00

###20241107

In [ ]:
run_simulation("2024-11-06", "2024-11-07")
fig = plot_kbar_analysis(start_date="2024-11-07", end_date=None)

找到 2 個檔案

未找到狀態檔，從頭開始

[1/2] CLEAN_TICK_Daily_2024_11_06.parquet
  載入: 108617 筆ticks, 228 根K線
  初始化模擬器...
  ✓ 進度已保存

[2/2] CLEAN_TICK_Daily_2024_11_07.parquet
  載入: 115722 筆ticks, 228 根K線
NEW_HIGH,start_2024-11-06 13:45:00,end_2024-11-06 15:20:00_ilow23271.0_ihigh23475
1
NEW_LOW,start_2024-11-06 15:15:00,end_2024-11-06 16:00:00_ilow23351_ihigh23465
2
NEW_HIGH,start_2024-11-06 15:55:00,end_2024-11-06 16:35:00_ilow23351_ihigh23433
3
NEW_LOW,start_2024-11-06 16:30:00,end_2024-11-06 16:50:00_ilow23374_ihigh23417
4
NEW_HIGH,start_2024-11-06 16:45:00,end_2024-11-06 17:10:00_ilow23388_ihigh23449
5
2024-11-06 17:10:00 2024-11-06 13:45:00 2024-11-06 17:10:00 非減速下跌
2024-11-06 17:15:00 2024-11-06 13:45:00 2024-11-06 17:10:00 非減速下跌
2024-11-06 17:20:00 2024-11-06 13:45:00 2024-11-06 17:10:00 非減速下跌
2024-11-06 17:25:00 2024-11-06 13:45:00 2024-11-06 17:10:00 非減速下跌
2024-11-06 17:30:00 2024-11-06 13:45:00 2024-11-06 17:10:00 非減速下跌
NEW_LOW,start_2024-11-06 17:05:00,end_2024-11-06 17:35:00_ilow23318_ihig

###20250115(CHECK)

In [ ]:
run_simulation("2025-01-14", "2025-01-15")
fig = plot_kbar_analysis(start_date="2025-01-15", end_date=None)

找到 2 個檔案

未找到狀態檔，從頭開始

[1/2] CLEAN_TICK_Daily_2025_01_14.parquet
  載入: 84842 筆ticks, 228 根K線
  初始化模擬器...
  ✓ 進度已保存

[2/2] CLEAN_TICK_Daily_2025_01_15.parquet
  載入: 60439 筆ticks, 225 根K線
  ❌ 錯誤: list index out of range


所有檔案處理完成

✓ 總共 230 根K線

✓ 狀態檔已清理
📊 讀取資料中...
⚠️ 無法讀取 kbar_csv [Errno 2] No such file or directory: 'kbars.csv'
None
⚠️ 無法讀取 trade_csv: [Errno 2] No such file or directory: 'trade_log.csv'
⚠️ 無法讀取 ss_log: [Errno 2] No such file or directory: 'ss_log.csv'
⚠️ 無法讀取 key_price_log: [Errno 2] No such file or directory: 'key_price_log.csv'


Traceback (most recent call last):
  File "/tmp/ipython-input-4094339611.py", line 119, in run_simulation
    sim.run(ticks, is_first_file=False)
  File "/tmp/ipython-input-1496396680.py", line 750, in run
    self.process_tick(tick)
  File "/tmp/ipython-input-1496396680.py", line 368, in process_tick
    self._finalize_bar(self.current_bar)
  File "/tmp/ipython-input-1496396680.py", line 624, in _finalize_bar
    flush_stream(self.down_stream, "NEW_LOW")
  File "/tmp/ipython-input-1496396680.py", line 530, in flush_stream
    "left_open": stream[-3]["open"],
                 ~~~~~~^^^^
IndexError: list index out of range


AttributeError: 'NoneType' object has no attribute 'columns'

###20250224

In [ ]:
run_simulation("2025-02-21", "2025-02-24")
fig = plot_kbar_analysis(start_date="2025-02-24",  end_date=None)

###20250311

In [ ]:
run_simulation("2025-03-09", "2025-03-11")
fig = plot_kbar_analysis(start_date="2025-03-11",end_date=None )

###20250506

In [ ]:
run_simulation("2025-05-03", "2025-05-06")
fig = plot_kbar_analysis(start_date="2025-05-06", end_date=None)

###20250526

In [ ]:
run_simulation("2025-05-23", "2025-05-26")
fig = plot_kbar_analysis(start_date="2025-05-26", end_date=None)

##震盪

###20240422

In [ ]:
run_simulation("2024-04-19", "2024-04-22")
fig = plot_kbar_analysis(start_date="2024-04-22", end_date=None)

###20250616

In [ ]:
run_simulation("2025-06-13", "2025-06-16")
fig = plot_kbar_analysis(start_date="2025-06-16", end_date=None)

###20240624

In [ ]:
run_simulation("2024-06-21", "2024-06-24")
fig = plot_kbar_analysis(start_date="2024-06-24", end_date=None)

###20250714

In [ ]:
run_simulation("2025-07-11", "2025-07-14")
fig = plot_kbar_analysis(start_date="2025-07-14", end_date=None)

###20240722

In [ ]:
run_simulation("2024-07-19", "2024-07-22")
fig = plot_kbar_analysis(start_date="2024-07-22",end_date=None)

###20240726

In [ ]:
run_simulation("2024-07-25", "2024-07-26")
fig = plot_kbar_analysis(start_date="2024-07-26", end_date=None)

###20240803

In [ ]:
run_simulation("2024-08-01", "2024-08-03")
fig = plot_kbar_analysis(start_date="2024-08-03", end_date=None)

###20240805

In [ ]:
run_simulation("2024-08-02", "2024-08-05")
fig = plot_kbar_analysis(start_date="2024-08-05", end_date=None)

###20240812

In [ ]:
run_simulation("2024-08-09", "2024-08-12")
fig = plot_kbar_analysis(
    kbar_csv_path="kbars.csv",
    trade_csv_path="trade_log.csv",
    key_price_log_path="key_price_log.csv",
    start_date="2024-08-12",
    end_date=None
    )

###20240813

In [ ]:
run_simulation("2024-08-12", "2024-08-13")
fig = plot_kbar_analysis(start_date="2024-08-13", end_date=None)

###20240919

In [ ]:
run_simulation("2024-09-18", "2024-09-19")
fig = plot_kbar_analysis(start_date="2024-09-19", end_date=None)

###20240930

In [ ]:
run_simulation("2024-09-27", "2024-09-30")
fig = plot_kbar_analysis(start_date="2024-09-30", end_date=None)

###20241210

In [ ]:
run_simulation("2024-12-09", "2024-12-10")
fig = plot_kbar_analysis(start_date="2024-12-10", end_date=None)

###20250113

In [ ]:
run_simulation("2025-01-10", "2025-01-13")
fig = plot_kbar_analysis(start_date="2025-01-13", end_date=None)

###20250103

In [ ]:
run_simulation("2025-01-02", "2025-01-03")
fig = plot_kbar_analysis(
    kbar_csv_path="kbars.csv",
    trade_csv_path="trade_log.csv",
    key_price_log_path="key_price_log.csv",
    start_date="2025-01-03",
    end_date=None
    )

###20250117

In [ ]:
run_simulation("2025-01-15", "2025-01-17")
fig = plot_kbar_analysis(
    kbar_csv_path="kbars.csv",
    trade_csv_path="trade_log.csv",
    key_price_log_path="key_price_log.csv",
    start_date="2025-01-17",
    end_date=None
    )#高點低於RESET時要清除KEY設定

###20250121

In [ ]:
run_simulation("2025-01-19", "2025-01-21")
fig = plot_kbar_analysis(
    kbar_csv_path="kbars.csv",
    trade_csv_path="trade_log.csv",
    key_price_log_path="key_price_log.csv",
    start_date="2025-01-21",
    end_date=None
    )

###20250210

In [ ]:
run_simulation("2025-02-07", "2025-02-10")
fig = plot_kbar_analysis(
    kbar_csv_path="kbars.csv",
    trade_csv_path="trade_log.csv",
    key_price_log_path="key_price_log.csv",
    start_date="2025-02-10",
    end_date=None
    )

###20250307

In [ ]:
run_simulation("2025-03-06", "2025-03-07")
fig = plot_kbar_analysis(start_date="2025-03-07", end_date=None)

###20250312

In [ ]:
run_simulation("2025-03-11", "2025-03-12")
fig = plot_kbar_analysis(start_date="2025-03-12", end_date=None)

###20250402

In [ ]:
run_simulation("2025-04-01", "2025-04-02")
fig = plot_kbar_analysis(
    kbar_csv_path="kbars.csv",
    trade_csv_path="trade_log.csv",
    key_price_log_path="key_price_log.csv",
    start_date="2025-04-02",
    end_date=None
    )

###20250411

In [ ]:
run_simulation("2025-04-10", "2025-04-11")
fig = plot_kbar_analysis(start_date="2025-04-11",end_date=None)

###20250414


In [ ]:
run_simulation("2025-04-11", "2025-04-14")
fig = plot_kbar_analysis(
    kbar_csv_path="kbars.csv",
    trade_csv_path="trade_log.csv",
    key_price_log_path="key_price_log.csv",
    start_date="2025-04-14",
    end_date=None
    )

###20250415

In [ ]:
run_simulation("2025-04-14", "2025-04-15")
fig = plot_kbar_analysis(start_date="2025-04-15", end_date=None)

###20250416

In [ ]:
run_simulation("2025-04-15", "2025-04-16")
fig = plot_kbar_analysis(start_date="2025-04-16", end_date=None)

###20250418

In [ ]:
run_simulation("2025-04-17", "2025-04-18")
fig = plot_kbar_analysis(start_date="2025-04-18", end_date=None)

###20250506

In [ ]:
run_simulation("2025-05-05", "2025-05-06")
fig = plot_kbar_analysis(start_date="2025-05-06", end_date=None)

###20250529(CHECK)

In [ ]:
run_simulation("2025-05-28", "2025-05-29")
fig = plot_kbar_analysis(start_date="2025-05-29", end_date=None)

###20250625(V)

In [ ]:
run_simulation("2025-06-23", "2025-06-25")
fig = plot_kbar_analysis(
    kbar_csv_path="kbars.csv",
    trade_csv_path="trade_log.csv",
    key_price_log_path="key_price_log.csv",
    start_date="2025-06-25",
    end_date=None
    )

###20250626

In [ ]:
run_simulation("2025-06-25", "2025-06-26")
fig = plot_kbar_analysis(start_date="2025-06-26", end_date=None)

###20250715

In [ ]:
run_simulation("2025-07-13", "2025-07-15")
fig = plot_kbar_analysis(
    kbar_csv_path="kbars.csv",
    trade_csv_path="trade_log.csv",
    key_price_log_path="key_price_log.csv",
    start_date="2025-07-15",
    end_date=None
    )

###20250821

In [ ]:
run_simulation("2025-08-20", "2025-08-21")
fig = plot_kbar_analysis(
    kbar_csv_path="kbars.csv",
    trade_csv_path="trade_log.csv",
    key_price_log_path="key_price_log.csv",
    start_date="2025-08-21",
    end_date=None
    )

#如何標記翻亞當、頭頭低、抬腳

In [ ]:
import shutil
import os

# 原始檔案在 Colab 本地端
src_files = {
    "kbars.csv": "kbars1025A.csv",   # 重新命名
    "trade_log.csv": "trade_log1025A.csv",
    "key_price_log.csv": "key_price_log1025A.csv"
}

# 複製並重新命名到 Google Drive
for src, new_name in src_files.items():
    src_path = f"/content/{src}"          # Colab 本地端原始檔
    dst_path = os.path.join(drive_path, new_name)  # 目標路徑

    if os.path.exists(src_path):
        shutil.copy(src_path, dst_path)
        print(f"✅ 已複製 {src_path} → {dst_path}")
    else:
        print(f"⚠️ 找不到檔案：{src_path}")

✅ 已複製 /content/kbars.csv → /content/drive/My Drive/Colab Notebooks/DATA/kbars1025A.csv
✅ 已複製 /content/trade_log.csv → /content/drive/My Drive/Colab Notebooks/DATA/trade_log1025A.csv
✅ 已複製 /content/key_price_log.csv → /content/drive/My Drive/Colab Notebooks/DATA/key_price_log1025A.csv
